# Plan for US Patent Phrase to Phrase Matching

Objectives:
- Establish a fast, trustworthy baseline with solid CV.
- Iterate with feature engineering and stronger text models to reach medal threshold.

Workflow:
1) Environment + GPU check; set up constraints for PyTorch/cu121 when needed.
2) Data EDA: inspect train/test, target distribution, CPC breakdown, text lengths.
3) CV protocol: GroupKFold by anchor to avoid leakage; multiple seeds.
4) Baselines:
   - v0: SBERT embeddings (MiniLM) + cosine similarity; simple linear calibration.
   - v1: patent-specific model (e.g., msmarco-distilbert-base or mpnet) with pooling.
5) Fine-tune: Contrastive regression (SimCSE-style) or Siamese on GPU with early stopping.
6) Ensembling: blend diverse backbones/seeds; possibly add TF-IDF ridge for diversity.
7) Error analysis: OOF-driven iterations; adjust CPC-aware calibration if helpful.

Discipline:
- Log per-fold timings and Pearson OOF.
- Cache embeddings and OOF predictions.
- Change one thing at a time; request expert reviews at each milestone.

Next:
- Run GPU sanity check and quick data overview.

In [ ]:
# GPU check and quick data overview
import os, sys, subprocess, time, textwrap, json, random
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 120)

print('=== GPU sanity check ===', flush=True)
try:
    out = subprocess.run(['bash','-lc','nvidia-smi || true'], capture_output=True, text=True, check=False)
    print(out.stdout)
except Exception as e:
    print('nvidia-smi failed:', e)

print('Python:', sys.version)
print('CWD:', os.getcwd())

train_path = 'train.csv'
test_path = 'test.csv'
train = pd.read_csv(train_path)
test = pd.read_csv(test_path)
print('Train shape:', train.shape, 'Test shape:', test.shape)
print('Train columns:', list(train.columns))
print('Test columns:', list(test.columns))

# Basic target and CPC stats
if 'score' in train.columns:
    print('Score describe:')
    print(train['score'].describe())
    print('Score quantiles:', train['score'].quantile([0,0.05,0.25,0.5,0.75,0.95,1]).to_dict())

for col in ['anchor','target','context','id','score']:
    if col in train.columns:
        print(f"train[{col}] non-null: {train[col].notna().sum()}")

if 'anchor' in train.columns:
    print('Unique anchors:', train['anchor'].nunique())
if 'context' in train.columns:
    print('Unique CPC contexts:', train['context'].nunique())

print('\nTrain head:')
print(train.head(3))
print('\nTest head:')
print(test.head(3))

# Quick length stats
for col in ['anchor','target']:
    if col in train.columns:
        lens = train[col].astype(str).str.len()
        print(f'{col} length mean={lens.mean():.1f} p95={lens.quantile(0.95):.0f} max={lens.max()}')

print('\nChecklist next:')
print('- Build GroupKFold by anchor (5 folds) and cache folds')
print('- Map CPC code to title text and decide input template')
print('- Set up fast baseline with mpnet/e5 embeddings + cosine + per-fold calibration')
print('- Prepare TF-IDF ridge for diversity, then cross-encoder fine-tune')

In [ ]:
# Build and cache 5-fold GroupKFold (optionally stratified) by anchor; deduplicate rows
import pandas as pd, numpy as np, os, time
from sklearn.model_selection import GroupKFold
try:
    from sklearn.model_selection import StratifiedGroupKFold
    HAS_SGKF = True
except Exception:
    HAS_SGKF = False

t0 = time.time()
train = pd.read_csv('train.csv')

# Deduplicate by (anchor, target, context): average score
dup_cols = ['anchor','target','context']
before = len(train)
agg = (train.groupby(dup_cols, as_index=False)
              .agg(score=('score','mean'),
                   n=('id','count')))
dedup = agg.copy()
after = len(dedup)
print(f'Dedup: {before} -> {after} rows; {before-after} merged (via mean score).')

# Prepare stratification labels
def bin_score(s, bins=(0.0, 0.125, 0.375, 0.625, 0.875, 1.01)):
    return np.digitize(s, bins, right=True)
y_bins = bin_score(dedup['score'].values)
dedup['cpc_section'] = dedup['context'].astype(str).str[0].fillna('X')
strat_label = (pd.Series(y_bins, index=dedup.index).astype(str) + '_' + dedup['cpc_section']).values

# Groups by anchor
groups = dedup['anchor'].values

NFOLDS = 5
dedup['fold'] = -1

if HAS_SGKF:
    print('Using StratifiedGroupKFold by anchor with strat on bins+CPC')
    sgkf = StratifiedGroupKFold(n_splits=NFOLDS, shuffle=True, random_state=42)
    for fold, (_, val_idx) in enumerate(sgkf.split(dedup, strat_label, groups=groups)):
        dedup.loc[val_idx, 'fold'] = fold
else:
    print('StratifiedGroupKFold not available; falling back to GroupKFold by anchor')
    gkf = GroupKFold(n_splits=NFOLDS)
    for fold, (_, val_idx) in enumerate(gkf.split(dedup, groups=groups)):
        dedup.loc[val_idx, 'fold'] = fold

assert (dedup['fold']>=0).all(), 'Fold assignment failed'

# Save cached folds and deduped train
dedup.to_csv('train_folds.csv', index=False)
print('Saved train_folds.csv with shape', dedup.shape)
dedup[['anchor','target','context','score','fold']].head(3).to_csv('train_folds_head.csv', index=False)
print('Fold distribution by CPC section:')
print(dedup.groupby(['fold', 'cpc_section']).size().unstack(fill_value=0).head())
print('Score bins per fold:')
print(pd.crosstab(dedup['fold'], y_bins))
print(f'Done in {time.time()-t0:.2f}s')

# Note: Downstream models will merge back on (anchor,target,context) to align folds.

In [ ]:
# Install cu121 torch stack and core NLP deps; sanity check GPU in torch
import sys, subprocess, os, shutil, time
from pathlib import Path

def pip(*args):
    print('> pip', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)

# Uninstall any preexisting torch stack to avoid conflicts
for pkg in ('torch','torchvision','torchaudio'):
    subprocess.run([sys.executable, '-m', 'pip', 'uninstall', '-y', pkg], check=False)

# Clean stray site dirs that can shadow correct wheels (idempotent)
for d in (
    '/app/.pip-target/torch',
    '/app/.pip-target/torch-2.8.0.dist-info',
    '/app/.pip-target/torch-2.4.1.dist-info',
    '/app/.pip-target/torchvision',
    '/app/.pip-target/torchvision-0.23.0.dist-info',
    '/app/.pip-target/torchvision-0.19.1.dist-info',
    '/app/.pip-target/torchaudio',
    '/app/.pip-target/torchaudio-2.8.0.dist-info',
    '/app/.pip-target/torchaudio-2.4.1.dist-info',
    '/app/.pip-target/torchgen',
    '/app/.pip-target/functorch',
):
    if os.path.exists(d):
        print('Removing', d)
        shutil.rmtree(d, ignore_errors=True)

# Install EXACT cu121 stack
pip('install',
    '--index-url', 'https://download.pytorch.org/whl/cu121',
    '--extra-index-url', 'https://pypi.org/simple',
    'torch==2.4.1', 'torchvision==0.19.1', 'torchaudio==2.4.1')

# Constraints to freeze versions
Path('constraints.txt').write_text('torch==2.4.1\ntorchvision==0.19.1\ntorchaudio==2.4.1\n')

# Install transformers/accelerate/datasets + sentencepiece; avoid upgrading torch
pip('install', '-c', 'constraints.txt',
    'transformers==4.44.2', 'accelerate==0.34.2',
    'datasets==2.21.0', 'evaluate==0.4.2', 'sentencepiece', 'scikit-learn',
    '--upgrade-strategy', 'only-if-needed')

import torch
print('torch:', torch.__version__, 'built CUDA:', getattr(torch.version, 'cuda', None))
print('CUDA available:', torch.cuda.is_available())
assert str(getattr(torch.version,'cuda','')).startswith('12.1'), f'Wrong CUDA build: {torch.version.cuda}'
assert torch.cuda.is_available(), 'CUDA not available in torch'
print('GPU:', torch.cuda.get_device_name(0))

In [2]:
# Try to fix CUDA visibility for torch by setting library paths, then sanity check
import os, sys, ctypes, shutil

# Ensure NVIDIA libs are on the path
cuda_paths = [
    '/usr/local/nvidia/lib64',
    '/usr/local/cuda/lib64',
    '/usr/local/cuda-12.1/lib64',
    '/usr/lib/x86_64-linux-gnu',
]
ld = os.environ.get('LD_LIBRARY_PATH', '')
parts = [p for p in ld.split(':') if p]
for p in cuda_paths:
    if os.path.exists(p) and p not in parts:
        parts.append(p)
os.environ['LD_LIBRARY_PATH'] = ':'.join(parts)
os.environ.setdefault('CUDA_HOME', '/usr/local/cuda')

print('LD_LIBRARY_PATH=', os.environ['LD_LIBRARY_PATH'])
print('CUDA_HOME=', os.environ.get('CUDA_HOME'))
print('nvidia libs present:', os.path.exists('/usr/local/nvidia/lib64'))

# Try to preload libcuda
for lib in ('libcuda.so.1','libcuda.so'):
    try:
        ctypes.CDLL(lib)
        print('Loaded', lib)
        break
    except Exception as e:
        print('Could not load', lib, e)

import torch
print('torch:', torch.__version__, 'built CUDA:', getattr(torch.version, 'cuda', None))
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    print('GPU:', torch.cuda.get_device_name(0))
    print('Device count:', torch.cuda.device_count())
else:
    print('Env CUDA_VISIBLE_DEVICES=', os.environ.get('CUDA_VISIBLE_DEVICES'))
    # Print a quick diagnostic of available libcuda files
    for path in cuda_paths:
        if os.path.isdir(path):
            try:
                files = [f for f in os.listdir(path) if 'cuda' in f.lower()]
                print(path, 'contains:', files[:10])
            except Exception:
                pass

LD_LIBRARY_PATH= /usr/local/nvidia/lib64:/usr/local/cuda/lib64:/usr/local/cuda-12.1/lib64:/usr/lib/x86_64-linux-gnu
CUDA_HOME= /usr/local/cuda
nvidia libs present: True
Loaded libcuda.so.1
torch: 2.4.1+cu121 built CUDA: 12.1
CUDA available: False
Env CUDA_VISIBLE_DEVICES= None
/usr/local/nvidia/lib64 contains: []
/usr/local/cuda/lib64 contains: ['libcudart.so.12.1.105', 'libcudart.so.12', 'libcudart.so', 'libcudart_static.a', 'libcudadevrt.a']
/usr/local/cuda-12.1/lib64 contains: ['libcudart.so.12.1.105', 'libcudart.so.12', 'libcudart.so', 'libcudart_static.a', 'libcudadevrt.a']
/usr/lib/x86_64-linux-gnu contains: ['libcudadebugger.so.1', 'libcuda.so.1', 'libcuda.so.550.144.06', 'libcudadebugger.so.550.144.06', 'libcuda.so']


In [3]:
# Embedding baseline (CPU for now): all-mpnet-base-v2 with CPC section, cosine + per-fold linear calibration
import time, math, gc, os
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from transformers import AutoTokenizer, AutoModel
import torch
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr

SEED = 42
torch.manual_seed(SEED); np.random.seed(SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

# CPC section short titles
CPC_SEC = {
    'A': 'Human necessities',
    'B': 'Operations/Transport',
    'C': 'Chemistry/Metallurgy',
    'D': 'Textiles/Paper',
    'E': 'Constructions',
    'F': 'Mechanical/Heating',
    'G': 'Physics',
    'H': 'Electricity',
}

def cpc_prefix(code: str) -> str:
    if not isinstance(code, str) or len(code) == 0:
        return 'CPC:UNK'
    sec = code[0]
    title = CPC_SEC.get(sec, 'Unknown')
    return f'CPC:{code} ({title}) | '

def build_texts(df: pd.DataFrame):
    # Prepend CPC short section info to each text
    cpc = df['context'].astype(str).fillna('')
    anchors = (cpc.map(cpc_prefix) + df['anchor'].astype(str)).values
    targets = (cpc.map(cpc_prefix) + df['target'].astype(str)).values
    return anchors, targets

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0]  # (bs, seq, hid)
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, dim=1)
    sum_mask = input_mask_expanded.sum(dim=1).clamp(min=1e-9)
    return sum_embeddings / sum_mask

def encode_texts(model_name, texts, batch_size=128, max_length=128):
    tok = AutoTokenizer.from_pretrained(model_name)
    mdl = AutoModel.from_pretrained(model_name)
    mdl.to(device)
    mdl.eval()
    embs = np.zeros((len(texts), mdl.config.hidden_size), dtype=np.float32)
    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size), desc=f'Encoding {model_name}'):
            batch = texts[i:i+batch_size]
            enc = tok(list(batch), padding=True, truncation=True, max_length=max_length, return_tensors='pt')
            enc = {k: v.to(device) for k, v in enc.items()}
            out = mdl(**enc)
            pooled = mean_pooling(out, enc['attention_mask'])
            # L2-normalize
            pooled = torch.nn.functional.normalize(pooled, p=2, dim=1)
            embs[i:i+batch_size] = pooled.detach().cpu().numpy()
    del mdl
    gc.collect()
    return embs

def cosine_sim(a, b):
    # a: (n,d) L2-normalized, b: (n,d) L2-normalized
    return np.sum(a * b, axis=1)

t_start = time.time()
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
folds = pd.read_csv('train_folds.csv')

# Align folds with train by (anchor,target,context)
key_cols = ['anchor','target','context']
assert set(key_cols).issubset(train.columns), 'Missing key cols in train'
train = train.merge(folds[key_cols+['fold']], on=key_cols, how='left')
assert (train['fold']>=0).all(), 'Fold merge failed'

# Build CPC-aware texts
tr_a, tr_b = build_texts(train)
te_a, te_b = build_texts(test)

# Deduplicate texts for efficient encoding
uniq_a, inv_a = np.unique(tr_a, return_inverse=True)
uniq_b, inv_b = np.unique(tr_b, return_inverse=True)
uniq_te_a, inv_te_a = np.unique(te_a, return_inverse=True)
uniq_te_b, inv_te_b = np.unique(te_b, return_inverse=True)
print('Unique train anchors:', len(uniq_a), 'Unique train targets:', len(uniq_b))
print('Unique test anchors:', len(uniq_te_a), 'Unique test targets:', len(uniq_te_b))

MODEL = 'sentence-transformers/all-mpnet-base-v2'
BATCH = 128 if device.type == 'cuda' else 64
MAX_LEN = 128

ua_emb = encode_texts(MODEL, uniq_a, batch_size=BATCH, max_length=MAX_LEN)
ub_emb = encode_texts(MODEL, uniq_b, batch_size=BATCH, max_length=MAX_LEN)
utea_emb = encode_texts(MODEL, uniq_te_a, batch_size=BATCH, max_length=MAX_LEN)
uteb_emb = encode_texts(MODEL, uniq_te_b, batch_size=BATCH, max_length=MAX_LEN)

# Map back
tr_a_emb = ua_emb[inv_a]
tr_b_emb = ub_emb[inv_b]
te_a_emb = utea_emb[inv_te_a]
te_b_emb = uteb_emb[inv_te_b]

# Cosine similarities
train_cos = cosine_sim(tr_a_emb, tr_b_emb)
test_cos = cosine_sim(te_a_emb, te_b_emb)
print('Cosine range train:', float(train_cos.min()), float(train_cos.max()))
print('Cosine range test:', float(test_cos.min()), float(test_cos.max()))

# Per-fold linear calibration (with intercept) on cosine -> score
oof = np.zeros(len(train), dtype=np.float32)
test_preds_folds = []
from collections import namedtuple
Calib = namedtuple('Calib', ['a','b'])
calibs = []
for f in sorted(train['fold'].unique()):
    f0 = time.time()
    tr_idx = np.where(train['fold'] != f)[0]
    va_idx = np.where(train['fold'] == f)[0]
    lr = LinearRegression()
    lr.fit(train_cos[tr_idx].reshape(-1,1), train.loc[tr_idx, 'score'].values)
    oof[va_idx] = lr.predict(train_cos[va_idx].reshape(-1,1)).astype(np.float32)
    test_preds_folds.append(lr.predict(test_cos.reshape(-1,1)).astype(np.float32))
    a = float(lr.coef_[0]); b = float(lr.intercept_); calibs.append(Calib(a,b))
    fold_pear = pearsonr(oof[va_idx], train.loc[va_idx, 'score'].values)[0]
    print(f'Fold {f}: pearson={fold_pear:.6f} (elapsed {time.time()-f0:.1f}s)')

oof_clip = np.clip(oof, 0.0, 1.0)
oof_pear = pearsonr(oof_clip, train['score'].values)[0]
print('OOF Pearson (clipped [0,1]):', round(oof_pear, 6))

# Test predictions: average calibrated per-fold predictions, then clip
test_stack = np.vstack(test_preds_folds)
test_pred = np.clip(test_stack.mean(axis=0), 0.0, 1.0).astype(np.float32)

# Save artifacts
np.save('oof_mpnet_cos.npy', oof_clip)
np.save('test_mpnet_cos.npy', test_pred)
pd.DataFrame({'id': train['id'], 'oof': oof_clip}).to_csv('oof_mpnet.csv', index=False)
sub = pd.DataFrame({'id': test['id'], 'score': test_pred})
sub.to_csv('submission.csv', index=False)
print('Saved submission.csv; shape', sub.shape)
print('Total elapsed:', round((time.time()-t_start)/60, 2), 'min')

/app/.pip-target/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu
Unique train anchors: 1684 Unique train targets: 32310
Unique test anchors: 1307 Unique test targets: 3642


/app/.pip-target/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Encoding sentence-transformers/all-mpnet-base-v2:   0%|          | 0/27 [00:00<?, ?it/s]

Encoding sentence-transformers/all-mpnet-base-v2:   4%|▎         | 1/27 [00:00<00:08,  3.22it/s]

...

Encoding sentence-transformers/all-mpnet-base-v2:  96%|█████████▋| 55/57 [00:09<00:00,  6.18it/s]

Encoding sentence-transformers/all-mpnet-base-v2:  98%|█████████▊| 56/57 [00:10<00:00,  6.13it/s]

Encoding sentence-transformers/all-mpnet-base-v2: 100%|██████████| 57/57 [00:10<00:00,  6.26it/s]

Encoding sentence-transformers/all-mpnet-base-v2: 100%|██████████| 57/57 [00:10<00:00,  5.58it/s]

Cosine range train: 0.209250807762146 1.000000238418579
Cosine range test: 0.293619841337204 1.0000001192092896
Fold 0: pearson=0.552162 (elapsed 0.0s)
Fold 1: pearson=0.537914 (elapsed 0.0s)
Fold 2: pearson=0.531651 (elapsed 0.0s)
Fold 3: pearson=0.546173 (elapsed 0.0s)
Fold 4: pearson=0.539455 (elapsed 0.0s)
OOF Pearson (clipped [0,1]): 0.542784
Saved submission.csv; shape (3648, 2)
Total elapsed: 1.94 min


In [29]:
# TF-IDF pairwise cosine baseline (CPU): build separate A/B vectors, cosine as 1D feature, per-fold calibration
import time, numpy as np, pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Ridge, LinearRegression
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr
from scipy.sparse import hstack

t0 = time.time()
SEED = 42
np.random.seed(SEED)

def cpc_sec(code: str) -> str:
    return (code if isinstance(code, str) and len(code)>0 else 'UNK')[0]
def cpc_tag(code: str) -> str:
    return f" [CPC:{cpc_sec(code)}]"

def build_pair_texts(df: pd.DataFrame, use_cpc: bool = False):
    suf = df['context'].astype(str).map(lambda x: cpc_tag(x) if use_cpc else '')
    A = (df['anchor'].astype(str) + suf).values
    B = (df['target'].astype(str) + suf).values
    return A, B

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Merge folds_by_id (Fix A)
folds = pd.read_csv('folds_by_id.csv')
train = train.merge(folds, on='id', how='left', validate='one_to_one')
assert (train['fold']>=0).all(), 'Fold merge by id failed'

# V0: no CPC suffix (avoid identical text inflation). Later try use_cpc=True as ablation.
use_cpc = False
A_tr, B_tr = build_pair_texts(train, use_cpc=use_cpc)
A_te, B_te = build_pair_texts(test, use_cpc=use_cpc)
y = train['score'].values.astype(np.float32)

# Fit TF-IDF on combined corpus (anchors and targets, train+test) once
corpus = np.concatenate([A_tr, B_tr, A_te, B_te])
print('Fitting TF-IDF (char 3-5)...', flush=True)
vec_char = TfidfVectorizer(analyzer='char', ngram_range=(3,5), min_df=2, max_features=400000)
Xc = vec_char.fit_transform(corpus)
print('Fitting TF-IDF (word 1-2)...', flush=True)
vec_word = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df=2, max_features=400000)
Xw = vec_word.fit_transform(corpus)

nA_tr, nB_tr, nA_te, nB_te = len(A_tr), len(B_tr), len(A_te), len(B_te)
ix = 0
Xc_A_tr = Xc[ix:ix+nA_tr]; ix += nA_tr
Xc_B_tr = Xc[ix:ix+nB_tr]; ix += nB_tr
Xc_A_te = Xc[ix:ix+nA_te]; ix += nA_te
Xc_B_te = Xc[ix:ix+nB_te]; ix += nB_te

iw = 0
Xw_A_tr = Xw[iw:iw+nA_tr]; iw += nA_tr
Xw_B_tr = Xw[iw:iw+nB_tr]; iw += nB_tr
Xw_A_te = Xw[iw:iw+nA_te]; iw += nA_te
Xw_B_te = Xw[iw:iw+nB_te]; iw += nB_te

# Combine char+word spaces for A and B, then compute cosine diagonals
XA_tr = hstack([Xc_A_tr, Xw_A_tr]).tocsr()
XB_tr = hstack([Xc_B_tr, Xw_B_tr]).tocsr()
XA_te = hstack([Xc_A_te, Xw_A_te]).tocsr()
XB_te = hstack([Xc_B_te, Xw_B_te]).tocsr()
print('Shapes A/B:', XA_tr.shape, XB_tr.shape, XA_te.shape, XB_te.shape)

def rowwise_cosine(XA, XB):
    # Efficient block-wise diag of cosine_similarity
    n = XA.shape[0]
    out = np.empty(n, dtype=np.float32)
    bs = 4096
    for i in range(0, n, bs):
        j = min(i+bs, n)
        C = cosine_similarity(XA[i:j], XB[i:j])
        out[i:j] = np.diag(C).astype(np.float32)
    return out

cos_tr = rowwise_cosine(XA_tr, XB_tr)
cos_te = rowwise_cosine(XA_te, XB_te)
print('Cosine stats train:', float(cos_tr.min()), float(cos_tr.max()), 'mean', float(cos_tr.mean()))

# Per-fold calibration: isotonic vs linear on 1D cosine
oof_iso = np.zeros_like(y)
oof_lin = np.zeros_like(y)
test_iso_f, test_lin_f = [], []
for f in sorted(train['fold'].unique()):
    tr_idx = np.where(train['fold'] != f)[0]
    va_idx = np.where(train['fold'] == f)[0]
    raw_p = pearsonr(cos_tr[va_idx], y[va_idx])[0]
    print(f'Fold {f}: raw-cos pearson={raw_p:.6f}')
    iso = IsotonicRegression(out_of_bounds='clip')
    iso.fit(cos_tr[tr_idx], y[tr_idx])
    oof_iso[va_idx] = iso.predict(cos_tr[va_idx]).astype(np.float32)
    test_iso_f.append(iso.predict(cos_te).astype(np.float32))
    lr = LinearRegression()
    lr.fit(cos_tr[tr_idx].reshape(-1,1), y[tr_idx])
    oof_lin[va_idx] = lr.predict(cos_tr[va_idx].reshape(-1,1)).astype(np.float32)
    test_lin_f.append(lr.predict(cos_te.reshape(-1,1)).astype(np.float32))
    print(f'Fold {f}: iso pearson={pearsonr(oof_iso[va_idx], y[va_idx])[0]:.6f} | lin pearson={pearsonr(oof_lin[va_idx], y[va_idx])[0]:.6f}', flush=True)

# IMPORTANT: Do NOT clip OOF before Pearson
pear_iso_raw = pearsonr(oof_iso, y)[0]
pear_lin_raw = pearsonr(oof_lin, y)[0]
use_iso = pear_iso_raw >= pear_lin_raw
print('OOF Pearson TF-IDF RAW (iso):', round(pear_iso_raw,6), '| RAW (lin):', round(pear_lin_raw,6), '| using', 'iso' if use_iso else 'lin')

if use_iso:
    test_pred = np.clip(np.mean(np.vstack(test_iso_f), axis=0), 0.0, 1.0).astype(np.float32)
    oof_save = oof_iso  # raw
else:
    test_pred = np.clip(np.mean(np.vstack(test_lin_f), axis=0), 0.0, 1.0).astype(np.float32)
    oof_save = oof_lin  # raw

pd.DataFrame({'id': train['id'], 'oof': oof_save}).to_csv('oof_tfidf_ridge.csv', index=False)
pd.DataFrame({'id': test['id'], 'score': test_pred}).to_csv('submission_tfidf.csv', index=False)
print('Saved submission_tfidf.csv')
print('Total elapsed:', round((time.time()-t0)/60, 2), 'min')

Fitting TF-IDF (char 3-5)...


Fitting TF-IDF (word 1-2)...


Shapes A/B: (32825, 58790) (32825, 58790) (3648, 58790) (3648, 58790)


Cosine stats train: 0.0 1.0 mean 0.21684813499450684
Fold 0: raw-cos pearson=0.449665
Fold 0: iso pearson=0.463125 | lin pearson=0.449665


Fold 1: raw-cos pearson=0.426334
Fold 1: iso pearson=0.443953 | lin pearson=0.426334


Fold 2: raw-cos pearson=0.428118
Fold 2: iso pearson=0.451013 | lin pearson=0.428118


Fold 3: raw-cos pearson=0.442905
Fold 3: iso pearson=0.459122 | lin pearson=0.442905


Fold 4: raw-cos pearson=0.429036
Fold 4: iso pearson=0.444920 | lin pearson=0.429036


OOF Pearson TF-IDF RAW (iso): 0.452018 | RAW (lin): 0.434731 | using iso
Saved submission_tfidf.csv
Total elapsed: 0.04 min


In [28]:
# all-mpnet-base-v2 (CPU): symmetric CPC-first (5 words) + per-fold whitening (remove top PCs), RAW OOF, per-fold isotonic vs linear
import time, os, gc, numpy as np, pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.isotonic import IsotonicRegression
from sklearn.decomposition import PCA
from scipy.stats import pearsonr
import subprocess, sys

def pip(*args):
    print('> pip', *args, flush=True)
    subprocess.run([sys.executable, '-m', 'pip', *args], check=True)
try:
    import sentence_transformers  # noqa
except Exception:
    pip('install', 'sentence-transformers==3.0.1', '--no-deps')
from sentence_transformers import SentenceTransformer

SEED = 42
np.random.seed(SEED)

# CPC mapping (comprehensive, aligned with e5 cell)
CPC_DESCRIPTIONS = {
    'A61B':'Diagnosis; Surgery; Identification','A61F':'Filters; Prostheses; Braces','A61K':'Preparations for medical, dental, or toilet purposes','A61M':'Devices for introducing/circulating media into the body',
    'B01D':'Separation; Sorption; Membranes','B29C':'Shaping or joining of plastics','B32B':'Layered products','B60K':'Arrangement or mounting of vehicle power plant or auxiliaries','B60R':'Vehicles, vehicle fittings, or vehicle parts','B60T':'Vehicle brake control systems or parts',
    'C01B':'Non-metallic elements; Compounds thereof','C04B':'Cement; Concrete; Ceramics','C07C':'Acyclic or carbocyclic compounds','C07D':'Heterocyclic compounds','C07K':'Peptides','C08F':'Macromolecular compounds by polymerising monomers','C08L':'Compositions of macromolecular compounds','C09K':'Materials for miscellaneous applications',
    'C12N':'Microorganisms or enzymes; Genetic engineering','C12P':'Fermentation or enzyme-using processes','C12Q':'Measuring/testing involving enzymes or microorganisms',
    'E04B':'Building constructions; Walls; Floors; Roofs','F16B':'Screws; Rivets; Joints','F16F':'Springs; Shock-absorbers; Dampers','F16L':'Pipes; Joints or fittings for pipes',
    'G01N':'Investigating or analysing materials','G02B':'Optical elements, systems, or apparatus','G02F':'Control of light','G06F':'Electric digital data processing','G06K':'Recognition/Presentation of data','G06Q':'Data processing for administrative/financial/managerial',
    'H01L':'Semiconductor devices','H01M':'Batteries; Fuel cells','H02J':'Power supply/distribution','H04L':'Transmission of digital information','H04N':'Pictorial communication','H04W':'Wireless communication networks','H05K':'Printed circuits; Electric assemblies',
    'A01':'Agriculture; Forestry; Animal husbandry; Hunting; Trapping; Fishing','A21':'Baking; Edible doughs','A23':'Foods or foodstuffs; Treatment thereof','A24':'Tobacco; Cigars; Cigarettes; Simulated smoking devices; Smokers\' requisites','A41':'Wearing apparel','A42':'Headwear','A43':'Footwear','A45':'Hand or travelling articles','A46':'Brushware','A47':'Furniture; Domestic articles or appliances; Coffee mills; Spice mills; Suction cleaners','A61':'Medical or veterinary science; Hygiene','A62':'Life-saving; Fire-fighting','A63':'Sports; Games; Amusements',
    'B01':'Physical or chemical processes or apparatus','B02':'Crushing; pulverising; disintegrating','B03':'Separation of solid materials using liquids or jigs','B04':'Centrifugal apparatus or machines','B05':'Spraying or atomising; Applying fluent materials to surfaces','B06':'Mechanical vibrations','B07':'Separating solids; Sorting','B08':'Cleaning','B09':'Disposal of solid waste; Soil reclamation','B21':'Mechanical metal-working without removing material','B22':'Casting; Powder metallurgy','B23':'Machine tools; Metal-working n.e.c.','B24':'Grinding; Polishing','B25':'Hand tools; Portable power-driven tools; Manipulators','B26':'Hand cutting tools; Cutting; Severing','B27':'Working/preserving wood; Nailing or stapling','B28':'Working cement, clay, or stone','B29':'Working of plastics; Plastic state substances','B30':'Presses','B31':'Making paper-like articles','B32':'Layered products','B33':'Additive manufacturing','B41':'Printing; Typewriters; Stamps','B42':'Bookbinding; Albums; Files','B43':'Writing or drawing implements; Bureau accessories','B44':'Decorative arts','B60':'Vehicles in general','B61':'Railways','B62':'Land vehicles not on rails','B63':'Ships or waterborne vessels','B64':'Aircraft; Aviation; Cosmonautics','B65':'Conveying; Packing; Storing','B66':'Hoisting; Lifting; Hauling','B67':'Opening/closing containers; Liquid handling','B68':'Saddlery; Upholstery','B81':'Microstructural technology','B82':'Nanotechnology',
    'C01':'Inorganic chemistry','C02':'Treatment of water, wastewater, sewage, sludge','C03':'Glass; Mineral or slag wool','C04':'Cements; Concrete; Ceramics; Refractories','C05':'Fertilisers','C06':'Explosives; Matches','C07':'Organic chemistry','C08':'Organic macromolecular compounds; Polymers','C09':'Dyes; Paints; Polishes; Adhesives; Misc compositions','C10':'Petroleum/gas/coke industries; Fuels; Lubricants; Peat','C11':'Oils, fats, waxes; Detergents; Candles','C12':'Biochemistry; Microbiology; Enzymology; Fermentation','C13':'Sugar industry','C14':'Skins; Hides; Leather','C21':'Metallurgy of iron','C22':'Metallurgy; Alloys; Treatment of non-ferrous metals','C23':'Coating metallic material; Chemical surface treatment','C25':'Electrolytic/electrophoretic processes','C30':'Crystal growth','C40':'Combinatorial technology',
    'D01':'Threads or fibres; Spinning','D02':'Yarns; Finishing; Warping or beaming','D03':'Weaving','D04':'Braiding; Lace-making; Knitting; Non-wovens','D05':'Sewing; Embroidering; Tufting','D06':'Textile treatment; Laundering; Flexible materials','D07':'Ropes; Cables other than electric','D21':'Paper-making; Cellulose production',
    'E01':'Road/rail/bridge construction','E02':'Hydraulic engineering; Foundations; Soil shifting','E03':'Water supply; Sewerage','E04':'Building','E05':'Locks; Keys; Fittings; Safes','E06':'Doors; Windows; Shutters; Ladders','E21':'Earth/rock drilling; Mining',
    'F01':'Machines or engines in general','F02':'Combustion engines; Hot-gas plants','F03':'Machines or engines for liquids; Wind/spring motors','F04':'Positive-displacement machines; Pumps','F15':'Fluid-pressure actuators; Hydraulics; Pneumatics','F16':'Engineering elements or units','F17':'Storing/distributing gases or liquids','F21':'Lighting','F22':'Steam generation','F23':'Combustion apparatus; Processes','F24':'Heating; Ranges; Ventilating','F25':'Refrigeration or cooling; Heat pumps','F26':'Drying','F27':'Furnaces; Kilns; Ovens','F28':'Heat exchange in general','F41':'Weapons','F42':'Ammunition; Blasting',
    'G01':'Measuring; Testing','G02':'Optics','G03':'Photography; Cinematography; Non-optical waves','G04':'Horology','G05':'Controlling; Regulating','G06':'Computing; Calculating; Counting','G07':'Checking-devices','G08':'Signalling','G09':'Educating; Cryptography; Display; Advertising; Seals','G10':'Musical instruments; Acoustics','G11':'Information storage','G12':'Instrument details','G16':'ICT adapted for specific fields','G21':'Nuclear physics; Nuclear engineering',
    'H01':'Electric elements','H02':'Electric power: generation, conversion, distribution','H03':'Electronic circuitry','H04':'Electric communication technique','H05':'Electric techniques not otherwise provided for'
}
SECTION = {'A':'Human necessities','B':'Performing operations; Transporting','C':'Chemistry; Metallurgy','D':'Textiles; Paper','E':'Fixed constructions','F':'Mechanical engineering; Lighting; Heating; Weapons; Blasting','G':'Physics','H':'Electricity'}

def get_cpc_text(code: str) -> str:
    if not isinstance(code, str) or not code:
        return 'General technology'
    code = code.strip().upper()
    for k in (4,3,2):
        pref = code[:k]
        if pref in CPC_DESCRIPTIONS:
            return CPC_DESCRIPTIONS[pref]
    return SECTION.get(code[0], 'General technology')

def _truncate_words(text: str, n: int = 5) -> str:
    if not text: return ''
    w = text.split()
    return ' '.join(w[:n])

def build_texts_mpnet(df: pd.DataFrame):
    cpc_full = df['context'].astype(str).map(get_cpc_text)
    cpc_5w = cpc_full.map(lambda x: _truncate_words(x, 5))
    a = ("domain: " + cpc_5w + " [SEP] " + df['anchor'].astype(str)).values
    b = ("domain: " + cpc_5w + " [SEP] " + df['target'].astype(str)).values
    return a, b

def l2n(x: np.ndarray, eps: float = 1e-12) -> np.ndarray:
    n = np.linalg.norm(x, axis=1, keepdims=True) + eps
    return x / n

def fit_whitener(X: np.ndarray, k: int = 2):
    mu = X.mean(axis=0, keepdims=True)
    Xc = X - mu
    pca = PCA(n_components=k, random_state=SEED).fit(Xc)
    P = pca.components_
    def transform(Y: np.ndarray) -> np.ndarray:
        Yc = Y - mu
        Z = Yc - (Yc @ P.T) @ P
        return l2n(Z)
    return transform

t0 = time.time()
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Merge folds_by_id (strict id join)
folds = pd.read_csv('folds_by_id.csv')
train = train.merge(folds, on='id', how='left', validate='one_to_one')
assert (train['fold']>=0).all(), 'Fold merge by id failed'

# Symmetric CPC-first (5 words) template
tr_a, tr_b = build_texts_mpnet(train)
te_a, te_b = build_texts_mpnet(test)

# Union-dedup of all texts to minimize CPU encode
all_texts = np.concatenate([tr_a, tr_b, te_a, te_b])
uniq_texts, inv_idx = np.unique(all_texts, return_inverse=True)
n_tr = len(train); n_te = len(test); n_a_tr = len(tr_a); n_b_tr = len(tr_b)
print('mpnet (sym CPC-first, 5w) unique texts to encode:', len(uniq_texts))

MODEL = 'sentence-transformers/all-mpnet-base-v2'
st_model = SentenceTransformer(MODEL, device='cpu')  # CPU-only
st_model.max_seq_length = 256
embs = st_model.encode(uniq_texts.tolist(), batch_size=48, normalize_embeddings=True, convert_to_numpy=True, show_progress_bar=True)
assert embs.shape[0] == len(uniq_texts)

# Map back
idx_tr_a = inv_idx[0:n_a_tr]
idx_tr_b = inv_idx[n_a_tr:n_a_tr+n_b_tr]
idx_te_a = inv_idx[n_a_tr+n_b_tr:n_a_tr+n_b_tr+n_te]
idx_te_b = inv_idx[n_a_tr+n_b_tr+n_te:]
tr_a_emb = embs[idx_tr_a]
tr_b_emb = embs[idx_tr_b]
te_a_emb = embs[idx_te_a]
te_b_emb = embs[idx_te_b]

def cosine_sim_rows(a, b):
    return np.sum(a * b, axis=1).astype(np.float32)

# Diagnostics: pre-whitening cosine stats
train_cos0 = cosine_sim_rows(tr_a_emb, tr_b_emb)
print('mpnet pre-whiten cos mean/std:', float(train_cos0.mean()), float(train_cos0.std()))

# Per-fold whitening + calibration
y = train['score'].values.astype(np.float32)
oof_iso = np.zeros(n_tr, dtype=np.float32)
oof_lin = np.zeros(n_tr, dtype=np.float32)
test_preds_iso_folds, test_preds_lin_folds = [], []
K_PCS = 2
for f in sorted(train['fold'].unique()):
    tr_idx = np.where(train['fold'] != f)[0]
    va_idx = np.where(train['fold'] == f)[0]
    # Fit whitener on train-only embeddings (A||B)
    W = fit_whitener(np.vstack([tr_a_emb[tr_idx], tr_b_emb[tr_idx]]), k=K_PCS)
    wa_tr_a = W(tr_a_emb[tr_idx]); wb_tr_b = W(tr_b_emb[tr_idx])
    wa_va_a = W(tr_a_emb[va_idx]); wb_va_b = W(tr_b_emb[va_idx])
    wa_te_a = W(te_a_emb);        wb_te_b = W(te_b_emb)

    cos_tr = cosine_sim_rows(wa_tr_a, wb_tr_b)
    cos_va = cosine_sim_rows(wa_va_a, wb_va_b)
    cos_te = cosine_sim_rows(wa_te_a, wb_te_b)

    raw_p = pearsonr(cos_va, y[va_idx])[0]
    iso = IsotonicRegression(out_of_bounds='clip')
    iso.fit(cos_tr, y[tr_idx])
    oof_iso[va_idx] = iso.predict(cos_va).astype(np.float32)
    test_preds_iso_folds.append(iso.predict(cos_te).astype(np.float32))

    lr = LinearRegression()
    lr.fit(cos_tr.reshape(-1,1), y[tr_idx])
    oof_lin[va_idx] = lr.predict(cos_va.reshape(-1,1)).astype(np.float32)
    test_preds_lin_folds.append(lr.predict(cos_te.reshape(-1,1)).astype(np.float32))
    print(f'mpnet Fold {f}: raw-whiten-cos={raw_p:.6f} | iso={pearsonr(oof_iso[va_idx], y[va_idx])[0]:.6f} | lin={pearsonr(oof_lin[va_idx], y[va_idx])[0]:.6f}', flush=True)

oof_iso_pear = pearsonr(oof_iso, y)[0]
oof_lin_pear = pearsonr(oof_lin, y)[0]
use_iso = oof_iso_pear >= oof_lin_pear
print('OOF Pearson (mpnet sym CPC + whitening, RAW): iso=', round(oof_iso_pear,6), '| lin=', round(oof_lin_pear,6), '| using', 'iso' if use_iso else 'lin')

if use_iso:
    test_stack = np.vstack(test_preds_iso_folds)
    oof_save = oof_iso
else:
    test_stack = np.vstack(test_preds_lin_folds)
    oof_save = oof_lin
test_pred = np.clip(test_stack.mean(axis=0), 0.0, 1.0).astype(np.float32)

# Save RAW OOF (unclipped) and clipped test preds
pd.DataFrame({'id': train['id'], 'oof': oof_save}).to_csv('oof_mpnet_st.csv', index=False)
pd.DataFrame({'id': test['id'], 'score': test_pred}).to_csv('submission_mpnet_st.csv', index=False)
print('Saved submission_mpnet_st.csv; elapsed', round((time.time()-t0)/60,2), 'min')

del embs, tr_a_emb, tr_b_emb, te_a_emb, te_b_emb
gc.collect()

mpnet (sym CPC-first, 5w) unique texts to encode: 37266


/app/.pip-target/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Batches:   0%|          | 0/777 [00:00<?, ?it/s]

Batches:   0%|          | 1/777 [00:00<04:48,  2.69it/s]

Batches:   0%|          | 2/777 [00:00<03:35,  3.59it/s]

...

mpnet Fold 2: raw-whiten-cos=0.496223 | iso=0.563248 | lin=0.496223


mpnet Fold 3: raw-whiten-cos=0.502849 | iso=0.574043 | lin=0.502849


mpnet Fold 4: raw-whiten-cos=0.506306 | iso=0.570513 | lin=0.506306


OOF Pearson (mpnet sym CPC + whitening, RAW): iso= 0.570447 | lin= 0.501517 | using iso
Saved submission_mpnet_st.csv; elapsed 2.0 min


4174

In [34]:
# e5-base-v2 SentenceTransformer baseline (CPU): query/passage prompts with optional compact CPC suffix, per-fold isotonic vs linear calibration
import time, numpy as np, pandas as pd
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr
from sentence_transformers import SentenceTransformer

SEED = 42
np.random.seed(SEED)

def cpc_sec(code: str) -> str:
    return (code if isinstance(code, str) and len(code)>0 else 'UNK')[0]
def cpc_tag(code: str) -> str:
    return f" [CPC:{cpc_sec(code)}]"

def build_e5_texts(df: pd.DataFrame, use_cpc: bool = False):
    a = df['anchor'].astype(str).tolist()
    b = df['target'].astype(str).tolist()
    c = df['context'].astype(str).tolist()
    if use_cpc:
        anchors = np.array(['query: ' + x + cpc_tag(cx) for x, cx in zip(a, c)])
        targets = np.array(['passage: ' + x + cpc_tag(cx) for x, cx in zip(b, c)])
    else:
        anchors = np.array(['query: ' + x for x in a])
        targets = np.array(['passage: ' + x for x in b])
    return anchors, targets

t0 = time.time()
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Use folds_by_id (Fix A): merge by id only
folds = pd.read_csv('folds_by_id.csv')
assert {'id','fold'}.issubset(folds.columns)
train = train.merge(folds, on='id', how='left', validate='one_to_one')
assert (train['fold']>=0).all(), 'Fold merge by id failed'

# Toggle: set use_cpc=True for V1 ablation with compact CPC suffix
use_cpc = True
tr_a, tr_b = build_e5_texts(train, use_cpc=use_cpc)
te_a, te_b = build_e5_texts(test, use_cpc=use_cpc)

# Union dedup encode
all_texts = np.concatenate([tr_a, tr_b, te_a, te_b])
uniq_texts, inv_idx = np.unique(all_texts, return_inverse=True)
n_tr = len(train); n_te = len(test)
n_a_tr = len(tr_a); n_b_tr = len(tr_b)
print('e5 unique texts to encode:', len(uniq_texts))

MODEL = 'intfloat/e5-base-v2'
st_model = SentenceTransformer(MODEL, device='cpu')
st_model.max_seq_length = 128
embs = st_model.encode(uniq_texts.tolist(), batch_size=32, normalize_embeddings=True, convert_to_numpy=True, show_progress_bar=True)
assert embs.shape[0] == len(uniq_texts)

idx_tr_a = inv_idx[0:n_a_tr]
idx_tr_b = inv_idx[n_a_tr:n_a_tr+n_b_tr]
idx_te_a = inv_idx[n_a_tr+n_b_tr:n_a_tr+n_b_tr+n_te]
idx_te_b = inv_idx[n_a_tr+n_b_tr+n_te:]
tr_a_emb = embs[idx_tr_a]
tr_b_emb = embs[idx_tr_b]
te_a_emb = embs[idx_te_a]
te_b_emb = embs[idx_te_b]

def cosine_sim_rows(a, b):
    return np.sum(a * b, axis=1)
train_cos = cosine_sim_rows(tr_a_emb, tr_b_emb).astype(np.float32)
test_cos = cosine_sim_rows(te_a_emb, te_b_emb).astype(np.float32)
print('e5 cosine stats train: min', float(train_cos.min()), 'max', float(train_cos.max()), 'mean', float(train_cos.mean()))

y = train['score'].values.astype(np.float32)
oof_iso = np.zeros(n_tr, dtype=np.float32)
oof_lin = np.zeros(n_tr, dtype=np.float32)
test_preds_iso_folds = []
test_preds_lin_folds = []
for f in sorted(train['fold'].unique()):
    tr_idx = np.where(train['fold'] != f)[0]
    va_idx = np.where(train['fold'] == f)[0]
    raw_p = pearsonr(train_cos[va_idx], y[va_idx])[0]
    print(f'e5 Fold {f}: raw-cos pearson={raw_p:.6f}')
    # Isotonic calibration per fold
    iso = IsotonicRegression(out_of_bounds='clip')
    iso.fit(train_cos[tr_idx], y[tr_idx])
    oof_iso[va_idx] = iso.predict(train_cos[va_idx]).astype(np.float32)
    test_preds_iso_folds.append(iso.predict(test_cos).astype(np.float32))
    # Linear calibration per fold
    lr = LinearRegression()
    lr.fit(train_cos[tr_idx].reshape(-1,1), y[tr_idx])
    oof_lin[va_idx] = lr.predict(train_cos[va_idx].reshape(-1,1)).astype(np.float32)
    test_preds_lin_folds.append(lr.predict(test_cos.reshape(-1,1)).astype(np.float32))
    print(f'e5 Fold {f}: iso pearson={pearsonr(oof_iso[va_idx], y[va_idx])[0]:.6f} | lin pearson={pearsonr(oof_lin[va_idx], y[va_idx])[0]:.6f}', flush=True)

oof_iso_c = np.clip(oof_iso, 0.0, 1.0)
oof_lin_c = np.clip(oof_lin, 0.0, 1.0)
oof_iso_pear = pearsonr(oof_iso_c, y)[0]
oof_lin_pear = pearsonr(oof_lin_c, y)[0]
use_iso = oof_iso_pear >= oof_lin_pear
print('OOF Pearson e5 (iso):', round(oof_iso_pear, 6), '| (lin):', round(oof_lin_pear, 6), '| using', 'iso' if use_iso else 'lin', '| use_cpc=', use_cpc)

if use_iso:
    test_stack = np.vstack(test_preds_iso_folds)
    test_pred = np.clip(test_stack.mean(axis=0), 0.0, 1.0).astype(np.float32)
    pd.DataFrame({'id': train['id'], 'oof': oof_iso_c}).to_csv('oof_e5_base.csv', index=False)
else:
    test_stack = np.vstack(test_preds_lin_folds)
    test_pred = np.clip(test_stack.mean(axis=0), 0.0, 1.0).astype(np.float32)
    pd.DataFrame({'id': train['id'], 'oof': oof_lin_c}).to_csv('oof_e5_base.csv', index=False)

pd.DataFrame({'id': test['id'], 'score': test_pred}).to_csv('submission_e5_base.csv', index=False)
print('Saved submission_e5_base.csv; elapsed', round((time.time()-t0)/60,2), 'min')

e5 unique texts to encode: 34619


Batches:   0%|          | 0/1082 [00:00<?, ?it/s]

Batches:   0%|          | 1/1082 [00:00<04:37,  3.90it/s]

Batches:   0%|          | 2/1082 [00:00<03:34,  5.03it/s]

Batches:   0%|          | 3/1082 [00:00<03:18,  5.43it/s]

...

e5 Fold 1: raw-cos pearson=0.575352
e5 Fold 1: iso pearson=0.620792 | lin pearson=0.575352


e5 Fold 2: raw-cos pearson=0.578126
e5 Fold 2: iso pearson=0.622547 | lin pearson=0.578126


e5 Fold 3: raw-cos pearson=0.589342
e5 Fold 3: iso pearson=0.626598 | lin pearson=0.589342


e5 Fold 4: raw-cos pearson=0.566578
e5 Fold 4: iso pearson=0.610806 | lin pearson=0.566578


OOF Pearson e5 (iso): 0.622448 | (lin): 0.577503 | using iso | use_cpc= True
Saved submission_e5_base.csv; elapsed 1.63 min


In [9]:
# Rebuild folds by id on original train (Fix A): StratifiedGroupKFold by anchor; save [id, fold]
import time, pandas as pd, numpy as np
from sklearn.model_selection import StratifiedGroupKFold

t0 = time.time()
train = pd.read_csv('train.csv')
assert {'id','anchor','target','context','score'}.issubset(train.columns), 'Missing columns in train.csv'

# Build stratification: score bins + CPC section
bins = pd.cut(train['score'], [-1,0.125,0.375,0.625,0.875,1.01], labels=False).astype(int)
cpc_sec = train['context'].astype(str).str[0].fillna('X')
strat = (bins.astype(str) + '_' + cpc_sec).values
groups = train['anchor'].values

NFOLDS = 5
folds = np.full(len(train), -1, dtype=int)
sgkf = StratifiedGroupKFold(n_splits=NFOLDS, shuffle=True, random_state=42)
for f, (_, val_idx) in enumerate(sgkf.split(train, strat, groups=groups)):
    folds[val_idx] = f
assert (folds >= 0).all(), 'Fold assignment failed'

folds_df = train[['id']].copy()
folds_df['fold'] = folds
folds_df.to_csv('folds_by_id.csv', index=False)
print('Saved folds_by_id.csv with shape', folds_df.shape, 'in', f"{time.time()-t0:.2f}s")

# Diagnostics: each anchor in exactly one fold
chk = pd.DataFrame({'anchor': train['anchor'], 'fold': folds})
g = chk.groupby('anchor')['fold'].nunique()
viol = int((g != 1).sum())
print('Anchors with >1 fold assignment:', viol)
assert viol == 0, 'Anchor split across folds! Check SGKF settings.'

print('Fold sizes:', pd.Series(folds).value_counts().sort_index().to_dict())
print('CPC section x fold head:')
print(pd.crosstab(folds, cpc_sec).head())

# Sanity: Pearson plumbing check (global mean predictor ~ 0 corr)
from scipy.stats import pearsonr
y = train['score'].values.astype(np.float32)
const_pred = np.full_like(y, y.mean())
print('Pearson(constant mean, y)=', float(pearsonr(const_pred, y)[0]))

Saved folds_by_id.csv with shape (32825, 2) in 0.25s
Anchors with >1 fold assignment: 0
Fold sizes: {0: 6086, 1: 6515, 2: 6630, 3: 6941, 4: 6653}
CPC section x fold head:
context    A     B     C    D    E    F     G     H
row_0                                              
0        766  1382   821  133  345  650  1000   989
1        789  1320   848  333  181  717  1058  1269
2        651  1364  1025  257  308  653  1255  1117
3        673  1622  1148  228  263  962   952  1093
4        792  1520   928  206  295  666  1155  1091
Pearson(constant mean, y)= nan


/tmp/ipykernel_542/801916040.py:42: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  print('Pearson(constant mean, y)=', float(pearsonr(const_pred, y)[0]))


In [35]:
# Blend OOFs (e5, mpnet, tfidf, bge) with constrained non-negative weights summing to 1; grid-search weights to maximize OOF Pearson
import numpy as np, pandas as pd, time
from scipy.stats import pearsonr

t0 = time.time()
train = pd.read_csv('train.csv')[['id','score']].rename(columns={'score':'y'})
o1 = pd.read_csv('oof_e5_base.csv').rename(columns={'oof':'e5'})
o2 = pd.read_csv('oof_mpnet_st.csv').rename(columns={'oof':'mpnet'})
o3 = pd.read_csv('oof_tfidf_ridge.csv').rename(columns={'oof':'tfidf'})
o4 = pd.read_csv('oof_bge.csv').rename(columns={'oof':'bge'})
df = train.merge(o1, on='id', how='left').merge(o2, on='id', how='left').merge(o3, on='id', how='left').merge(o4, on='id', how='left')
assert df[['e5','mpnet','tfidf','bge']].notna().all().all(), 'Missing OOF values in blend merge'

y = df['y'].values.astype(np.float32)
e5 = df['e5'].values.astype(np.float32)
mp = df['mpnet'].values.astype(np.float32)
tf = df['tfidf'].values.astype(np.float32)
bg = df['bge'].values.astype(np.float32)

def score_w(w1, w2, w3, w4):
    s = w1*e5 + w2*mp + w3*tf + w4*bg  # DO NOT clip OOF before Pearson
    return pearsonr(s, y)[0]

best = (-1.0, (0.0,0.0,0.0,0.0))
grid = np.arange(0.0, 1.01, 0.05)
for w1 in grid:
    for w2 in grid:
        for w3 in grid:
            w4 = 1.0 - w1 - w2 - w3
            if w4 < -1e-9:
                continue
            if w4 < 0:
                w4 = 0.0
            p = score_w(w1,w2,w3,w4)
            if p > best[0]:
                best = (p, (w1,w2,w3,w4))
print('Best OOF Pearson blend (RAW):', round(best[0], 6), 'weights (e5, mpnet, tfidf, bge)=', best[1])

# Apply weights to test submissions
sub1 = pd.read_csv('submission_e5_base.csv').rename(columns={'score':'e5'})
sub2 = pd.read_csv('submission_mpnet_st.csv').rename(columns={'score':'mpnet'})
sub3 = pd.read_csv('submission_tfidf.csv').rename(columns={'score':'tfidf'})
sub4 = pd.read_csv('submission_bge.csv').rename(columns={'score':'bge'})
sub = sub1.merge(sub2, on='id', how='inner').merge(sub3, on='id', how='inner').merge(sub4, on='id', how='inner')
w1,w2,w3,w4 = best[1]
sub['score'] = np.clip(w1*sub['e5'] + w2*sub['mpnet'] + w3*sub['tfidf'] + w4*sub['bge'], 0.0, 1.0).astype(np.float32)  # clip only final test
sub[['id','score']].to_csv('submission.csv', index=False)
print('Saved submission.csv; elapsed', round((time.time()-t0)/60,2), 'min')

Best OOF Pearson blend (RAW): 0.634142 weights (e5, mpnet, tfidf, bge)= (0.6000000000000001, 0.15000000000000002, 0.0, 0.2499999999999999)
Saved submission.csv; elapsed 0.01 min


In [27]:
# e5-base-v2 (CPU) symmetric CPC-first with SECTION-only tag + per-fold whitening (remove top PCs), RAW OOF
import os, time, gc, numpy as np, pandas as pd
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from scipy.stats import pearsonr
from sentence_transformers import SentenceTransformer

SEED = 42
np.random.seed(SEED)

# --- CPC mapping (comprehensive) ---
CPC_DESCRIPTIONS = {
    'A61B':'Diagnosis; Surgery; Identification','A61F':'Filters; Prostheses; Braces','A61K':'Preparations for medical, dental, or toilet purposes','A61M':'Devices for introducing/circulating media into the body',
    'B01D':'Separation; Sorption; Membranes','B29C':'Shaping or joining of plastics','B32B':'Layered products','B60K':'Arrangement or mounting of vehicle power plant or auxiliaries','B60R':'Vehicles, vehicle fittings, or vehicle parts','B60T':'Vehicle brake control systems or parts',
    'C01B':'Non-metallic elements; Compounds thereof','C04B':'Cement; Concrete; Ceramics','C07C':'Acyclic or carbocyclic compounds','C07D':'Heterocyclic compounds','C07K':'Peptides','C08F':'Macromolecular compounds by polymerising monomers','C08L':'Compositions of macromolecular compounds','C09K':'Materials for miscellaneous applications',
    'C12N':'Microorganisms or enzymes; Genetic engineering','C12P':'Fermentation or enzyme-using processes','C12Q':'Measuring/testing involving enzymes or microorganisms',
    'E04B':'Building constructions; Walls; Floors; Roofs','F16B':'Screws; Rivets; Joints','F16F':'Springs; Shock-absorbers; Dampers','F16L':'Pipes; Joints or fittings for pipes',
    'G01N':'Investigating or analysing materials','G02B':'Optical elements, systems, or apparatus','G02F':'Control of light','G06F':'Electric digital data processing','G06K':'Recognition/Presentation of data','G06Q':'Data processing for administrative/financial/managerial',
    'H01L':'Semiconductor devices','H01M':'Batteries; Fuel cells','H02J':'Power supply/distribution','H04L':'Transmission of digital information','H04N':'Pictorial communication','H04W':'Wireless communication networks','H05K':'Printed circuits; Electric assemblies',
    'A01': 'Agriculture; Forestry; Animal husbandry; Hunting; Trapping; Fishing','A21': 'Baking; Edible doughs','A23': 'Foods or foodstuffs; Treatment thereof','A24': "Tobacco; Cigars; Cigarettes; Simulated smoking devices; Smokers' requisites",
    'A41': 'Wearing apparel','A42': 'Headwear','A43': 'Footwear','A45': 'Hand or travelling articles','A46': 'Brushware','A47': 'Furniture; Domestic articles or appliances; Coffee mills; Spice mills; Suction cleaners','A61': 'Medical or veterinary science; Hygiene','A62': 'Life-saving; Fire-fighting','A63': 'Sports; Games; Amusements',
    'B01': 'Physical or chemical processes or apparatus','B02': 'Crushing; pulverising; disintegrating','B03': 'Separation of solid materials using liquids or jigs','B04': 'Centrifugal apparatus or machines','B05': 'Spraying or atomising; Applying fluent materials to surfaces','B06': 'Mechanical vibrations','B07': 'Separating solids; Sorting','B08': 'Cleaning','B09': 'Disposal of solid waste; Soil reclamation',
    'B21': 'Mechanical metal-working without removing material','B22': 'Casting; Powder metallurgy','B23': 'Machine tools; Metal-working n.e.c.','B24': 'Grinding; Polishing','B25': 'Hand tools; Portable power-driven tools; Manipulators','B26': 'Hand cutting tools; Cutting; Severing','B27': 'Working/preserving wood; Nailing or stapling','B28': 'Working cement, clay, or stone','B29': 'Working of plastics; Plastic state substances','B30': 'Presses','B31': 'Making paper-like articles','B32': 'Layered products','B33': 'Additive manufacturing','B41': 'Printing; Typewriters; Stamps','B42': 'Bookbinding; Albums; Files','B43': 'Writing or drawing implements; Bureau accessories','B44': 'Decorative arts','B60': 'Vehicles in general','B61': 'Railways','B62': 'Land vehicles not on rails','B63': 'Ships or waterborne vessels','B64': 'Aircraft; Aviation; Cosmonautics','B65': 'Conveying; Packing; Storing','B66': 'Hoisting; Lifting; Hauling','B67': 'Opening/closing containers; Liquid handling','B68': 'Saddlery; Upholstery','B81': 'Microstructural technology','B82': 'Nanotechnology',
    'C01': 'Inorganic chemistry','C02': 'Treatment of water, wastewater, sewage, sludge','C03': 'Glass; Mineral or slag wool','C04': 'Cements; Concrete; Ceramics; Refractories','C05': 'Fertilisers','C06': 'Explosives; Matches','C07': 'Organic chemistry','C08': 'Organic macromolecular compounds; Polymers','C09': 'Dyes; Paints; Polishes; Adhesives; Misc compositions','C10': 'Petroleum/gas/coke industries; Fuels; Lubricants; Peat','C11': 'Oils, fats, waxes; Detergents; Candles','C12': 'Biochemistry; Microbiology; Enzymology; Fermentation','C13': 'Sugar industry','C14': 'Skins; Hides; Leather',
    'C21': 'Metallurgy of iron','C22': 'Metallurgy; Alloys; Treatment of non-ferrous metals','C23': 'Coating metallic material; Chemical surface treatment','C25': 'Electrolytic/electrophoretic processes','C30': 'Crystal growth','C40': 'Combinatorial technology',
    'D01': 'Threads or fibres; Spinning','D02': 'Yarns; Finishing; Warping or beaming','D03': 'Weaving','D04': 'Braiding; Lace-making; Knitting; Non-wovens','D05': 'Sewing; Embroidering; Tufting','D06': 'Textile treatment; Laundering; Flexible materials','D07': 'Ropes; Cables other than electric','D21': 'Paper-making; Cellulose production',
    'E01': 'Road/rail/bridge construction','E02': 'Hydraulic engineering; Foundations; Soil shifting','E03': 'Water supply; Sewerage','E04': 'Building','E05': 'Locks; Keys; Fittings; Safes','E06': 'Doors; Windows; Shutters; Ladders','E21': 'Earth/rock drilling; Mining',
    'F01': 'Machines or engines in general','F02': 'Combustion engines; Hot-gas plants','F03': 'Machines or engines for liquids; Wind/spring motors','F04': 'Positive-displacement machines; Pumps','F15': 'Fluid-pressure actuators; Hydraulics; Pneumatics','F16': 'Engineering elements or units','F17': 'Storing/distributing gases or liquids','F21': 'Lighting','F22': 'Steam generation','F23': 'Combustion apparatus; Processes','F24': 'Heating; Ranges; Ventilating','F25': 'Refrigeration or cooling; Heat pumps','F26': 'Drying','F27': 'Furnaces; Kilns; Ovens','F28': 'Heat exchange in general','F41': 'Weapons','F42': 'Ammunition; Blasting',
    'G01': 'Measuring; Testing','G02': 'Optics','G03': 'Photography; Cinematography; Non-optical waves','G04': 'Horology','G05': 'Controlling; Regulating','G06': 'Computing; Calculating; Counting','G07': 'Checking-devices','G08': 'Signalling','G09': 'Educating; Cryptography; Display; Advertising; Seals','G10': 'Musical instruments; Acoustics','G11': 'Information storage','G12': 'Instrument details','G16': 'ICT adapted for specific fields','G21': 'Nuclear physics; Nuclear engineering',
    'H01': 'Electric elements','H02': 'Electric power: generation, conversion, distribution','H03': 'Electronic circuitry','H04': 'Electric communication technique','H05': 'Electric techniques not otherwise provided for',
}
SECTION = {
    'A':'Human necessities','B':'Performing operations; Transporting','C':'Chemistry; Metallurgy','D':'Textiles; Paper',
    'E':'Fixed constructions','F':'Mechanical engineering; Lighting; Heating; Weapons; Blasting','G':'Physics','H':'Electricity'
}

def get_section_text(code: str) -> str:
    if not isinstance(code, str) or not code:
        return 'General technology'
    sec = code.strip().upper()[0]
    return SECTION.get(sec, 'General technology')

def l2n(x: np.ndarray, eps: float = 1e-12) -> np.ndarray:
    n = np.linalg.norm(x, axis=1, keepdims=True) + eps
    return x / n

def fit_whitener(X: np.ndarray, k: int = 2):
    mu = X.mean(axis=0, keepdims=True)
    Xc = X - mu
    pca = PCA(n_components=k, random_state=SEED).fit(Xc)
    P = pca.components_
    def transform(Y: np.ndarray) -> np.ndarray:
        Yc = Y - mu
        Z = Yc - (Yc @ P.T) @ P
        return l2n(Z)
    return transform

def build_texts(df: pd.DataFrame):
    sec = df['context'].astype(str).map(get_section_text)
    anchors = ("passage: " + sec + " [SEP] " + df['anchor'].astype(str)).values
    targets = ("passage: " + sec + " [SEP] " + df['target'].astype(str)).values
    return anchors, targets

# --- Data ---
t0 = time.time()
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
folds = pd.read_csv('folds_by_id.csv')
train = train.merge(folds, on='id', how='left', validate='one_to_one')
assert (train['fold']>=0).all(), 'Fold merge by id failed'

# Symmetric CPC-first with SECTION-only tag for e5
tr_a, tr_b = build_texts(train)
te_a, te_b = build_texts(test)

# Encode unique texts
all_texts = np.concatenate([tr_a, tr_b, te_a, te_b])
uniq_texts, inv_idx = np.unique(all_texts, return_inverse=True)
n_tr = len(train); n_te = len(test); n_a_tr = len(tr_a); n_b_tr = len(tr_b)
print('e5 (sym SECTION tag) unique texts:', len(uniq_texts))

os.environ.setdefault('OMP_NUM_THREADS', '8')
os.environ.setdefault('MKL_NUM_THREADS', '8')
MODEL = 'intfloat/e5-base-v2'
st_model = SentenceTransformer(MODEL, device='cpu')
st_model.max_seq_length = 256
embs = st_model.encode(uniq_texts.tolist(), batch_size=48, normalize_embeddings=True, convert_to_numpy=True, show_progress_bar=True)
assert embs.shape[0] == len(uniq_texts)

idx_tr_a = inv_idx[0:n_a_tr]
idx_tr_b = inv_idx[n_a_tr:n_a_tr+n_b_tr]
idx_te_a = inv_idx[n_a_tr+n_b_tr:n_a_tr+n_b_tr+n_te]
idx_te_b = inv_idx[n_a_tr+n_b_tr+n_te:]
tr_a_emb = embs[idx_tr_a]; tr_b_emb = embs[idx_tr_b]
te_a_emb = embs[idx_te_a]; te_b_emb = embs[idx_te_b]

def cosine_rows(a, b):
    return np.sum(a*b, axis=1).astype(np.float32)

# Baseline cosine (pre-whitening) stats
train_cos0 = cosine_rows(tr_a_emb, tr_b_emb)
print('pre-whiten cos mean/std:', float(train_cos0.mean()), float(train_cos0.std()))

# Per-fold whitening before cosine + calibration
y = train['score'].values.astype(np.float32)
oof_iso = np.zeros(n_tr, dtype=np.float32)
oof_lin = np.zeros(n_tr, dtype=np.float32)
test_iso_f, test_lin_f = [], []
K_PCS = 3  # increase PCs removal since CPC still dominated previously

for f in sorted(train['fold'].unique()):
    f0 = time.time()
    tr_idx = np.where(train['fold'] != f)[0]
    va_idx = np.where(train['fold'] == f)[0]
    W = fit_whitener(np.vstack([tr_a_emb[tr_idx], tr_b_emb[tr_idx]]), k=K_PCS)
    wa_tr_a = W(tr_a_emb[tr_idx]); wb_tr_b = W(tr_b_emb[tr_idx])
    wa_va_a = W(tr_a_emb[va_idx]); wb_va_b = W(tr_b_emb[va_idx])
    wa_te_a = W(te_a_emb);        wb_te_b = W(te_b_emb)

    cos_tr = cosine_rows(wa_tr_a, wb_tr_b)
    cos_va = cosine_rows(wa_va_a, wb_va_b)
    cos_te = cosine_rows(wa_te_a, wb_te_b)

    raw_p = pearsonr(cos_va, y[va_idx])[0]
    iso = IsotonicRegression(out_of_bounds='clip')
    iso.fit(cos_tr, y[tr_idx])
    oof_iso[va_idx] = iso.predict(cos_va).astype(np.float32)
    test_iso_f.append(iso.predict(cos_te).astype(np.float32))

    lr = LinearRegression()
    lr.fit(cos_tr.reshape(-1,1), y[tr_idx])
    oof_lin[va_idx] = lr.predict(cos_va.reshape(-1,1)).astype(np.float32)
    test_lin_f.append(lr.predict(cos_te.reshape(-1,1)).astype(np.float32))
    print(f'Fold {f}: raw-whiten-cos pearson={raw_p:.6f} | iso={pearsonr(oof_iso[va_idx], y[va_idx])[0]:.6f} | lin={pearsonr(oof_lin[va_idx], y[va_idx])[0]:.6f} | {time.time()-f0:.1f}s', flush=True)

oof_iso_p = pearsonr(oof_iso, y)[0]
oof_lin_p = pearsonr(oof_lin, y)[0]
use_iso = oof_iso_p >= oof_lin_p
print('OOF Pearson e5 sym SECTION + whitening (RAW): iso=', round(oof_iso_p,6), ' lin=', round(oof_lin_p,6), ' -> using', 'iso' if use_iso else 'lin')

if use_iso:
    oof = oof_iso
    test_pred = np.mean(np.vstack(test_iso_f), axis=0).astype(np.float32)
else:
    oof = oof_lin
    test_pred = np.mean(np.vstack(test_lin_f), axis=0).astype(np.float32)

pd.DataFrame({'id': train['id'], 'oof': oof}).to_csv('oof_e5_base.csv', index=False)
sub = pd.DataFrame({'id': test['id'], 'score': np.clip(test_pred, 0.0, 1.0).astype(np.float32)})
sub.to_csv('submission_e5_base.csv', index=False)
print('Saved oof_e5_base.csv & submission_e5_base.csv; elapsed', round((time.time()-t0)/60,2), 'min')

del embs, tr_a_emb, tr_b_emb, te_a_emb, te_b_emb
gc.collect()

e5 (sym SECTION tag) unique texts: 34347


Batches:   0%|          | 0/716 [00:00<?, ?it/s]

Batches:   0%|          | 1/716 [00:00<05:15,  2.26it/s]

Batches:   0%|          | 2/716 [00:00<03:16,  3.63it/s]

Batches:   0%|          | 3/716 [00:00<03:07,  3.81it/s]

...

Fold 2: raw-whiten-cos pearson=0.486124 | iso=0.557398 | lin=0.486124 | 0.5s


Fold 3: raw-whiten-cos pearson=0.493090 | iso=0.556454 | lin=0.493090 | 0.5s


Fold 4: raw-whiten-cos pearson=0.504141 | iso=0.566779 | lin=0.504141 | 0.5s


OOF Pearson e5 sym SECTION + whitening (RAW): iso= 0.567382  lin= 0.502279  -> using iso
Saved oof_e5_base.csv & submission_e5_base.csv; elapsed 1.46 min


4174

In [31]:
# BGE-base-en-v1.5 (CPU): symmetric instruction + CPC-first (10 words) with context-residual and per-fold whitening; RAW OOF
import time, os, gc, numpy as np, pandas as pd
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from scipy.stats import pearsonr
from sentence_transformers import SentenceTransformer

SEED = 42
np.random.seed(SEED)

# CPC mapping (reuse from earlier, with fallbacks)
CPC_DESCRIPTIONS = {
    'A61B':'Diagnosis; Surgery; Identification','A61F':'Filters; Prostheses; Braces','A61K':'Preparations for medical, dental, or toilet purposes','A61M':'Devices for introducing/circulating media into the body',
    'B01D':'Separation; Sorption; Membranes','B29C':'Shaping or joining of plastics','B32B':'Layered products','B60K':'Arrangement or mounting of vehicle power plant or auxiliaries','B60R':'Vehicles, vehicle fittings, or vehicle parts','B60T':'Vehicle brake control systems or parts',
    'C01B':'Non-metallic elements; Compounds thereof','C04B':'Cement; Concrete; Ceramics','C07C':'Acyclic or carbocyclic compounds','C07D':'Heterocyclic compounds','C07K':'Peptides','C08F':'Macromolecular compounds by polymerising monomers','C08L':'Compositions of macromolecular compounds','C09K':'Materials for miscellaneous applications',
    'C12N':'Microorganisms or enzymes; Genetic engineering','C12P':'Fermentation or enzyme-using processes','C12Q':'Measuring/testing involving enzymes or microorganisms',
    'E04B':'Building constructions; Walls; Floors; Roofs','F16B':'Screws; Rivets; Joints','F16F':'Springs; Shock-absorbers; Dampers','F16L':'Pipes; Joints or fittings for pipes',
    'G01N':'Investigating or analysing materials','G02B':'Optical elements, systems, or apparatus','G02F':'Control of light','G06F':'Electric digital data processing','G06K':'Recognition/Presentation of data','G06Q':'Data processing for administrative/financial/managerial',
    'H01L':'Semiconductor devices','H01M':'Batteries; Fuel cells','H02J':'Power supply/distribution','H04L':'Transmission of digital information','H04N':'Pictorial communication','H04W':'Wireless communication networks','H05K':'Printed circuits; Electric assemblies',
    'A01':'Agriculture; Forestry; Animal husbandry; Hunting; Trapping; Fishing','A21':'Baking; Edible doughs','A23':'Foods or foodstuffs; Treatment thereof','A24':'Tobacco; Cigars; Cigarettes; Simulated smoking devices; Smokers\' requisites','A41':'Wearing apparel','A42':'Headwear','A43':'Footwear','A45':'Hand or travelling articles','A46':'Brushware','A47':'Furniture; Domestic articles or appliances; Coffee mills; Spice mills; Suction cleaners','A61':'Medical or veterinary science; Hygiene','A62':'Life-saving; Fire-fighting','A63':'Sports; Games; Amusements',
    'B01':'Physical or chemical processes or apparatus','B02':'Crushing; pulverising; disintegrating','B03':'Separation of solid materials using liquids or jigs','B04':'Centrifugal apparatus or machines','B05':'Spraying or atomising; Applying fluent materials to surfaces','B06':'Mechanical vibrations','B07':'Separating solids; Sorting','B08':'Cleaning','B09':'Disposal of solid waste; Soil reclamation','B21':'Mechanical metal-working without removing material','B22':'Casting; Powder metallurgy','B23':'Machine tools; Metal-working n.e.c.','B24':'Grinding; Polishing','B25':'Hand tools; Portable power-driven tools; Manipulators','B26':'Hand cutting tools; Cutting; Severing','B27':'Working/preserving wood; Nailing or stapling','B28':'Working cement, clay, or stone','B29':'Working of plastics; Plastic state substances','B30':'Presses','B31':'Making paper-like articles','B32':'Layered products','B33':'Additive manufacturing','B41':'Printing; Typewriters; Stamps','B42':'Bookbinding; Albums; Files','B43':'Writing or drawing implements; Bureau accessories','B44':'Decorative arts','B60':'Vehicles in general','B61':'Railways','B62':'Land vehicles not on rails','B63':'Ships or waterborne vessels','B64':'Aircraft; Aviation; Cosmonautics','B65':'Conveying; Packing; Storing','B66':'Hoisting; Lifting; Hauling','B67':'Opening/closing containers; Liquid handling','B68':'Saddlery; Upholstery','B81':'Microstructural technology','B82':'Nanotechnology',
    'C01':'Inorganic chemistry','C02':'Treatment of water, wastewater, sewage, sludge','C03':'Glass; Mineral or slag wool','C04':'Cements; Concrete; Ceramics; Refractories','C05':'Fertilisers','C06':'Explosives; Matches','C07':'Organic chemistry','C08':'Organic macromolecular compounds; Polymers','C09':'Dyes; Paints; Polishes; Adhesives; Misc compositions','C10':'Petroleum/gas/coke industries; Fuels; Lubricants; Peat','C11':'Oils, fats, waxes; Detergents; Candles','C12':'Biochemistry; Microbiology; Enzymology; Fermentation','C13':'Sugar industry','C14':'Skins; Hides; Leather',
    'C21':'Metallurgy of iron','C22':'Metallurgy; Alloys; Treatment of non-ferrous metals','C23':'Coating metallic material; Chemical surface treatment','C25':'Electrolytic/electrophoretic processes','C30':'Crystal growth','C40':'Combinatorial technology',
    'D01':'Threads or fibres; Spinning','D02':'Yarns; Finishing; Warping or beaming','D03':'Weaving','D04':'Braiding; Lace-making; Knitting; Non-wovens','D05':'Sewing; Embroidering; Tufting','D06':'Textile treatment; Laundering; Flexible materials','D07':'Ropes; Cables other than electric','D21':'Paper-making; Cellulose production',
    'E01':'Road/rail/bridge construction','E02':'Hydraulic engineering; Foundations; Soil shifting','E03':'Water supply; Sewerage','E04':'Building','E05':'Locks; Keys; Fittings; Safes','E06':'Doors; Windows; Shutters; Ladders','E21':'Earth/rock drilling; Mining',
    'F01':'Machines or engines in general','F02':'Combustion engines; Hot-gas plants','F03':'Machines or engines for liquids; Wind/spring motors','F04':'Positive-displacement machines; Pumps','F15':'Fluid-pressure actuators; Hydraulics; Pneumatics','F16':'Engineering elements or units','F17':'Storing/distributing gases or liquids','F21':'Lighting','F22':'Steam generation','F23':'Combustion apparatus; Processes','F24':'Heating; Ranges; Ventilating','F25':'Refrigeration or cooling; Heat pumps','F26':'Drying','F27':'Furnaces; Kilns; Ovens','F28':'Heat exchange in general','F41':'Weapons','F42':'Ammunition; Blasting',
    'G01':'Measuring; Testing','G02':'Optics','G03':'Photography; Cinematography; Non-optical waves','G04':'Horology','G05':'Controlling; Regulating','G06':'Computing; Calculating; Counting','G07':'Checking-devices','G08':'Signalling','G09':'Educating; Cryptography; Display; Advertising; Seals','G10':'Musical instruments; Acoustics','G11':'Information storage','G12':'Instrument details','G16':'ICT adapted for specific fields','G21':'Nuclear physics; Nuclear engineering',
    'H01':'Electric elements','H02':'Electric power: generation, conversion, distribution','H03':'Electronic circuitry','H04':'Electric communication technique','H05':'Electric techniques not otherwise provided for'
}
SECTION = {'A':'Human necessities','B':'Performing operations; Transporting','C':'Chemistry; Metallurgy','D':'Textiles; Paper','E':'Fixed constructions','F':'Mechanical engineering; Lighting; Heating; Weapons; Blasting','G':'Physics','H':'Electricity'}

def get_cpc_text(code: str) -> str:
    if not isinstance(code, str) or not code:
        return 'General technology'
    code = code.strip().upper()
    for k in (4,3,2):
        pref = code[:k]
        if pref in CPC_DESCRIPTIONS:
            return CPC_DESCRIPTIONS[pref]
    return SECTION.get(code[0], 'General technology')

def truncate_words(text: str, n: int = 10) -> str:
    w = text.split() if isinstance(text, str) else []
    return ' '.join(w[:n])

def build_bge_texts(df: pd.DataFrame):
    cpc_full = df['context'].astype(str).map(get_cpc_text)
    cpc_10w = cpc_full.map(lambda x: truncate_words(x, 10))
    # Symmetric instruction template
    a = ("Represent the sentence for similarity: " + cpc_10w + " [SEP] " + df['anchor'].astype(str)).values
    b = ("Represent the sentence for similarity: " + cpc_10w + " [SEP] " + df['target'].astype(str)).values
    cpc_only = ("Represent the sentence for similarity: " + cpc_10w).values
    return a, b, cpc_only

def l2n(x: np.ndarray, eps: float = 1e-12) -> np.ndarray:
    n = np.linalg.norm(x, axis=1, keepdims=True) + eps
    return x / n

def fit_whitener(X: np.ndarray, k: int = 1):
    mu = X.mean(axis=0, keepdims=True)
    Xc = X - mu
    if k > 0:
        pca = PCA(n_components=k, random_state=SEED).fit(Xc)
        P = pca.components_
    else:
        P = None
    def transform(Y: np.ndarray) -> np.ndarray:
        Yc = Y - mu
        if P is not None:
            Z = Yc - (Yc @ P.T) @ P
        else:
            Z = Yc
        return l2n(Z)
    return transform

def cosine_rows(a, b):
    return np.sum(a*b, axis=1).astype(np.float32)

t0 = time.time()
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
folds = pd.read_csv('folds_by_id.csv')
train = train.merge(folds, on='id', how='left', validate='one_to_one')
assert (train['fold']>=0).all(), 'Fold merge by id failed'

# Build texts
tr_a, tr_b, tr_cpc = build_bge_texts(train)
te_a, te_b, te_cpc = build_bge_texts(test)

# Deduplicate all strings to encode once
all_texts = np.concatenate([tr_a, tr_b, tr_cpc, te_a, te_b, te_cpc])
uniq_texts, inv = np.unique(all_texts, return_inverse=True)
n_tr = len(train); n_te = len(test)
n_a_tr = len(tr_a); n_b_tr = len(tr_b); n_c_tr = len(tr_cpc)
print('BGE unique texts to encode:', len(uniq_texts))

model_name = 'BAAI/bge-base-en-v1.5'
st_model = SentenceTransformer(model_name, device='cpu')
st_model.max_seq_length = 256
embs = st_model.encode(uniq_texts.tolist(), batch_size=48, normalize_embeddings=True, convert_to_numpy=True, show_progress_bar=True)
assert embs.shape[0] == len(uniq_texts)

# Map back
idx = 0
ia_tr_a = inv[idx:idx+n_a_tr]; idx += n_a_tr
ia_tr_b = inv[idx:idx+n_b_tr]; idx += n_b_tr
ia_tr_c = inv[idx:idx+n_c_tr]; idx += n_c_tr
ia_te_a = inv[idx:idx+n_te]; idx += n_te
ia_te_b = inv[idx:idx+n_te]; idx += n_te
ia_te_c = inv[idx:idx+n_te]; idx += n_te

tr_a_emb0 = embs[ia_tr_a]; tr_b_emb0 = embs[ia_tr_b]; tr_cpc_emb0 = embs[ia_tr_c]
te_a_emb0 = embs[ia_te_a]; te_b_emb0 = embs[ia_te_b]; te_cpc_emb0 = embs[ia_te_c]
del embs; gc.collect()

# Context-residual: r = norm(emb_with_cpc) - alpha * norm(emb_cpc_only); then L2 renorm
alpha = 0.3
def apply_residual(X_with, X_cpc, a=alpha):
    Xa = l2n(X_with) - a * l2n(X_cpc)
    return l2n(Xa)

tr_a_emb = apply_residual(tr_a_emb0, tr_cpc_emb0, alpha)
tr_b_emb = apply_residual(tr_b_emb0, tr_cpc_emb0, alpha)
te_a_emb = apply_residual(te_a_emb0, te_cpc_emb0, alpha)
te_b_emb = apply_residual(te_b_emb0, te_cpc_emb0, alpha)

# Per-fold whitening and calibration
y = train['score'].values.astype(np.float32)
oof_iso = np.zeros(n_tr, dtype=np.float32)
oof_lin = np.zeros(n_tr, dtype=np.float32)
test_iso_f, test_lin_f = [], []
K_PCS = 1
for f in sorted(train['fold'].unique()):
    tr_idx = np.where(train['fold'] != f)[0]
    va_idx = np.where(train['fold'] == f)[0]
    W = fit_whitener(np.vstack([tr_a_emb[tr_idx], tr_b_emb[tr_idx]]), k=K_PCS)
    wa_tr_a = W(tr_a_emb[tr_idx]); wb_tr_b = W(tr_b_emb[tr_idx])
    wa_va_a = W(tr_a_emb[va_idx]); wb_va_b = W(tr_b_emb[va_idx])
    wa_te_a = W(te_a_emb);        wb_te_b = W(te_b_emb)

    cos_tr = cosine_rows(wa_tr_a, wb_tr_b)
    cos_va = cosine_rows(wa_va_a, wb_va_b)
    cos_te = cosine_rows(wa_te_a, wb_te_b)

    raw_p = pearsonr(cos_va, y[va_idx])[0]
    iso = IsotonicRegression(out_of_bounds='clip')
    iso.fit(cos_tr, y[tr_idx])
    oof_iso[va_idx] = iso.predict(cos_va).astype(np.float32)
    test_iso_f.append(iso.predict(cos_te).astype(np.float32))

    lr = LinearRegression()
    lr.fit(cos_tr.reshape(-1,1), y[tr_idx])
    oof_lin[va_idx] = lr.predict(cos_va.reshape(-1,1)).astype(np.float32)
    test_lin_f.append(lr.predict(cos_te.reshape(-1,1)).astype(np.float32))
    print(f'BGE Fold {f}: raw-cos={raw_p:.6f} | iso={pearsonr(oof_iso[va_idx], y[va_idx])[0]:.6f} | lin={pearsonr(oof_lin[va_idx], y[va_idx])[0]:.6f}', flush=True)

oof_iso_p = pearsonr(oof_iso, y)[0]
oof_lin_p = pearsonr(oof_lin, y)[0]
use_iso = oof_iso_p >= oof_lin_p
print('OOF Pearson BGE residual (RAW): iso=', round(oof_iso_p,6), ' lin=', round(oof_lin_p,6), ' -> using', 'iso' if use_iso else 'lin')

if use_iso:
    oof = oof_iso
    test_pred = np.mean(np.vstack(test_iso_f), axis=0).astype(np.float32)
else:
    oof = oof_lin
    test_pred = np.mean(np.vstack(test_lin_f), axis=0).astype(np.float32)

pd.DataFrame({'id': train['id'], 'oof': oof}).to_csv('oof_bge.csv', index=False)
pd.DataFrame({'id': test['id'], 'score': np.clip(test_pred, 0.0, 1.0).astype(np.float32)}).to_csv('submission_bge.csv', index=False)
print('Saved oof_bge.csv & submission_bge.csv; elapsed', round((time.time()-t0)/60,2), 'min')
del tr_a_emb0, tr_b_emb0, tr_cpc_emb0, te_a_emb0, te_b_emb0, te_cpc_emb0, tr_a_emb, tr_b_emb, te_a_emb, te_b_emb
gc.collect()

BGE unique texts to encode: 37371


Batches:   0%|          | 0/779 [00:00<?, ?it/s]

Batches:   0%|          | 1/779 [00:00<06:00,  2.16it/s]

Batches:   0%|          | 2/779 [00:00<04:07,  3.13it/s]

Batches:   0%|          | 3/779 [00:00<03:50,  3.37it/s]

...

BGE Fold 2: raw-cos=0.519857 | iso=0.599609 | lin=0.519857


BGE Fold 3: raw-cos=0.548265 | iso=0.610782 | lin=0.548265


BGE Fold 4: raw-cos=0.518261 | iso=0.603110 | lin=0.518261


OOF Pearson BGE residual (RAW): iso= 0.603238  lin= 0.528497  -> using iso
Saved oof_bge.csv & submission_bge.csv; elapsed 2.34 min


0

In [33]:
# e5-base-v2 (CPU): asymmetric anchor-only CPC (passage template), per-fold whitening k=1 and iso/lin; RAW OOF
import time, os, gc, numpy as np, pandas as pd
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LinearRegression
from sklearn.decomposition import PCA
from scipy.stats import pearsonr
from sentence_transformers import SentenceTransformer

SEED = 42
np.random.seed(SEED)

# CPC mapping and helpers (reuse compact map approach)
CPC_DESCRIPTIONS = {
    'A61B':'Diagnosis; Surgery; Identification','A61F':'Filters; Prostheses; Braces','A61K':'Preparations for medical, dental, or toilet purposes','A61M':'Devices for introducing/circulating media into the body',
    'B01D':'Separation; Sorption; Membranes','B29C':'Shaping or joining of plastics','B32B':'Layered products','B60K':'Arrangement or mounting of vehicle power plant or auxiliaries','B60R':'Vehicles, vehicle fittings, or vehicle parts','B60T':'Vehicle brake control systems or parts',
    'C01B':'Non-metallic elements; Compounds thereof','C04B':'Cement; Concrete; Ceramics','C07C':'Acyclic or carbocyclic compounds','C07D':'Heterocyclic compounds','C07K':'Peptides','C08F':'Macromolecular compounds by polymerising monomers','C08L':'Compositions of macromolecular compounds','C09K':'Materials for miscellaneous applications',
    'C12N':'Microorganisms or enzymes; Genetic engineering','C12P':'Fermentation or enzyme-using processes','C12Q':'Measuring/testing involving enzymes or microorganisms',
    'E04B':'Building constructions; Walls; Floors; Roofs','F16B':'Screws; Rivets; Joints','F16F':'Springs; Shock-absorbers; Dampers','F16L':'Pipes; Joints or fittings for pipes',
    'G01N':'Investigating or analysing materials','G02B':'Optical elements, systems, or apparatus','G02F':'Control of light','G06F':'Electric digital data processing','G06K':'Recognition/Presentation of data','G06Q':'Data processing for administrative/financial/managerial',
    'H01L':'Semiconductor devices','H01M':'Batteries; Fuel cells','H02J':'Power supply/distribution','H04L':'Transmission of digital information','H04N':'Pictorial communication','H04W':'Wireless communication networks','H05K':'Printed circuits; Electric assemblies',
    'A01':'Agriculture; Forestry; Animal husbandry; Hunting; Trapping; Fishing','A21':'Baking; Edible doughs','A23':'Foods or foodstuffs; Treatment thereof','A24':'Tobacco; Cigars; Cigarettes; Simulated smoking devices; Smokers\' requisites','A41':'Wearing apparel','A42':'Headwear','A43':'Footwear','A45':'Hand or travelling articles','A46':'Brushware','A47':'Furniture; Domestic articles or appliances; Coffee mills; Spice mills; Suction cleaners','A61':'Medical or veterinary science; Hygiene','A62':'Life-saving; Fire-fighting','A63':'Sports; Games; Amusements',
    'B01':'Physical or chemical processes or apparatus','B02':'Crushing; pulverising; disintegrating','B03':'Separation of solid materials using liquids or jigs','B04':'Centrifugal apparatus or machines','B05':'Spraying or atomising; Applying fluent materials to surfaces','B06':'Mechanical vibrations','B07':'Separating solids; Sorting','B08':'Cleaning','B09':'Disposal of solid waste; Soil reclamation','B21':'Mechanical metal-working without removing material','B22':'Casting; Powder metallurgy','B23':'Machine tools; Metal-working n.e.c.','B24':'Grinding; Polishing','B25':'Hand tools; Portable power-driven tools; Manipulators','B26':'Hand cutting tools; Cutting; Severing','B27':'Working/preserving wood; Nailing or stapling','B28':'Working cement, clay, or stone','B29':'Working of plastics; Plastic state substances','B30':'Presses','B31':'Making paper-like articles','B32':'Layered products','B33':'Additive manufacturing','B41':'Printing; Typewriters; Stamps','B42':'Bookbinding; Albums; Files','B43':'Writing or drawing implements; Bureau accessories','B44':'Decorative arts','B60':'Vehicles in general','B61':'Railways','B62':'Land vehicles not on rails','B63':'Ships or waterborne vessels','B64':'Aircraft; Aviation; Cosmonautics','B65':'Conveying; Packing; Storing','B66':'Hoisting; Lifting; Hauling','B67':'Opening/closing containers; Liquid handling','B68':'Saddlery; Upholstery','B81':'Microstructural technology','B82':'Nanotechnology',
    'C01':'Inorganic chemistry','C02':'Treatment of water, wastewater, sewage, sludge','C03':'Glass; Mineral or slag wool','C04':'Cements; Concrete; Ceramics; Refractories','C05':'Fertilisers','C06':'Explosives; Matches','C07':'Organic chemistry','C08':'Organic macromolecular compounds; Polymers','C09':'Dyes; Paints; Polishes; Adhesives; Misc compositions','C10':'Petroleum/gas/coke industries; Fuels; Lubricants; Peat','C11':'Oils, fats, waxes; Detergents; Candles','C12':'Biochemistry; Microbiology; Enzymology; Fermentation','C13':'Sugar industry','C14':'Skins; Hides; Leather',
    'C21':'Metallurgy of iron','C22':'Metallurgy; Alloys; Treatment of non-ferrous metals','C23':'Coating metallic material; Chemical surface treatment','C25':'Electrolytic/electrophoretic processes','C30':'Crystal growth','C40':'Combinatorial technology',
    'D01':'Threads or fibres; Spinning','D02':'Yarns; Finishing; Warping or beaming','D03':'Weaving','D04':'Braiding; Lace-making; Knitting; Non-wovens','D05':'Sewing; Embroidering; Tufting','D06':'Textile treatment; Laundering; Flexible materials','D07':'Ropes; Cables other than electric','D21':'Paper-making; Cellulose production',
    'E01':'Road/rail/bridge construction','E02':'Hydraulic engineering; Foundations; Soil shifting','E03':'Water supply; Sewerage','E04':'Building','E05':'Locks; Keys; Fittings; Safes','E06':'Doors; Windows; Shutters; Ladders','E21':'Earth/rock drilling; Mining',
    'F01':'Machines or engines in general','F02':'Combustion engines; Hot-gas plants','F03':'Machines or engines for liquids; Wind/spring motors','F04':'Positive-displacement machines; Pumps','F15':'Fluid-pressure actuators; Hydraulics; Pneumatics','F16':'Engineering elements or units','F17':'Storing/distributing gases or liquids','F21':'Lighting','F22':'Steam generation','F23':'Combustion apparatus; Processes','F24':'Heating; Ranges; Ventilating','F25':'Refrigeration or cooling; Heat pumps','F26':'Drying','F27':'Furnaces; Kilns; Ovens','F28':'Heat exchange in general','F41':'Weapons','F42':'Ammunition; Blasting',
    'G01':'Measuring; Testing','G02':'Optics','G03':'Photography; Cinematography; Non-optical waves','G04':'Horology','G05':'Controlling; Regulating','G06':'Computing; Calculating; Counting','G07':'Checking-devices','G08':'Signalling','G09':'Educating; Cryptography; Display; Advertising; Seals','G10':'Musical instruments; Acoustics','G11':'Information storage','G12':'Instrument details','G16':'ICT adapted for specific fields','G21':'Nuclear physics; Nuclear engineering',
    'H01':'Electric elements','H02':'Electric power: generation, conversion, distribution','H03':'Electronic circuitry','H04':'Electric communication technique','H05':'Electric techniques not otherwise provided for'
}
SECTION = {'A':'Human necessities','B':'Performing operations; Transporting','C':'Chemistry; Metallurgy','D':'Textiles; Paper','E':'Fixed constructions','F':'Mechanical engineering; Lighting; Heating; Weapons; Blasting','G':'Physics','H':'Electricity'}

def get_cpc_text(code: str) -> str:
    if not isinstance(code, str) or not code:
        return 'General technology'
    code = code.strip().upper()
    for k in (4,3,2):
        pref = code[:k]
        if pref in CPC_DESCRIPTIONS:
            return CPC_DESCRIPTIONS[pref]
    return SECTION.get(code[0], 'General technology')

def truncate_words(text: str, n: int = 10) -> str:
    w = text.split() if isinstance(text, str) else []
    return ' '.join(w[:n])

def build_e5_asym_texts(df: pd.DataFrame):
    cpc_full = df['context'].astype(str).map(get_cpc_text)
    cpc_10w = cpc_full.map(lambda x: truncate_words(x, 10))
    a = ("passage: domain: " + cpc_10w + " [SEP] " + df['anchor'].astype(str)).values
    b = ("passage: " + df['target'].astype(str)).values
    return a, b

def l2n(x: np.ndarray, eps: float = 1e-12) -> np.ndarray:
    n = np.linalg.norm(x, axis=1, keepdims=True) + eps
    return x / n

def fit_whitener(X: np.ndarray, k: int = 1):
    mu = X.mean(axis=0, keepdims=True)
    Xc = X - mu
    if k > 0:
        pca = PCA(n_components=k, random_state=SEED).fit(Xc)
        P = pca.components_
    else:
        P = None
    def transform(Y: np.ndarray) -> np.ndarray:
        Yc = Y - mu
        if P is not None:
            Z = Yc - (Yc @ P.T) @ P
        else:
            Z = Yc
        return l2n(Z)
    return transform

def cosine_rows(a, b):
    return np.sum(a*b, axis=1).astype(np.float32)

t0 = time.time()
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
folds = pd.read_csv('folds_by_id.csv')
train = train.merge(folds, on='id', how='left', validate='one_to_one')
assert (train['fold']>=0).all(), 'Fold merge by id failed'

tr_a, tr_b = build_e5_asym_texts(train)
te_a, te_b = build_e5_asym_texts(test)

# Encode unique strings once
all_texts = np.concatenate([tr_a, tr_b, te_a, te_b])
uniq_texts, inv = np.unique(all_texts, return_inverse=True)
n_tr = len(train); n_te = len(test); n_a_tr = len(tr_a); n_b_tr = len(tr_b)
print('e5-asym unique texts to encode:', len(uniq_texts))

model_name = 'intfloat/e5-base-v2'
st_model = SentenceTransformer(model_name, device='cpu')
st_model.max_seq_length = 256
embs = st_model.encode(uniq_texts.tolist(), batch_size=48, normalize_embeddings=True, convert_to_numpy=True, show_progress_bar=True)
assert embs.shape[0] == len(uniq_texts)

idx = 0
ia_tr_a = inv[idx:idx+n_a_tr]; idx += n_a_tr
ia_tr_b = inv[idx:idx+n_b_tr]; idx += n_b_tr
ia_te_a = inv[idx:idx+n_te]; idx += n_te
ia_te_b = inv[idx:idx+n_te]; idx += n_te

tr_a_emb = embs[ia_tr_a]; tr_b_emb = embs[ia_tr_b]
te_a_emb = embs[ia_te_a]; te_b_emb = embs[ia_te_b]
del embs; gc.collect()

# Per-fold whitening (k=1) and calibration
y = train['score'].values.astype(np.float32)
oof_iso = np.zeros(n_tr, dtype=np.float32)
oof_lin = np.zeros(n_tr, dtype=np.float32)
test_iso_f, test_lin_f = [], []
K_PCS = 1
for f in sorted(train['fold'].unique()):
    tr_idx = np.where(train['fold'] != f)[0]
    va_idx = np.where(train['fold'] == f)[0]
    W = fit_whitener(np.vstack([tr_a_emb[tr_idx], tr_b_emb[tr_idx]]), k=K_PCS)
    wa_tr_a = W(tr_a_emb[tr_idx]); wb_tr_b = W(tr_b_emb[tr_idx])
    wa_va_a = W(tr_a_emb[va_idx]); wb_va_b = W(tr_b_emb[va_idx])
    wa_te_a = W(te_a_emb);        wb_te_b = W(te_b_emb)

    cos_tr = cosine_rows(wa_tr_a, wb_tr_b)
    cos_va = cosine_rows(wa_va_a, wb_va_b)
    cos_te = cosine_rows(wa_te_a, wb_te_b)

    raw_p = pearsonr(cos_va, y[va_idx])[0]
    iso = IsotonicRegression(out_of_bounds='clip')
    iso.fit(cos_tr, y[tr_idx])
    oof_iso[va_idx] = iso.predict(cos_va).astype(np.float32)
    test_iso_f.append(iso.predict(cos_te).astype(np.float32))

    lr = LinearRegression()
    lr.fit(cos_tr.reshape(-1,1), y[tr_idx])
    oof_lin[va_idx] = lr.predict(cos_va.reshape(-1,1)).astype(np.float32)
    test_lin_f.append(lr.predict(cos_te.reshape(-1,1)).astype(np.float32))
    print(f'e5-asym Fold {f}: raw-cos={raw_p:.6f} | iso={pearsonr(oof_iso[va_idx], y[va_idx])[0]:.6f} | lin={pearsonr(oof_lin[va_idx], y[va_idx])[0]:.6f}', flush=True)

oof_iso_p = pearsonr(oof_iso, y)[0]
oof_lin_p = pearsonr(oof_lin, y)[0]
use_iso = oof_iso_p >= oof_lin_p
print('OOF Pearson e5-asym (RAW): iso=', round(oof_iso_p,6), ' lin=', round(oof_lin_p,6), ' -> using', 'iso' if use_iso else 'lin')

if use_iso:
    oof = oof_iso
    test_pred = np.mean(np.vstack(test_iso_f), axis=0).astype(np.float32)
else:
    oof = oof_lin
    test_pred = np.mean(np.vstack(test_lin_f), axis=0).astype(np.float32)

pd.DataFrame({'id': train['id'], 'oof': oof}).to_csv('oof_e5_asym.csv', index=False)
pd.DataFrame({'id': test['id'], 'score': np.clip(test_pred, 0.0, 1.0).astype(np.float32)}).to_csv('submission_e5_asym.csv', index=False)
print('Saved oof_e5_asym.csv & submission_e5_asym.csv; elapsed', round((time.time()-t0)/60,2), 'min')
del tr_a_emb, tr_b_emb, te_a_emb, te_b_emb
gc.collect()

e5-asym unique texts to encode: 31039


Batches:   0%|          | 0/647 [00:00<?, ?it/s]

Batches:   0%|          | 1/647 [00:00<03:51,  2.80it/s]

Batches:   0%|          | 2/647 [00:00<03:23,  3.17it/s]

Batches:   0%|          | 3/647 [00:00<02:46,  3.86it/s]

...

e5-asym Fold 2: raw-cos=0.526769 | iso=0.537549 | lin=0.526769


e5-asym Fold 3: raw-cos=0.546558 | iso=0.554412 | lin=0.546558


e5-asym Fold 4: raw-cos=0.527586 | iso=0.532165 | lin=0.527586


OOF Pearson e5-asym (RAW): iso= 0.544407  lin= 0.536534  -> using iso
Saved oof_e5_asym.csv & submission_e5_asym.csv; elapsed 1.04 min


0

In [39]:
# Ridge stacker on OOF channels (e5_base, mpnet_st, bge, tfidf, ce_stsb, optional e5_asym); per-fold training; RAW OOF
import time, numpy as np, pandas as pd
from sklearn.linear_model import Ridge
from scipy.stats import pearsonr

t0 = time.time()
train = pd.read_csv('train.csv')[['id','score']].rename(columns={'score':'y'})
o_e5    = pd.read_csv('oof_e5_base.csv').rename(columns={'oof':'e5'})
o_mp    = pd.read_csv('oof_mpnet_st.csv').rename(columns={'oof':'mpnet'})
o_bge   = pd.read_csv('oof_bge.csv').rename(columns={'oof':'bge'})
o_tfidf = pd.read_csv('oof_tfidf_ridge.csv').rename(columns={'oof':'tfidf'})
o_ce    = pd.read_csv('oof_ce_stsb.csv').rename(columns={'oof':'ce'})

# Optional: include e5_asym if present
try:
    o_e5a = pd.read_csv('oof_e5_asym.csv').rename(columns={'oof':'e5_asym'})
    df = train.merge(o_e5, on='id').merge(o_mp, on='id').merge(o_bge, on='id').merge(o_tfidf, on='id').merge(o_ce, on='id').merge(o_e5a, on='id')
    use_cols = ['e5','mpnet','bge','tfidf','ce','e5_asym']
except Exception:
    df = train.merge(o_e5, on='id').merge(o_mp, on='id').merge(o_bge, on='id').merge(o_tfidf, on='id').merge(o_ce, on='id')
    use_cols = ['e5','mpnet','bge','tfidf','ce']

folds = pd.read_csv('folds_by_id.csv')
df = df.merge(folds, on='id', how='left', validate='one_to_one')
assert df['fold'].notna().all(), 'Missing fold assignments'

X_all = df[use_cols].values.astype(np.float32)
y_all = df['y'].values.astype(np.float32)
oof_stack = np.zeros_like(y_all, dtype=np.float32)

alpha = 1.0  # fixed to avoid leakage
for f in sorted(df['fold'].unique()):
    tr_idx = np.where(df['fold'].values != f)[0]
    va_idx = np.where(df['fold'].values == f)[0]
    mdl = Ridge(alpha=alpha, fit_intercept=True, random_state=42)
    mdl.fit(X_all[tr_idx], y_all[tr_idx])
    oof_stack[va_idx] = mdl.predict(X_all[va_idx]).astype(np.float32)
    print(f'Stacker fold {int(f)} pearson=', round(float(pearsonr(oof_stack[va_idx], y_all[va_idx])[0]), 6))

oof_p = pearsonr(oof_stack, y_all)[0]
print('OOF Pearson Ridge stack (RAW):', round(oof_p, 6), 'with cols', use_cols)
pd.DataFrame({'id': df['id'], 'oof': oof_stack}).to_csv('oof_stack_ridge.csv', index=False)

# Fit on full train OOF -> predict test blend
sub1 = pd.read_csv('submission_e5_base.csv').rename(columns={'score':'e5'})
sub2 = pd.read_csv('submission_mpnet_st.csv').rename(columns={'score':'mpnet'})
sub3 = pd.read_csv('submission_bge.csv').rename(columns={'score':'bge'})
sub4 = pd.read_csv('submission_tfidf.csv').rename(columns={'score':'tfidf'})
sub5 = pd.read_csv('submission_ce_stsb.csv').rename(columns={'score':'ce'})
sub = sub1.merge(sub2, on='id').merge(sub3, on='id').merge(sub4, on='id').merge(sub5, on='id')
if 'e5_asym' in use_cols:
    try:
        sub6 = pd.read_csv('submission_e5_asym.csv').rename(columns={'score':'e5_asym'})
        sub = sub.merge(sub6, on='id')
    except Exception:
        use_cols = [c for c in use_cols if c != 'e5_asym']

X_test = sub[use_cols].values.astype(np.float32)
mdl_full = Ridge(alpha=alpha, fit_intercept=True, random_state=42)
mdl_full.fit(X_all, y_all)
test_pred = mdl_full.predict(X_test).astype(np.float32)
sub_out = pd.DataFrame({'id': sub['id'], 'score': np.clip(test_pred, 0.0, 1.0)})
sub_out.to_csv('submission_stack_ridge.csv', index=False)
print('Saved oof_stack_ridge.csv and submission_stack_ridge.csv; elapsed', round((time.time()-t0)/60,2), 'min')

Stacker fold 0 pearson= 0.66238
Stacker fold 1 pearson= 0.65761
Stacker fold 2 pearson= 0.651261
Stacker fold 3 pearson= 0.661574
Stacker fold 4 pearson= 0.646624
OOF Pearson Ridge stack (RAW): 0.655586 with cols ['e5', 'mpnet', 'bge', 'tfidf', 'ce', 'e5_asym']
Saved oof_stack_ridge.csv and submission_stack_ridge.csv; elapsed 0.0 min


In [58]:
# Cross-Encoder STS baseline (CPU): stsb-roberta-base; context-aware pairs; RAW OOF with per-fold iso/lin calibration
import time, numpy as np, pandas as pd, re
from scipy.stats import pearsonr
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LinearRegression
from sentence_transformers import CrossEncoder

SEED = 42
np.random.seed(SEED)

# CPC mapping (4->3->2 title; RAW code fallback), truncate to 10 words
CPC_DESCRIPTIONS = {
    'A61B':'Diagnosis; Surgery; Identification','A61F':'Filters; Prostheses; Braces','A61K':'Preparations for medical, dental, or toilet purposes','A61M':'Devices for introducing/circulating media into the body',
    'B01D':'Separation; Sorption; Membranes','B29C':'Shaping or joining of plastics','B32B':'Layered products','B60K':'Arrangement or mounting of vehicle power plant or auxiliaries','B60R':'Vehicles, vehicle fittings, or vehicle parts','B60T':'Vehicle brake control systems or parts',
    'C01B':'Non-metallic elements; Compounds thereof','C04B':'Cement; Concrete; Ceramics','C07C':'Acyclic or carbocyclic compounds','C07D':'Heterocyclic compounds','C07K':'Peptides','C08F':'Macromolecular compounds by polymerising monomers','C08L':'Compositions of macromolecular compounds','C09K':'Materials for miscellaneous applications',
    'C12N':'Microorganisms or enzymes; Genetic engineering','C12P':'Fermentation or enzyme-using processes','C12Q':'Measuring/testing involving enzymes or microorganisms',
    'E04B':'Building constructions; Walls; Floors; Roofs','F16B':'Screws; Rivets; Joints','F16F':'Springs; Shock-absorbers; Dampers','F16L':'Pipes; Joints or fittings for pipes',
    'G01N':'Investigating or analysing materials','G02B':'Optical elements, systems, or apparatus','G02F':'Control of light','G06F':'Electric digital data processing','G06K':'Recognition/Presentation of data','G06Q':'Data processing for administrative/financial/managerial',
    'H01L':'Semiconductor devices','H01M':'Batteries; Fuel cells','H02J':'Power supply/distribution','H04L':'Transmission of digital information','H04N':'Pictorial communication','H04W':'Wireless communication networks','H05K':'Printed circuits; Electric assemblies',
    'A01':'Agriculture; Forestry; Animal husbandry; Hunting; Trapping; Fishing','A21':'Baking; Edible doughs','A23':'Foods or foodstuffs; Treatment thereof','A24':'Tobacco; Cigars; Cigarettes; Simulated smoking devices; Smokers\' requisites','A41':'Wearing apparel','A42':'Headwear','A43':'Footwear','A45':'Hand or travelling articles','A46':'Brushware','A47':'Furniture; Domestic articles or appliances; Coffee mills; Spice mills; Suction cleaners','A61':'Medical or veterinary science; Hygiene','A62':'Life-saving; Fire-fighting','A63':'Sports; Games; Amusements',
    'B01':'Physical or chemical processes or apparatus','B02':'Crushing; pulverising; disintegrating','B03':'Separation of solid materials using liquids or jigs','B04':'Centrifugal apparatus or machines','B05':'Spraying; atomising; Applying fluent materials to surfaces','B06':'Mechanical vibrations','B07':'Separating solids; Sorting','B08':'Cleaning','B09':'Disposal of solid waste; Soil reclamation','B21':'Mechanical metal-working without removing material','B22':'Casting; Powder metallurgy','B23':'Machine tools; Metal-working n.e.c.','B24':'Grinding; Polishing','B25':'Hand tools; Portable power-driven tools; Manipulators','B26':'Hand cutting tools; Cutting; Severing','B27':'Working/preserving wood; Nailing or stapling','B28':'Working cement, clay, or stone','B29':'Working of plastics; Plastic state substances','B30':'Presses','B31':'Making paper-like articles','B32':'Layered products','B33':'Additive manufacturing','B41':'Printing; Typewriters; Stamps','B42':'Bookbinding; Albums; Files','B43':'Writing or drawing implements; Bureau accessories','B44':'Decorative arts','B60':'Vehicles in general','B61':'Railways','B62':'Land vehicles not on rails','B63':'Ships or waterborne vessels','B64':'Aircraft; Aviation; Cosmonautics','B65':'Conveying; Packing; Storing','B66':'Hoisting; Lifting; Hauling','B67':'Opening/closing containers; Liquid handling','B68':'Saddlery; Upholstery','B81':'Microstructural technology','B82':'Nanotechnology',
    'C01':'Inorganic chemistry','C02':'Treatment of water, wastewater, sewage, sludge','C03':'Glass; Mineral or slag wool','C04':'Cements; Concrete; Ceramics; Refractories','C05':'Fertilisers','C06':'Explosives; Matches','C07':'Organic chemistry','C08':'Organic macromolecular compounds; Polymers','C09':'Dyes; Paints; Polishes; Adhesives; Misc compositions','C10':'Petroleum/gas/coke industries; Fuels; Lubricants; Peat','C11':'Oils, fats, waxes; Detergents; Candles','C12':'Biochemistry; Microbiology; Enzymology; Fermentation','C13':'Sugar industry','C14':'Skins; Hides; Leather',
    'C21':'Metallurgy of iron','C22':'Metallurgy; Alloys; Treatment of non-ferrous metals','C23':'Coating metallic material; Chemical surface treatment','C25':'Electrolytic/electrophoretic processes','C30':'Crystal growth','C40':'Combinatorial technology',
    'D01':'Threads or fibres; Spinning','D02':'Yarns; Finishing; Warping or beaming','D03':'Weaving','D04':'Braiding; Lace-making; Knitting; Non-wovens','D05':'Sewing; Embroidering; Tufting','D06':'Textile treatment; Laundering; Flexible materials','D07':'Ropes; Cables other than electric','D21':'Paper-making; Cellulose production',
    'E01':'Road/rail/bridge construction','E02':'Hydraulic engineering; Foundations; Soil shifting','E03':'Water supply; Sewerage','E04':'Building','E05':'Locks; Keys; Fittings; Safes','E06':'Doors; Windows; Shutters; Ladders','E21':'Earth/rock drilling; Mining',
    'F01':'Machines or engines in general','F02':'Combustion engines; Hot-gas plants','F03':'Machines or engines for liquids; Wind/spring motors','F04':'Positive-displacement machines; Pumps','F15':'Fluid-pressure actuators; Hydraulics; Pneumatics','F16':'Engineering elements or units','F17':'Storing/distributing gases or liquids','F21':'Lighting','F22':'Steam generation','F23':'Combustion apparatus; Processes','F24':'Heating; Ranges; Ventilating','F25':'Refrigeration or cooling; Heat pumps','F26':'Drying','F27':'Furnaces; Kilns; Ovens','F28':'Heat exchange in general','F41':'Weapons','F42':'Ammunition; Blasting',
    'G01':'Measuring; Testing','G02':'Optics','G03':'Photography; Cinematography; Non-optical waves','G04':'Horology','G05':'Controlling; Regulating','G06':'Computing; Calculating; Counting','G07':'Checking-devices','G08':'Signalling','G09':'Educating; Cryptography; Display; Advertising; Seals','G10':'Musical instruments; Acoustics','G11':'Information storage','G12':'Instrument details','G16':'ICT adapted for specific fields','G21':'Nuclear physics; Nuclear engineering',
    'H01':'Electric elements','H02':'Electric power: generation, conversion, distribution','H03':'Electronic circuitry','H04':'Electric communication technique','H05':'Electric techniques not otherwise provided for'
}

def get_cpc_text(code: str) -> str:
    if not isinstance(code, str) or not code:
        return 'GENERAL'
    code = code.strip().upper()
    for k in (4,3,2):
        pref = code[:k]
        if pref in CPC_DESCRIPTIONS:
            return CPC_DESCRIPTIONS[pref]
    return code  # RAW code fallback

def truncate_words(text: str, n: int = 10) -> str:
    w = text.split() if isinstance(text, str) else []
    return ' '.join(w[:n])

def build_ce_pairs_context_aware(df: pd.DataFrame):
    cpc_full = df['context'].astype(str).map(get_cpc_text)
    cpc_10w = cpc_full.map(lambda x: truncate_words(x, 10))
    # Pair: ("{CPC_10w} [SEP] {anchor}", target) — context on anchor side only
    a_side = (cpc_10w + ' [SEP] ' + df['anchor'].astype(str)).tolist()
    b_side = df['target'].astype(str).tolist()
    return list(zip(a_side, b_side))

t0 = time.time()
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
folds = pd.read_csv('folds_by_id.csv')
train = train.merge(folds, on='id', how='left', validate='one_to_one')
assert (train['fold']>=0).all(), 'Fold merge by id failed'

# Build context-aware pairs
train_pairs = build_ce_pairs_context_aware(train)
test_pairs = build_ce_pairs_context_aware(test)

model_name = 'cross-encoder/stsb-roberta-base'
ce = CrossEncoder(model_name, device='cpu', max_length=256, tokenizer_args={'use_fast': False})

print('Scoring train pairs with Cross-Encoder (context-aware)...', flush=True)
train_scores_raw = ce.predict(train_pairs, batch_size=64, show_progress_bar=True)
print('Scoring test pairs with Cross-Encoder (context-aware)...', flush=True)
test_scores_raw = ce.predict(test_pairs, batch_size=64, show_progress_bar=True)
train_scores_raw = np.asarray(train_scores_raw, dtype=np.float32)
test_scores_raw = np.asarray(test_scores_raw, dtype=np.float32)

# Per-fold calibration from raw CE scores to target, compare isotonic vs linear; evaluate on RAW OOF
y = train['score'].values.astype(np.float32)
oof_iso = np.zeros_like(y)
oof_lin = np.zeros_like(y)
test_iso_f, test_lin_f = [], []
for f in sorted(train['fold'].unique()):
    tr_idx = np.where(train['fold'] != f)[0]
    va_idx = np.where(train['fold'] == f)[0]
    raw_p = pearsonr(train_scores_raw[va_idx], y[va_idx])[0]
    iso = IsotonicRegression(out_of_bounds='clip')
    iso.fit(train_scores_raw[tr_idx], y[tr_idx])
    oof_iso[va_idx] = iso.predict(train_scores_raw[va_idx]).astype(np.float32)
    test_iso_f.append(iso.predict(test_scores_raw).astype(np.float32))
    lr = LinearRegression()
    lr.fit(train_scores_raw[tr_idx].reshape(-1,1), y[tr_idx])
    oof_lin[va_idx] = lr.predict(train_scores_raw[va_idx].reshape(-1,1)).astype(np.float32)
    test_lin_f.append(lr.predict(test_scores_raw.reshape(-1,1)).astype(np.float32))
    print(f'CE Fold {f}: raw={raw_p:.6f} | iso={pearsonr(oof_iso[va_idx], y[va_idx])[0]:.6f} | lin={pearsonr(oof_lin[va_idx], y[va_idx])[0]:.6f}', flush=True)

p_iso = pearsonr(oof_iso, y)[0]
p_lin = pearsonr(oof_lin, y)[0]
use_iso = p_iso >= p_lin
print('OOF Pearson Cross-Encoder (context-aware, RAW): iso=', round(p_iso,6), ' lin=', round(p_lin,6), ' -> using', 'iso' if use_iso else 'lin')

if use_iso:
    oof = oof_iso
    test_pred = np.mean(np.vstack(test_iso_f), axis=0).astype(np.float32)
else:
    oof = oof_lin
    test_pred = np.mean(np.vstack(test_lin_f), axis=0).astype(np.float32)

pd.DataFrame({'id': train['id'], 'oof': oof}).to_csv('oof_ce_stsb.csv', index=False)
pd.DataFrame({'id': test['id'], 'score': np.clip(test_pred, 0.0, 1.0)}).to_csv('submission_ce_stsb.csv', index=False)
print('Saved oof_ce_stsb.csv & submission_ce_stsb.csv; elapsed', round((time.time()-t0)/60,2), 'min')

Scoring train pairs with Cross-Encoder (context-aware)...


Batches:   0%|          | 0/513 [00:00<?, ?it/s]

Batches:   0%|          | 1/513 [00:00<03:43,  2.29it/s]

Batches:   0%|          | 2/513 [00:00<02:57,  2.88it/s]

Batches:   1%|          | 3/513 [00:01<02:46,  3.06it/s]

...

CE Fold 1: raw=0.526183 | iso=0.549955 | lin=0.526183


CE Fold 2: raw=0.517825 | iso=0.532603 | lin=0.517825


CE Fold 3: raw=0.548093 | iso=0.558382 | lin=0.548093


CE Fold 4: raw=0.524688 | iso=0.547481 | lin=0.524688


OOF Pearson Cross-Encoder (context-aware, RAW): iso= 0.548124  lin= 0.53028  -> using iso
Saved oof_ce_stsb.csv & submission_ce_stsb.csv; elapsed 2.78 min


In [40]:
# Feature-augmented stacker: add lexical overlaps and length features to OOF channels; per-fold Ridge; RAW OOF
import time, numpy as np, pandas as pd, re
from sklearn.linear_model import Ridge
from scipy.stats import pearsonr

t0 = time.time()
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
folds = pd.read_csv('folds_by_id.csv')
train = train.merge(folds, on='id', how='left', validate='one_to_one')

def tokenize(s):
    s = str(s).lower()
    return re.findall(r"\w+", s)

def jaccard(a, b):
    sa, sb = set(a), set(b)
    if not sa and not sb: return 0.0
    inter = len(sa & sb); uni = len(sa | sb)
    return inter / (uni + 1e-12)

def build_feats(df: pd.DataFrame):
    A = df['anchor'].astype(str).tolist()
    B = df['target'].astype(str).tolist()
    toks_a = [tokenize(x) for x in A]
    toks_b = [tokenize(x) for x in B]
    # Unigram Jaccard
    jac = np.array([jaccard(ta, tb) for ta, tb in zip(toks_a, toks_b)], dtype=np.float32)
    # Length features
    len_a = np.array([len(x) for x in A], dtype=np.float32)
    len_b = np.array([len(x) for x in B], dtype=np.float32)
    tok_a = np.array([len(t) for t in toks_a], dtype=np.float32)
    tok_b = np.array([len(t) for t in toks_b], dtype=np.float32)
    len_diff = np.abs(len_a - len_b)
    len_ratio = (np.minimum(len_a, len_b) + 1.0) / (np.maximum(len_a, len_b) + 1.0)
    tok_diff = np.abs(tok_a - tok_b)
    tok_ratio = (np.minimum(tok_a, tok_b) + 1.0) / (np.maximum(tok_a, tok_b) + 1.0)
    # CPC section one-hot (A..H)
    sec = df['context'].astype(str).str[:1].fillna('X')
    sec_dum = pd.get_dummies(sec, prefix='sec')
    feat_df = pd.DataFrame({
        'jac_uni': jac,
        'len_diff': len_diff,
        'len_ratio': len_ratio,
        'tok_diff': tok_diff,
        'tok_ratio': tok_ratio,
    })
    feat_df = pd.concat([feat_df, sec_dum.reset_index(drop=True)], axis=1)
    return feat_df

Xlex_tr = build_feats(train)
Xlex_te = build_feats(test)

# Load OOF channels
o_e5    = pd.read_csv('oof_e5_base.csv').rename(columns={'oof':'e5'})
o_mp    = pd.read_csv('oof_mpnet_st.csv').rename(columns={'oof':'mpnet'})
o_bge   = pd.read_csv('oof_bge.csv').rename(columns={'oof':'bge'})
o_tfidf = pd.read_csv('oof_tfidf_ridge.csv').rename(columns={'oof':'tfidf'})
o_ce    = pd.read_csv('oof_ce_stsb.csv').rename(columns={'oof':'ce'})
df_oof = train[['id','score','fold']].rename(columns={'score':'y'}).merge(o_e5, on='id').merge(o_mp, on='id').merge(o_bge, on='id').merge(o_tfidf, on='id').merge(o_ce, on='id')
use_cols = ['e5','mpnet','bge','tfidf','ce']
try:
    o_e5a = pd.read_csv('oof_e5_asym.csv').rename(columns={'oof':'e5_asym'})
    df_oof = df_oof.merge(o_e5a, on='id')
    use_cols.append('e5_asym')
except Exception:
    pass

# Merge lexical features
df_oof = pd.concat([df_oof.reset_index(drop=True), Xlex_tr.reset_index(drop=True)], axis=1)
y_all = df_oof['y'].values.astype(np.float32)
fold_arr = df_oof['fold'].values
X_base = df_oof[use_cols].values.astype(np.float32)
X_lex = Xlex_tr.values.astype(np.float32)
X_all = np.concatenate([X_base, X_lex], axis=1)

oof_stack = np.zeros_like(y_all, dtype=np.float32)
alpha = 1.0
for f in sorted(np.unique(fold_arr)):
    tr_idx = np.where(fold_arr != f)[0]
    va_idx = np.where(fold_arr == f)[0]
    mdl = Ridge(alpha=alpha, fit_intercept=True, random_state=42)
    mdl.fit(X_all[tr_idx], y_all[tr_idx])
    oof_stack[va_idx] = mdl.predict(X_all[va_idx]).astype(np.float32)
    print(f'Feat-Stack fold {int(f)} pearson=', round(float(pearsonr(oof_stack[va_idx], y_all[va_idx])[0]), 6))

p = pearsonr(oof_stack, y_all)[0]
print('OOF Pearson Feature-aug Ridge stack (RAW):', round(p,6), 'with cols', use_cols, '+ lexical')
pd.DataFrame({'id': df_oof['id'], 'oof': oof_stack}).to_csv('oof_stack_feat_ridge.csv', index=False)

# Build test matrix
sub1 = pd.read_csv('submission_e5_base.csv').rename(columns={'score':'e5'})
sub2 = pd.read_csv('submission_mpnet_st.csv').rename(columns={'score':'mpnet'})
sub3 = pd.read_csv('submission_bge.csv').rename(columns={'score':'bge'})
sub4 = pd.read_csv('submission_tfidf.csv').rename(columns={'score':'tfidf'})
sub5 = pd.read_csv('submission_ce_stsb.csv').rename(columns={'score':'ce'})
sub = sub1.merge(sub2, on='id').merge(sub3, on='id').merge(sub4, on='id').merge(sub5, on='id')
if 'e5_asym' in use_cols:
    try:
        sub6 = pd.read_csv('submission_e5_asym.csv').rename(columns={'score':'e5_asym'})
        sub = sub.merge(sub6, on='id')
    except Exception:
        use_cols = [c for c in use_cols if c != 'e5_asym']

X_base_te = sub[use_cols].values.astype(np.float32)
X_te = np.concatenate([X_base_te, Xlex_te.values.astype(np.float32)], axis=1)
mdl_full = Ridge(alpha=alpha, fit_intercept=True, random_state=42)
mdl_full.fit(X_all, y_all)
test_pred = mdl_full.predict(X_te).astype(np.float32)
pd.DataFrame({'id': sub['id'], 'score': np.clip(test_pred, 0.0, 1.0)}).to_csv('submission_stack_feat_ridge.csv', index=False)
print('Saved oof_stack_feat_ridge.csv and submission_stack_feat_ridge.csv; elapsed', round((time.time()-t0)/60,2), 'min')

Feat-Stack fold 0 pearson= 0.675809
Feat-Stack fold 1 pearson= 0.672463
Feat-Stack fold 2 pearson= 0.665684
Feat-Stack fold 3 pearson= 0.677915
Feat-Stack fold 4 pearson= 0.664339
OOF Pearson Feature-aug Ridge stack (RAW): 0.671029 with cols ['e5', 'mpnet', 'bge', 'tfidf', 'ce', 'e5_asym'] + lexical
Saved oof_stack_feat_ridge.csv and submission_stack_feat_ridge.csv; elapsed 0.01 min


In [41]:
# Set best current submission (feature-augmented ridge stack) as submission.csv
import pandas as pd
src = 'submission_stack_feat_ridge.csv'
dst = 'submission.csv'
df = pd.read_csv(src)
df.to_csv(dst, index=False)
print('submission.csv updated from', src, '-> shape', df.shape)

submission.csv updated from submission_stack_feat_ridge.csv -> shape (3648, 2)


In [47]:
# CPU feature extraction: e5-asym and mpnet-asym with asymmetric CPC templates; per-fold whitening (trimmed ks); distances = cosine + L1 + L2
# CPC fallback: raw CPC code when no 4/3/2 prefix match (not SECTION). Saves per-fold distance features for train/test.
import time, os, gc, re, numpy as np, pandas as pd
from sklearn.decomposition import PCA
from scipy.stats import pearsonr
from sentence_transformers import SentenceTransformer

SEED = 42
np.random.seed(SEED)
os.environ.setdefault('OMP_NUM_THREADS', '8')
os.environ.setdefault('MKL_NUM_THREADS', '8')

# --- CPC mapping helpers (4->3->2 prefix fallback, else RAW CODE) ---
CPC_DESCRIPTIONS = {
    'A61B':'Diagnosis; Surgery; Identification','A61F':'Filters; Prostheses; Braces','A61K':'Preparations for medical, dental, or toilet purposes','A61M':'Devices for introducing/circulating media into the body',
    'B01D':'Separation; Sorption; Membranes','B29C':'Shaping or joining of plastics','B32B':'Layered products','B60K':'Arrangement or mounting of vehicle power plant or auxiliaries','B60R':'Vehicles, vehicle fittings, or vehicle parts','B60T':'Vehicle brake control systems or parts',
    'C01B':'Non-metallic elements; Compounds thereof','C04B':'Cement; Concrete; Ceramics','C07C':'Acyclic or carbocyclic compounds','C07D':'Heterocyclic compounds','C07K':'Peptides','C08F':'Macromolecular compounds by polymerising monomers','C08L':'Compositions of macromolecular compounds','C09K':'Materials for miscellaneous applications',
    'C12N':'Microorganisms or enzymes; Genetic engineering','C12P':'Fermentation or enzyme-using processes','C12Q':'Measuring/testing involving enzymes or microorganisms',
    'E04B':'Building constructions; Walls; Floors; Roofs','F16B':'Screws; Rivets; Joints','F16F':'Springs; Shock-absorbers; Dampers','F16L':'Pipes; Joints or fittings for pipes',
    'G01N':'Investigating or analysing materials','G02B':'Optical elements, systems, or apparatus','G02F':'Control of light','G06F':'Electric digital data processing','G06K':'Recognition/Presentation of data','G06Q':'Data processing for administrative/financial/managerial',
    'H01L':'Semiconductor devices','H01M':'Batteries; Fuel cells','H02J':'Power supply/distribution','H04L':'Transmission of digital information','H04N':'Pictorial communication','H04W':'Wireless communication networks','H05K':'Printed circuits; Electric assemblies',
    'A01':'Agriculture; Forestry; Animal husbandry; Hunting; Trapping; Fishing','A21':'Baking; Edible doughs','A23':'Foods or foodstuffs; Treatment thereof','A24':'Tobacco; Cigars; Cigarettes; Simulated smoking devices; Smokers\' requisites','A41':'Wearing apparel','A42':'Headwear','A43':'Footwear','A45':'Hand or travelling articles','A46':'Brushware','A47':'Furniture; Domestic articles or appliances; Coffee mills; Spice mills; Suction cleaners','A61':'Medical or veterinary science; Hygiene','A62':'Life-saving; Fire-fighting','A63':'Sports; Games; Amusements',
    'B01':'Physical or chemical processes or apparatus','B02':'Crushing; pulverising; disintegrating','B03':'Separation of solid materials using liquids or jigs','B04':'Centrifugal apparatus or machines','B05':'Spraying or atomising; Applying fluent materials to surfaces','B06':'Mechanical vibrations','B07':'Separating solids; Sorting','B08':'Cleaning','B09':'Disposal of solid waste; Soil reclamation','B21':'Mechanical metal-working without removing material','B22':'Casting; Powder metallurgy','B23':'Machine tools; Metal-working n.e.c.','B24':'Grinding; Polishing','B25':'Hand tools; Portable power-driven tools; Manipulators','B26':'Hand cutting tools; Cutting; Severing','B27':'Working/preserving wood; Nailing or stapling','B28':'Working cement, clay, or stone','B29':'Working of plastics; Plastic state substances','B30':'Presses','B31':'Making paper-like articles','B32':'Layered products','B33':'Additive manufacturing','B41':'Printing; Typewriters; Stamps','B42':'Bookbinding; Albums; Files','B43':'Writing or drawing implements; Bureau accessories','B44':'Decorative arts','B60':'Vehicles in general','B61':'Railways','B62':'Land vehicles not on rails','B63':'Ships or waterborne vessels','B64':'Aircraft; Aviation; Cosmonautics','B65':'Conveying; Packing; Storing','B66':'Hoisting; Lifting; Hauling','B67':'Opening/closing containers; Liquid handling','B68':'Saddlery; Upholstery','B81':'Microstructural technology','B82':'Nanotechnology',
    'C01':'Inorganic chemistry','C02':'Treatment of water, wastewater, sewage, sludge','C03':'Glass; Mineral or slag wool','C04':'Cements; Concrete; Ceramics; Refractories','C05':'Fertilisers','C06':'Explosives; Matches','C07':'Organic chemistry','C08':'Organic macromolecular compounds; Polymers','C09':'Dyes; Paints; Polishes; Adhesives; Misc compositions','C10':'Petroleum/gas/coke industries; Fuels; Lubricants; Peat','C11':'Oils, fats, waxes; Detergents; Candles','C12':'Biochemistry; Microbiology; Enzymology; Fermentation','C13':'Sugar industry','C14':'Skins; Hides; Leather',
    'C21':'Metallurgy of iron','C22':'Metallurgy; Alloys; Treatment of non-ferrous metals','C23':'Coating metallic material; Chemical surface treatment','C25':'Electrolytic/electrophoretic processes','C30':'Crystal growth','C40':'Combinatorial technology',
    'D01':'Threads or fibres; Spinning','D02':'Yarns; Finishing; Warping or beaming','D03':'Weaving','D04':'Braiding; Lace-making; Knitting; Non-wovens','D05':'Sewing; Embroidering; Tufting','D06':'Textile treatment; Laundering; Flexible materials','D07':'Ropes; Cables other than electric','D21':'Paper-making; Cellulose production',
    'E01':'Road/rail/bridge construction','E02':'Hydraulic engineering; Foundations; Soil shifting','E03':'Water supply; Sewerage','E04':'Building','E05':'Locks; Keys; Fittings; Safes','E06':'Doors; Windows; Shutters; Ladders','E21':'Earth/rock drilling; Mining',
    'F01':'Machines or engines in general','F02':'Combustion engines; Hot-gas plants','F03':'Machines or engines for liquids; Wind/spring motors','F04':'Positive-displacement machines; Pumps','F15':'Fluid-pressure actuators; Hydraulics; Pneumatics','F16':'Engineering elements or units','F17':'Storing/distributing gases or liquids','F21':'Lighting','F22':'Steam generation','F23':'Combustion apparatus; Processes','F24':'Heating; Ranges; Ventilating','F25':'Refrigeration or cooling; Heat pumps','F26':'Drying','F27':'Furnaces; Kilns; Ovens','F28':'Heat exchange in general','F41':'Weapons','F42':'Ammunition; Blasting',
    'G01':'Measuring; Testing','G02':'Optics','G03':'Photography; Cinematography; Non-optical waves','G04':'Horology','G05':'Controlling; Regulating','G06':'Computing; Calculating; Counting','G07':'Checking-devices','G08':'Signalling','G09':'Educating; Cryptography; Display; Advertising; Seals','G10':'Musical instruments; Acoustics','G11':'Information storage','G12':'Instrument details','G16':'ICT adapted for specific fields','G21':'Nuclear physics; Nuclear engineering',
    'H01':'Electric elements','H02':'Electric power: generation, conversion, distribution','H03':'Electronic circuitry','H04':'Electric communication technique','H05':'Electric techniques not otherwise provided for'
}

def get_cpc_text(code: str) -> str:
    if not isinstance(code, str) or not code:
        return 'GENERAL'
    code = code.strip().upper()
    for k in (4,3,2):
        pref = code[:k]
        if pref in CPC_DESCRIPTIONS:
            return CPC_DESCRIPTIONS[pref]
    # Fallback: use raw code string (domain specificity) instead of SECTION
    return code

def truncate_words(text: str, n: int = 10) -> str:
    w = text.split() if isinstance(text, str) else []
    return ' '.join(w[:n])

def l2n(x: np.ndarray, eps: float = 1e-12) -> np.ndarray:
    n = np.linalg.norm(x, axis=1, keepdims=True) + eps
    return x / n

def fit_whitener(X: np.ndarray, k: int):
    mu = X.mean(axis=0, keepdims=True)
    Xc = X - mu
    if k > 0:
        pca = PCA(n_components=k, random_state=SEED).fit(Xc)
        P = pca.components_
    else:
        P = None
    def transform(Y: np.ndarray) -> np.ndarray:
        Yc = Y - mu
        if P is not None:
            Z = Yc - (Yc @ P.T) @ P
        else:
            Z = Yc
        return l2n(Z)
    return transform

def cosine_rows(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    return np.sum(a*b, axis=1).astype(np.float32)

def l1_rows(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    return np.sum(np.abs(a-b), axis=1).astype(np.float32)

def l2_rows(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    return np.sqrt(np.sum((a-b)*(a-b), axis=1)).astype(np.float32)

def build_e5_asym_texts(df: pd.DataFrame):
    cpc_full = df['context'].astype(str).map(get_cpc_text)
    cpc_10w = cpc_full.map(lambda x: truncate_words(x, 10))
    a = ("passage: domain: " + cpc_10w + " [SEP] " + df['anchor'].astype(str)).values
    b = ("passage: " + df['target'].astype(str)).values
    return a, b

def build_mpnet_asym_texts(df: pd.DataFrame):
    cpc_full = df['context'].astype(str).map(get_cpc_text)
    cpc_10w = cpc_full.map(lambda x: truncate_words(x, 10))
    a = ("domain: " + cpc_10w + " [SEP] " + df['anchor'].astype(str)).values
    b = (df['target'].astype(str)).values
    return a, b

def encode_unique(texts: np.ndarray, model_name: str, batch_size: int = 48, max_len: int = 256):
    st_model = SentenceTransformer(model_name, device='cpu')
    st_model.max_seq_length = max_len
    embs = st_model.encode(texts.tolist(), batch_size=batch_size, normalize_embeddings=True, convert_to_numpy=True, show_progress_bar=True)
    return embs.astype(np.float32)

t0_all = time.time()
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
folds = pd.read_csv('folds_by_id.csv')
train = train.merge(folds, on='id', how='left', validate='one_to_one')
assert (train['fold']>=0).all()

def process_channel(name: str, builder_fn, model_name: str, k_list=(0,), batch_size=48, max_len=256):
    print(f'\n=== Processing {name} ===', flush=True)
    tr_a, tr_b = builder_fn(train)
    te_a, te_b = builder_fn(test)
    all_texts = np.concatenate([tr_a, tr_b, te_a, te_b])
    uniq_texts, inv = np.unique(all_texts, return_inverse=True)
    n_tr, n_te = len(train), len(test)
    n_a_tr, n_b_tr = len(tr_a), len(tr_b)
    print(name, 'unique texts:', len(uniq_texts))
    embs = encode_unique(uniq_texts, model_name=model_name, batch_size=batch_size, max_len=max_len)
    idx = 0
    ia_tr_a = inv[idx:idx+n_a_tr]; idx += n_a_tr
    ia_tr_b = inv[idx:idx+n_b_tr]; idx += n_b_tr
    ia_te_a = inv[idx:idx+n_te]; idx += n_te
    ia_te_b = inv[idx:idx+n_te]; idx += n_te
    tr_a_emb = embs[ia_tr_a]; tr_b_emb = embs[ia_tr_b]
    te_a_emb = embs[ia_te_a]; te_b_emb = embs[ia_te_b]
    del embs; gc.collect()
    fold_arr = train['fold'].values
    for k in k_list:
        print(f'{name}: per-fold whitening k={k}', flush=True)
        # Prepare holders
        cos_oof = np.zeros(n_tr, dtype=np.float32)
        l1_oof = np.zeros(n_tr, dtype=np.float32)
        l2_oof = np.zeros(n_tr, dtype=np.float32)
        cos_te_agg = []
        l1_te_agg = []
        l2_te_agg = []
        for f in sorted(np.unique(fold_arr)):
            f0 = time.time()
            tr_idx = np.where(fold_arr != f)[0]
            va_idx = np.where(fold_arr == f)[0]
            W = fit_whitener(np.vstack([tr_a_emb[tr_idx], tr_b_emb[tr_idx]]), k=k)
            wa_tr_a = W(tr_a_emb[tr_idx]); wb_tr_b = W(tr_b_emb[tr_idx])
            wa_va_a = W(tr_a_emb[va_idx]); wb_va_b = W(tr_b_emb[va_idx])
            wa_te_a = W(te_a_emb);        wb_te_b = W(te_b_emb)
            # Distances
            cos_va = cosine_rows(wa_va_a, wb_va_b); cos_te = cosine_rows(wa_te_a, wb_te_b)
            l1_va  = l1_rows(wa_va_a, wb_va_b);     l1_te  = l1_rows(wa_te_a, wb_te_b)
            l2_va  = l2_rows(wa_va_a, wb_va_b);     l2_te  = l2_rows(wa_te_a, wb_te_b)
            cos_oof[va_idx] = cos_va
            l1_oof[va_idx] = l1_va
            l2_oof[va_idx] = l2_va
            cos_te_agg.append(cos_te);
            l1_te_agg.append(l1_te);
            l2_te_agg.append(l2_te);
            # quick log
            if 'score' in train.columns:
                yv = train.loc[va_idx, 'score'].values.astype(np.float32)
                rp = pearsonr(cos_va, yv)[0]
                print(f'  fold {int(f)}: raw-cos pearson={rp:.6f} | elapsed {time.time()-f0:.1f}s', flush=True)
        # Aggregate test (mean over fold whiteners effect)
        cos_te_mean = np.mean(np.vstack(cos_te_agg), axis=0).astype(np.float32)
        l1_te_mean = np.mean(np.vstack(l1_te_agg), axis=0).astype(np.float32)
        l2_te_mean = np.mean(np.vstack(l2_te_agg), axis=0).astype(np.float32)
        # Save
        tr_out = pd.DataFrame({'id': train['id'], f'{name}_cos_k{k}': cos_oof, f'{name}_l1_k{k}': l1_oof, f'{name}_l2_k{k}': l2_oof})
        te_out = pd.DataFrame({'id': test['id'], f'{name}_cos_k{k}': cos_te_mean, f'{name}_l1_k{k}': l1_te_mean, f'{name}_l2_k{k}': l2_te_mean})
        tr_path = f'feats_{name}_k{k}.csv'
        te_path = f'feats_{name}_k{k}_test.csv'
        tr_out.to_csv(tr_path, index=False)
        te_out.to_csv(te_path, index=False)
        print('Saved', tr_path, te_path, flush=True)
    # cleanup
    del tr_a_emb, tr_b_emb, te_a_emb, te_b_emb
    gc.collect()

# Run e5-asym (fixed k=0)
process_channel(name='e5_asym', builder_fn=build_e5_asym_texts, model_name='intfloat/e5-base-v2', k_list=(0,), batch_size=48, max_len=256)

# Run mpnet-asym (fixed k=1)
process_channel(name='mpnet_asym', builder_fn=build_mpnet_asym_texts, model_name='sentence-transformers/all-mpnet-base-v2', k_list=(1,), batch_size=48, max_len=256)

print('Done feature extraction for e5-asym and mpnet-asym in', round((time.time()-t0_all)/60,2), 'min')


=== Processing e5_asym ===


e5_asym unique texts: 31039


Batches:   0%|          | 0/647 [00:00<?, ?it/s]

Batches:   0%|          | 1/647 [00:00<04:00,  2.69it/s]

Batches:   0%|          | 2/647 [00:00<03:27,  3.11it/s]

...

  fold 2: raw-cos pearson=0.525190 | elapsed 0.6s


  fold 3: raw-cos pearson=0.534588 | elapsed 0.6s


  fold 4: raw-cos pearson=0.543447 | elapsed 0.6s


Saved feats_mpnet_asym_k1.csv feats_mpnet_asym_k1_test.csv


Done feature extraction for e5-asym and mpnet-asym in 2.04 min


In [52]:
# BGE distance features on CPU: residualized CPC channel with alpha in {0.25,0.30,0.35}, whitening k in {0,1}; plus no-CPC symmetric channel (k in {1})
# Saves per-fold cosine + L1 + L2 distances to CSV for train and per-fold test (suffix _tef{f})
import time, os, gc, numpy as np, pandas as pd, re
from sklearn.decomposition import PCA
from scipy.stats import pearsonr
from sentence_transformers import SentenceTransformer

SEED = 42
np.random.seed(SEED)
os.environ.setdefault('OMP_NUM_THREADS', '8')
os.environ.setdefault('MKL_NUM_THREADS', '8')

# --- CPC mapping (reuse) ---
CPC_DESCRIPTIONS = {
    'A61B':'Diagnosis; Surgery; Identification','A61F':'Filters; Prostheses; Braces','A61K':'Preparations for medical, dental, or toilet purposes','A61M':'Devices for introducing/circulating media into the body',
    'B01D':'Separation; Sorption; Membranes','B29C':'Shaping or joining of plastics','B32B':'Layered products','B60K':'Arrangement or mounting of vehicle power plant or auxiliaries','B60R':'Vehicles, vehicle fittings, or vehicle parts','B60T':'Vehicle brake control systems or parts',
    'C01B':'Non-metallic elements; Compounds thereof','C04B':'Cement; Concrete; Ceramics','C07C':'Acyclic or carbocyclic compounds','C07D':'Heterocyclic compounds','C07K':'Peptides','C08F':'Macromolecular compounds by polymerising monomers','C08L':'Compositions of macromolecular compounds','C09K':'Materials for miscellaneous applications',
    'C12N':'Microorganisms or enzymes; Genetic engineering','C12P':'Fermentation or enzyme-using processes','C12Q':'Measuring/testing involving enzymes or microorganisms',
    'E04B':'Building constructions; Walls; Floors; Roofs','F16B':'Screws; Rivets; Joints','F16F':'Springs; Shock-absorbers; Dampers','F16L':'Pipes; Joints or fittings for pipes',
    'G01N':'Investigating or analysing materials','G02B':'Optical elements, systems, or apparatus','G02F':'Control of light','G06F':'Electric digital data processing','G06K':'Recognition/Presentation of data','G06Q':'Data processing for administrative/financial/managerial',
    'H01L':'Semiconductor devices','H01M':'Batteries; Fuel cells','H02J':'Power supply/distribution','H04L':'Transmission of digital information','H04N':'Pictorial communication','H04W':'Wireless communication networks','H05K':'Printed circuits; Electric assemblies',
    'A01':'Agriculture; Forestry; Animal husbandry; Hunting; Trapping; Fishing','A21':'Baking; Edible doughs','A23':'Foods or foodstuffs; Treatment thereof','A24':'Tobacco; Cigars; Cigarettes; Simulated smoking devices; Smokers\' requisites','A41':'Wearing apparel','A42':'Headwear','A43':'Footwear','A45':'Hand or travelling articles','A46':'Brushware','A47':'Furniture; Domestic articles or appliances; Coffee mills; Spice mills; Suction cleaners','A61':'Medical or veterinary science; Hygiene','A62':'Life-saving; Fire-fighting','A63':'Sports; Games; Amusements',
    'B01':'Physical or chemical processes or apparatus','B02':'Crushing; pulverising; disintegrating','B03':'Separation of solid materials using liquids or jigs','B04':'Centrifugal apparatus or machines','B05':'Spraying; atomising; Applying fluent materials','B06':'Mechanical vibrations','B07':'Separating solids; Sorting','B08':'Cleaning','B09':'Disposal of solid waste; Soil reclamation','B21':'Mechanical metal-working without removing material','B22':'Casting; Powder metallurgy','B23':'Machine tools; Metal-working n.e.c.','B24':'Grinding; Polishing','B25':'Hand tools; Portable power-driven tools; Manipulators','B26':'Hand cutting tools; Cutting; Severing','B27':'Working/preserving wood; Nailing or stapling','B28':'Working cement, clay, or stone','B29':'Working of plastics; Plastic state substances','B30':'Presses','B31':'Making paper-like articles','B32':'Layered products','B33':'Additive manufacturing','B41':'Printing; Typewriters; Stamps','B42':'Bookbinding; Albums; Files','B43':'Writing or drawing implements; Bureau accessories','B44':'Decorative arts','B60':'Vehicles in general','B61':'Railways','B62':'Land vehicles not on rails','B63':'Ships or waterborne vessels','B64':'Aircraft; Aviation; Cosmonautics','B65':'Conveying; Packing; Storing','B66':'Hoisting; Lifting; Hauling','B67':'Opening/closing containers; Liquid handling','B68':'Saddlery; Upholstery','B81':'Microstructural technology','B82':'Nanotechnology',
    'C01':'Inorganic chemistry','C02':'Treatment of water, wastewater, sewage, sludge','C03':'Glass; Mineral or slag wool','C04':'Cements; Concrete; Ceramics; Refractories','C05':'Fertilisers','C06':'Explosives; Matches','C07':'Organic chemistry','C08':'Organic macromolecular compounds; Polymers','C09':'Dyes; Paints; Polishes; Adhesives; Misc compositions','C10':'Petroleum/gas/coke industries; Fuels; Lubricants; Peat','C11':'Oils, fats, waxes; Detergents; Candles','C12':'Biochemistry; Microbiology; Enzymology; Fermentation','C13':'Sugar industry','C14':'Skins; Hides; Leather',
    'C21':'Metallurgy of iron','C22':'Metallurgy; Alloys; Treatment of non-ferrous metals','C23':'Coating metallic material; Chemical surface treatment','C25':'Electrolytic/electrophoretic processes','C30':'Crystal growth','C40':'Combinatorial technology',
    'D01':'Threads or fibres; Spinning','D02':'Yarns; Finishing; Warping or beaming','D03':'Weaving','D04':'Braiding; Lace-making; Knitting; Non-wovens','D05':'Sewing; Embroidering; Tufting','D06':'Textile treatment; Laundering; Flexible materials','D07':'Ropes; Cables other than electric','D21':'Paper-making; Cellulose production',
    'E01':'Road/rail/bridge construction','E02':'Hydraulic engineering; Foundations; Soil shifting','E03':'Water supply; Sewerage','E04':'Building','E05':'Locks; Keys; Fittings; Safes','E06':'Doors; Windows; Shutters; Ladders','E21':'Earth/rock drilling; Mining',
    'F01':'Machines or engines in general','F02':'Combustion engines; Hot-gas plants','F03':'Machines or engines for liquids; Wind/spring motors','F04':'Positive-displacement machines; Pumps','F15':'Fluid-pressure actuators; Hydraulics; Pneumatics','F16':'Engineering elements or units','F17':'Storing/distributing gases or liquids','F21':'Lighting','F22':'Steam generation','F23':'Combustion apparatus; Processes','F24':'Heating; Ranges; Ventilating','F25':'Refrigeration or cooling; Heat pumps','F26':'Drying','F27':'Furnaces; Kilns; Ovens','F28':'Heat exchange in general','F41':'Weapons','F42':'Ammunition; Blasting',
    'G01':'Measuring; Testing','G02':'Optics','G03':'Photography; Cinematography; Non-optical waves','G04':'Horology','G05':'Controlling; Regulating','G06':'Computing; Calculating; Counting','G07':'Checking-devices','G08':'Signalling','G09':'Educating; Cryptography; Display; Advertising; Seals','G10':'Musical instruments; Acoustics','G11':'Information storage','G12':'Instrument details','G16':'ICT for specific fields','G21':'Nuclear physics; Nuclear engineering',
    'H01':'Electric elements','H02':'Electric power: generation, conversion, distribution','H03':'Electronic circuitry','H04':'Electric communication technique','H05':'Electric techniques not otherwise provided for'
}
SECTION = {'A':'Human necessities','B':'Performing operations; Transporting','C':'Chemistry; Metallurgy','D':'Textiles; Paper','E':'Fixed constructions','F':'Mechanical engineering; Lighting; Heating; Weapons; Blasting','G':'Physics','H':'Electricity'}

def get_cpc_text(code: str) -> str:
    if not isinstance(code, str) or not code:
        return 'GENERAL'
    code = code.strip().upper()
    for k in (4,3,2):
        pref = code[:k]
        if pref in CPC_DESCRIPTIONS:
            return CPC_DESCRIPTIONS[pref]
    # Fallback: use raw code string (domain specificity) instead of SECTION
    return code

def truncate_words(text: str, n: int = 10) -> str:
    w = text.split() if isinstance(text, str) else []
    return ' '.join(w[:n])

def l2n(x: np.ndarray, eps: float = 1e-12) -> np.ndarray:
    n = np.linalg.norm(x, axis=1, keepdims=True) + eps
    return x / n

def fit_whitener(X: np.ndarray, k: int):
    mu = X.mean(axis=0, keepdims=True)
    Xc = X - mu
    if k > 0:
        pca = PCA(n_components=k, random_state=SEED).fit(Xc)
        P = pca.components_
    else:
        P = None
    def transform(Y: np.ndarray) -> np.ndarray:
        Yc = Y - mu
        if P is not None:
            Z = Yc - (Yc @ P.T) @ P
        else:
            Z = Yc
        return l2n(Z)
    return transform

def cosine_rows(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    return np.sum(a*b, axis=1).astype(np.float32)

def l1_rows(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    return np.sum(np.abs(a-b), axis=1).astype(np.float32)

def l2_rows(a: np.ndarray, b: np.ndarray) -> np.ndarray:
    return np.sqrt(np.sum((a-b)*(a-b), axis=1)).astype(np.float32)

def build_bge_with_cpc(df: pd.DataFrame):
    cpc_full = df['context'].astype(str).map(get_cpc_text)
    cpc_10w = cpc_full.map(lambda x: truncate_words(x, 10))
    a = ("Represent the sentence for similarity: " + cpc_10w + " [SEP] " + df['anchor'].astype(str)).values
    b = ("Represent the sentence for similarity: " + cpc_10w + " [SEP] " + df['target'].astype(str)).values
    c = ("Represent the sentence for similarity: " + cpc_10w).values  # CPC-only
    return a, b, c

def build_bge_nocpc(df: pd.DataFrame):
    a = ("Represent the sentence for similarity: " + df['anchor'].astype(str)).values
    b = ("Represent the sentence for similarity: " + df['target'].astype(str)).values
    return a, b

def encode_unique(texts: np.ndarray, model_name: str, batch_size: int = 48, max_len: int = 256):
    st_model = SentenceTransformer(model_name, device='cpu')
    st_model.max_seq_length = max_len
    embs = st_model.encode(texts.tolist(), batch_size=batch_size, normalize_embeddings=True, convert_to_numpy=True, show_progress_bar=True)
    return embs.astype(np.float32)

t0_all = time.time()
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
folds = pd.read_csv('folds_by_id.csv')
train = train.merge(folds, on='id', how='left', validate='one_to_one')
assert (train['fold']>=0).all()

# --- BGE residual channel ---
def process_bge_residual(model_name='BAAI/bge-base-en-v1.5', alphas=(0.25,0.30,0.35), k_list=(0,1), batch_size=48, max_len=256):
    print('\n=== Processing BGE residual ===', flush=True)
    tr_a, tr_b, tr_c = build_bge_with_cpc(train)
    te_a, te_b, te_c = build_bge_with_cpc(test)
    all_texts = np.concatenate([tr_a, tr_b, tr_c, te_a, te_b, te_c])
    uniq_texts, inv = np.unique(all_texts, return_inverse=True)
    print('BGE residual unique texts:', len(uniq_texts))
    embs = encode_unique(uniq_texts, model_name=model_name, batch_size=batch_size, max_len=max_len)
    n_tr, n_te = len(train), len(test)
    n_a_tr, n_b_tr, n_c_tr = len(tr_a), len(tr_b), len(tr_c)
    idx = 0
    ia_tr_a = inv[idx:idx+n_a_tr]; idx += n_a_tr
    ia_tr_b = inv[idx:idx+n_b_tr]; idx += n_b_tr
    ia_tr_c = inv[idx:idx+n_c_tr]; idx += n_c_tr
    ia_te_a = inv[idx:idx+n_te]; idx += n_te
    ia_te_b = inv[idx:idx+n_te]; idx += n_te
    ia_te_c = inv[idx:idx+n_te]; idx += n_te
    tr_a_emb0 = embs[ia_tr_a]; tr_b_emb0 = embs[ia_tr_b]; tr_c_emb0 = embs[ia_tr_c]
    te_a_emb0 = embs[ia_te_a]; te_b_emb0 = embs[ia_te_b]; te_c_emb0 = embs[ia_te_c]
    del embs; gc.collect()
    fold_arr = train['fold'].values
    for a in alphas:
        # residualize
        tr_a_emb = l2n(tr_a_emb0) - a * l2n(tr_c_emb0); tr_a_emb = l2n(tr_a_emb)
        tr_b_emb = l2n(tr_b_emb0) - a * l2n(tr_c_emb0); tr_b_emb = l2n(tr_b_emb)
        te_a_emb = l2n(te_a_emb0) - a * l2n(te_c_emb0); te_a_emb = l2n(te_a_emb)
        te_b_emb = l2n(te_b_emb0) - a * l2n(te_c_emb0); te_b_emb = l2n(te_b_emb)
        for k in k_list:
            print(f'BGE residual: alpha={a} k={k}', flush=True)
            cos_oof = np.zeros(n_tr, dtype=np.float32)
            l1_oof  = np.zeros(n_tr, dtype=np.float32)
            l2_oof  = np.zeros(n_tr, dtype=np.float32)
            # store test per fold
            te_cos_pf = {}
            te_l1_pf  = {}
            te_l2_pf  = {}
            for f in sorted(np.unique(fold_arr)):
                f0 = time.time()
                tr_idx = np.where(fold_arr != f)[0]
                va_idx = np.where(fold_arr == f)[0]
                W = fit_whitener(np.vstack([tr_a_emb[tr_idx], tr_b_emb[tr_idx]]), k=k)
                wa_tr_a = W(tr_a_emb[tr_idx]); wb_tr_b = W(tr_b_emb[tr_idx])
                wa_va_a = W(tr_a_emb[va_idx]); wb_va_b = W(tr_b_emb[va_idx])
                wa_te_a = W(te_a_emb);        wb_te_b = W(te_b_emb)
                cos_va = cosine_rows(wa_va_a, wb_va_b); l1_va = l1_rows(wa_va_a, wb_va_b); l2_va = l2_rows(wa_va_a, wb_va_b)
                cos_te = cosine_rows(wa_te_a, wb_te_b); l1_te = l1_rows(wa_te_a, wb_te_b); l2_te = l2_rows(wa_te_a, wb_te_b)
                cos_oof[va_idx] = cos_va; l1_oof[va_idx] = l1_va; l2_oof[va_idx] = l2_va
                te_cos_pf[f] = cos_te; te_l1_pf[f] = l1_te; te_l2_pf[f] = l2_te
                yv = train.loc[va_idx, 'score'].values.astype(np.float32)
                print(f'  fold {int(f)}: raw-cos pearson={pearsonr(cos_va, yv)[0]:.6f} | {time.time()-f0:.1f}s', flush=True)
            # Save train OOF
            tr_out = pd.DataFrame({'id': train['id'],
                                   f'bge_res_a{str(a).replace(".","")}_cos_k{k}': cos_oof,
                                   f'bge_res_a{str(a).replace(".","")}_l1_k{k}': l1_oof,
                                   f'bge_res_a{str(a).replace(".","")}_l2_k{k}': l2_oof})
            tr_path = f'feats_bge_res_a{str(a).replace(".","")}_k{k}.csv'
            tr_out.to_csv(tr_path, index=False)
            # Save test per-fold columns with suffix _tef{f}
            te_out = pd.DataFrame({'id': test['id']})
            for f in sorted(te_cos_pf.keys()):
                te_out[f'bge_res_a{str(a).replace(".","")}_cos_k{k}_tef{int(f)}'] = te_cos_pf[f].astype(np.float32)
                te_out[f'bge_res_a{str(a).replace(".","")}_l1_k{k}_tef{int(f)}']  = te_l1_pf[f].astype(np.float32)
                te_out[f'bge_res_a{str(a).replace(".","")}_l2_k{k}_tef{int(f)}']  = te_l2_pf[f].astype(np.float32)
            te_path = f'feats_bge_res_a{str(a).replace(".","")}_k{k}_test.csv'
            te_out.to_csv(te_path, index=False)
            print('Saved', tr_path, te_path, flush=True)
    del tr_a_emb0, tr_b_emb0, tr_c_emb0, te_a_emb0, te_b_emb0, te_c_emb0
    gc.collect()

# --- BGE no-CPC channel ---
def process_bge_nocpc(model_name='BAAI/bge-base-en-v1.5', k_list=(1,), batch_size=48, max_len=256):
    print('\n=== Processing BGE no-CPC ===', flush=True)
    tr_a, tr_b = build_bge_nocpc(train)
    te_a, te_b = build_bge_nocpc(test)
    all_texts = np.concatenate([tr_a, tr_b, te_a, te_b])
    uniq_texts, inv = np.unique(all_texts, return_inverse=True)
    print('BGE no-CPC unique texts:', len(uniq_texts))
    embs = encode_unique(uniq_texts, model_name=model_name, batch_size=batch_size, max_len=max_len)
    n_tr, n_te = len(train), len(test); n_a_tr, n_b_tr = len(tr_a), len(tr_b)
    idx = 0
    ia_tr_a = inv[idx:idx+n_a_tr]; idx += n_a_tr
    ia_tr_b = inv[idx:idx+n_b_tr]; idx += n_b_tr
    ia_te_a = inv[idx:idx+n_te]; idx += n_te
    ia_te_b = inv[idx:idx+n_te]; idx += n_te
    tr_a_emb = embs[ia_tr_a]; tr_b_emb = embs[ia_tr_b]
    te_a_emb = embs[ia_te_a]; te_b_emb = embs[ia_te_b]
    del embs; gc.collect()
    fold_arr = train['fold'].values
    for k in k_list:
        print(f'BGE no-CPC: k={k}', flush=True)
        cos_oof = np.zeros(n_tr, dtype=np.float32)
        l1_oof  = np.zeros(n_tr, dtype=np.float32)
        l2_oof  = np.zeros(n_tr, dtype=np.float32)
        te_cos_pf = {}; te_l1_pf = {}; te_l2_pf = {}
        for f in sorted(np.unique(fold_arr)):
            f0 = time.time()
            tr_idx = np.where(fold_arr != f)[0]
            va_idx = np.where(fold_arr == f)[0]
            W = fit_whitener(np.vstack([tr_a_emb[tr_idx], tr_b_emb[tr_idx]]), k=k)
            wa_tr_a = W(tr_a_emb[tr_idx]); wb_tr_b = W(tr_b_emb[tr_idx])
            wa_va_a = W(tr_a_emb[va_idx]); wb_va_b = W(tr_b_emb[va_idx])
            wa_te_a = W(te_a_emb);        wb_te_b = W(te_b_emb)
            cos_va = cosine_rows(wa_va_a, wb_va_b); l1_va = l1_rows(wa_va_a, wb_va_b); l2_va = l2_rows(wa_va_a, wb_va_b)
            cos_te = cosine_rows(wa_te_a, wb_te_b); l1_te = l1_rows(wa_te_a, wb_te_b); l2_te = l2_rows(wa_te_a, wb_te_b)
            cos_oof[va_idx] = cos_va; l1_oof[va_idx] = l1_va; l2_oof[va_idx] = l2_va
            te_cos_pf[f] = cos_te; te_l1_pf[f] = l1_te; te_l2_pf[f] = l2_te
            yv = train.loc[va_idx, 'score'].values.astype(np.float32)
            print(f'  fold {int(f)}: raw-cos pearson={pearsonr(cos_va, yv)[0]:.6f} | {time.time()-f0:.1f}s', flush=True)
        tr_out = pd.DataFrame({'id': train['id'], f'bge_nocpc_cos_k{k}': cos_oof, f'bge_nocpc_l1_k{k}': l1_oof, f'bge_nocpc_l2_k{k}': l2_oof})
        te_out = pd.DataFrame({'id': test['id']})
        for f in sorted(te_cos_pf.keys()):
            te_out[f'bge_nocpc_cos_k{k}_tef{int(f)}'] = te_cos_pf[f].astype(np.float32)
            te_out[f'bge_nocpc_l1_k{k}_tef{int(f)}']  = te_l1_pf[f].astype(np.float32)
            te_out[f'bge_nocpc_l2_k{k}_tef{int(f)}']  = te_l2_pf[f].astype(np.float32)
        tr_path = f'feats_bge_nocpc_k{k}.csv'; te_path = f'feats_bge_nocpc_k{k}_test.csv'
        tr_out.to_csv(tr_path, index=False); te_out.to_csv(te_path, index=False)
        print('Saved', tr_path, te_path, flush=True)
    del tr_a_emb, tr_b_emb, te_a_emb, te_b_emb
    gc.collect()

# Execute feature extraction
process_bge_residual(alphas=(0.25,0.30,0.35), k_list=(0,1), batch_size=48, max_len=256)
process_bge_nocpc(k_list=(1,), batch_size=48, max_len=256)
print('Done BGE feature extraction in', round((time.time()-t0_all)/60,2), 'min')


=== Processing BGE residual ===


BGE residual unique texts: 37372


Batches:   0%|          | 0/779 [00:00<?, ?it/s]

Batches:   0%|          | 1/779 [00:00<05:03,  2.57it/s]

Batches:   0%|          | 2/779 [00:00<03:43,  3.48it/s]

...

  fold 2: raw-cos pearson=0.572038 | 0.6s


  fold 3: raw-cos pearson=0.592760 | 0.6s


  fold 4: raw-cos pearson=0.568811 | 0.6s


Saved feats_bge_nocpc_k1.csv feats_bge_nocpc_k1_test.csv


Done BGE feature extraction in 3.69 min


In [56]:
# Raw-distance stacker with per-fold BGE-residual selection, no L2, simple nonlinearities; uses stsb CE feature
import time, numpy as np, pandas as pd, re, scipy.sparse as sp
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge, HuberRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr

t0 = time.time()
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
folds = pd.read_csv('folds_by_id.csv')
train = train.merge(folds, on='id', how='left', validate='one_to_one')
assert (train['fold']>=0).all()

def merge_feats(df_ids, paths):
    df = df_ids.copy()
    for p in paths:
        tmp = pd.read_csv(p)
        df = df.merge(tmp, on='id', how='left')
    return df

# Fixed channels (keep cos, l1 only)
fixed_tr_paths = ['feats_e5_asym_k0.csv', 'feats_mpnet_asym_k1.csv', 'feats_bge_nocpc_k1.csv']
fixed_te_paths = ['feats_e5_asym_k0_test.csv', 'feats_mpnet_asym_k1_test.csv', 'feats_bge_nocpc_k1_test.csv']
Xtr_fixed_all = merge_feats(train[['id']], fixed_tr_paths)
Xte_fixed_all = merge_feats(test[['id']], fixed_te_paths)
fixed_cos_cols = ['e5_asym_cos_k0','mpnet_asym_cos_k1','bge_nocpc_cos_k1']
fixed_l1_cols  = ['e5_asym_l1_k0','mpnet_asym_l1_k1','bge_nocpc_l1_k1']

# Residual candidates: load all views
bge_res_candidates = [(a,k) for a in ('025','03','035') for k in (0,1)]
res_tr_paths = [f'feats_bge_res_a{a}_k{k}.csv' for (a,k) in bge_res_candidates]
res_te_paths = [f'feats_bge_res_a{a}_k{k}_test.csv' for (a,k) in bge_res_candidates]
Xtr_res_all = merge_feats(train[['id']], res_tr_paths)
Xte_res_all = merge_feats(test[['id']], res_te_paths)

# Build TF-IDF diagonal cosine (fit on TRAIN ONLY; transform test) to avoid leakage
def build_tfidf_diag_feature_train_only(tr_df, te_df):
    A_tr = tr_df['anchor'].astype(str).values; B_tr = tr_df['target'].astype(str).values
    A_te = te_df['anchor'].astype(str).values; B_te = te_df['target'].astype(str).values
    corpus_tr = np.concatenate([A_tr, B_tr])
    vec_c = TfidfVectorizer(analyzer='char', ngram_range=(3,5), min_df=2, max_features=300000)
    vec_w = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df=2, max_features=300000)
    Xc_tr = vec_c.fit_transform(corpus_tr); Xw_tr = vec_w.fit_transform(corpus_tr)
    # Split train A/B
    nA_tr, nB_tr = len(A_tr), len(B_tr)
    Xc_A_tr = Xc_tr[:nA_tr]; Xc_B_tr = Xc_tr[nA_tr: nA_tr+nB_tr]
    Xw_A_tr = Xw_tr[:nA_tr]; Xw_B_tr = Xw_tr[nA_tr: nA_tr+nB_tr]
    XA_tr = sp.hstack([Xc_A_tr, Xw_A_tr]).tocsr()
    XB_tr = sp.hstack([Xc_B_tr, Xw_B_tr]).tocsr()
    # Transform test using train-fitted vectorizers
    Xc_A_te = vec_c.transform(A_te); Xc_B_te = vec_c.transform(B_te)
    Xw_A_te = vec_w.transform(A_te); Xw_B_te = vec_w.transform(B_te)
    XA_te = sp.hstack([Xc_A_te, Xw_A_te]).tocsr()
    XB_te = sp.hstack([Xc_B_te, Xw_B_te]).tocsr()
    def row_diag_cos(XA, XB):
        n = XA.shape[0]; out = np.empty(n, dtype=np.float32); bs = 4096
        for i in range(0, n, bs):
            j = min(i+bs, n); C = cosine_similarity(XA[i:j], XB[i:j])
            out[i:j] = np.diag(C).astype(np.float32)
        return out
    return row_diag_cos(XA_tr, XB_tr), row_diag_cos(XA_te, XB_te)

tf_tr, tf_te = build_tfidf_diag_feature_train_only(train, test)

# Lexical features (includes len_diff for interactions)
def tokenize_words(s):
    return re.findall(r"\w+", str(s).lower())
def shingles(s, k=3):
    t = str(s).lower()
    if len(t) < k: return {t}
    return {t[i:i+k] for i in range(len(t)-k+1)}
def jaccard(a, b):
    if not a and not b: return 0.0
    return len(a & b) / (len(a | b) + 1e-12)
def build_lex(df):
    A = df['anchor'].astype(str).tolist(); B = df['target'].astype(str).tolist()
    tok_a = [tokenize_words(x) for x in A]; tok_b = [tokenize_words(x) for x in B]
    uni_j = np.array([jaccard(set(ta), set(tb)) for ta, tb in zip(tok_a, tok_b)], dtype=np.float32)
    c3_a = [shingles(x, 3) for x in A]; c3_b = [shingles(x, 3) for x in B]
    c3_j = np.array([jaccard(a, b) for a, b in zip(c3_a, c3_b)], dtype=np.float32)
    len_a = np.array([len(x) for x in A], dtype=np.float32)
    len_b = np.array([len(x) for x in B], dtype=np.float32)
    tok_na = np.array([len(t) for t in tok_a], dtype=np.float32)
    tok_nb = np.array([len(t) for t in tok_b], dtype=np.float32)
    len_diff = np.abs(len_a - len_b)
    len_ratio = (np.minimum(len_a, len_b)+1.0) / (np.maximum(len_a, len_b)+1.0)
    tok_diff = np.abs(tok_na - tok_nb)
    tok_ratio = (np.minimum(tok_na, tok_nb)+1.0) / (np.maximum(tok_na, tok_nb)+1.0)
    sec = df['context'].astype(str).str[0].fillna('X')
    sec_dum = pd.get_dummies(sec, prefix='sec')
    out = pd.DataFrame({'jac_uni': uni_j, 'jac_c3': c3_j, 'len_diff': len_diff, 'len_ratio': len_ratio, 'tok_diff': tok_diff, 'tok_ratio': tok_ratio})
    out = pd.concat([out.reset_index(drop=True), sec_dum.reset_index(drop=True)], axis=1)
    return out

Xlex_tr = build_lex(train)
Xlex_te = build_lex(test)

# CE feature (stsb calibrated expected) 
ce_tr = pd.read_csv('oof_ce_stsb.csv').rename(columns={'oof':'ce_score'})
ce_te = pd.read_csv('submission_ce_stsb.csv').rename(columns={'score':'ce_score'})

# Helper to add simple nonlinearities for cos features and interactions with len_diff, and Fisher z-transform
def fisher_z(x):
    x = np.clip(x.astype(np.float32), -0.999, 0.999)
    return np.arctanh(x).astype(np.float32)

def augment_with_nonlinear(Xdf, cos_cols, len_col=None, add_fz=True):
    for c in cos_cols:
        Xdf[c+'_one_minus'] = (1.0 - Xdf[c].astype(np.float32)).astype(np.float32)
        Xdf[c+'_sq'] = (Xdf[c].astype(np.float32) ** 2).astype(np.float32)
        if add_fz:
            Xdf[c+'_fz'] = fisher_z(Xdf[c].values.astype(np.float32))
        if len_col is not None and len_col in Xdf.columns:
            Xdf[c+'_x_len'] = (Xdf[c].astype(np.float32) * Xdf[len_col].astype(np.float32)).astype(np.float32)
    return Xdf

def add_anchor_rank_features(df_tr, df_te, cos_cols):
    # Compute per-anchor ranks/percentiles and gap-to-top within the combined (train+test) frame for this fold
    df_tr = df_tr.copy(); df_te = df_te.copy()
    comb = pd.concat([df_tr[['anchor'] + cos_cols].assign(_is_tr=1),
                      df_te[['anchor'] + cos_cols].assign(_is_tr=0)],
                     axis=0, ignore_index=True)
    for c in cos_cols:
        # group by anchor
        grp = comb.groupby('anchor')[c]
        # rank within group (1..n)
        comb[c+'_rank'] = grp.rank(method='min', ascending=False).astype(np.float32)
        # group size
        gsize = grp.transform('count').astype(np.float32)
        # percentile rank
        comb[c+'_pct'] = np.where(gsize>1, (comb[c+'_rank']-1.0)/(gsize-1.0), 0.5).astype(np.float32)
        # gap to top: max - value
        gmax = grp.transform('max').astype(np.float32)
        comb[c+'_gap_top'] = (gmax - comb[c].astype(np.float32)).astype(np.float32)
    # split back
    tr_feats = comb.loc[comb['_is_tr']==1, [col for col in comb.columns if col not in ('anchor','_is_tr')]].reset_index(drop=True)
    te_feats = comb.loc[comb['_is_tr']==0, [col for col in comb.columns if col not in ('anchor','_is_tr')]].reset_index(drop=True)
    # concat to frames
    df_tr = pd.concat([df_tr.reset_index(drop=True), tr_feats], axis=1)
    df_te = pd.concat([df_te.reset_index(drop=True), te_feats], axis=1)
    return df_tr, df_te

# Prepare base feature frames
feat_tr_base = train[['id','score','fold','anchor','target']].merge(Xtr_fixed_all[['id'] + fixed_cos_cols + fixed_l1_cols], on='id')
feat_te_base = test[['id','anchor','target']].merge(Xte_fixed_all[['id'] + list(Xte_fixed_all.columns.drop('id'))], on='id')
feat_tr_base['tfidf_diag'] = tf_tr; feat_te_base['tfidf_diag'] = tf_te
feat_tr_base = pd.concat([feat_tr_base.reset_index(drop=True), Xlex_tr.reset_index(drop=True)], axis=1)
feat_te_base = pd.concat([feat_te_base.reset_index(drop=True), Xlex_te.reset_index(drop=True)], axis=1)
feat_tr_base = feat_tr_base.merge(ce_tr, on='id', how='left')
feat_te_base = feat_te_base.merge(ce_te, on='id', how='left')

y = feat_tr_base['score'].values.astype(np.float32)
fold_arr = feat_tr_base['fold'].values.astype(int)

# Build candidate dicts for residual (train/test) for easy per-fold selection
res_train = {}  # key -> (cos, l1)
for (a,k) in bge_res_candidates:
    cos_c = f'bge_res_a{a}_cos_k{k}'; l1_c = f'bge_res_a{a}_l1_k{k}'
    if cos_c in Xtr_res_all.columns:
        res_train[(a,k)] = (Xtr_res_all[cos_c].values.astype(np.float32), Xtr_res_all[l1_c].values.astype(np.float32))

# Per-fold training with fold-specific residual view selection and per-fold test consumption for whitened channels
oof = np.zeros_like(y, dtype=np.float32)
test_preds = []
alpha = 1.0
best_keys = {}
for f in sorted(np.unique(fold_arr)):
    tr_idx = np.where(fold_arr != f)[0]
    va_idx = np.where(fold_arr == f)[0]
    # select best residual candidate by val Pearson on residual cosine
    best_p, best_key = -1.0, None
    for key, (cos_tr_vals, _) in res_train.items():
        p = pearsonr(cos_tr_vals[va_idx], y[va_idx])[0]
        if p > best_p:
            best_p, best_key = p, key
    best_keys[f] = (best_key, best_p)
    a,k = best_key
    # Assemble fold-specific frames including selected residual cos/l1
    cos_col = f'bge_res_a{a}_cos_k{k}'
    l1_col  = f'bge_res_a{a}_l1_k{k}'
    tr_fold = feat_tr_base.copy()
    te_fold = feat_te_base.copy()
    # Train residual uses OOF columns; Test residual must use fold-specific columns _tef{f}
    tr_fold['res_cos'] = Xtr_res_all[cos_col].values.astype(np.float32)
    tr_fold['res_l1']  = Xtr_res_all[l1_col].values.astype(np.float32)
    te_fold['res_cos'] = Xte_res_all[f'{cos_col}_tef{int(f)}'].values.astype(np.float32)
    te_fold['res_l1']  = Xte_res_all[f'{l1_col}_tef{int(f)}'].values.astype(np.float32)
    # For bge_nocpc fixed channel: override test with fold-specific whitened features
    te_fold['bge_nocpc_cos_k1'] = Xte_fixed_all[f'bge_nocpc_cos_k1_tef{int(f)}'].values.astype(np.float32)
    te_fold['bge_nocpc_l1_k1']  = Xte_fixed_all[f'bge_nocpc_l1_k1_tef{int(f)}'].values.astype(np.float32)
    # Nonlinearities on cosine columns + interactions with len_diff + Fisher z
    cos_cols_aug = fixed_cos_cols + ['res_cos', 'tfidf_diag']
    tr_fold = augment_with_nonlinear(tr_fold, cos_cols_aug, len_col='len_diff', add_fz=True)
    te_fold = augment_with_nonlinear(te_fold, cos_cols_aug, len_col='len_diff', add_fz=True)
    # Per-anchor rank/percentile/gap features (within fold) on main cosine channels
    rank_cols = fixed_cos_cols + ['res_cos']
    tr_fold, te_fold = add_anchor_rank_features(tr_fold, te_fold, rank_cols)
    # Select columns (drop id/score/fold/texts); include cos, (1-cos), cos^2, fz, l1, tfidf_diag, lexical, ce, and rank features
    drop_cols = {'id','score','fold','anchor','target'}
    use_cols = [c for c in tr_fold.columns if c not in drop_cols]
    Xtr = tr_fold.loc[tr_idx, use_cols].values.astype(np.float32)
    Xva = tr_fold.loc[va_idx, use_cols].values.astype(np.float32)
    Xte = te_fold[use_cols].values.astype(np.float32)
    ss = StandardScaler(with_mean=True, with_std=True)
    Xtr_s = ss.fit_transform(Xtr); Xva_s = ss.transform(Xva); Xte_s = ss.transform(Xte)
    mdl = Ridge(alpha=alpha, fit_intercept=True, random_state=42)
    mdl.fit(Xtr_s, y[tr_idx])
    oof[va_idx] = mdl.predict(Xva_s).astype(np.float32)
    test_preds.append(mdl.predict(Xte_s).astype(np.float32))
    print(f'Fold {int(f)} select res={best_key} pearson=', round(float(pearsonr(oof[va_idx], y[va_idx])[0]), 6), 'res-cos val p=', round(float(best_p),6))

p = pearsonr(oof, y)[0]
print('OOF Pearson RAW feature Ridge stack (per-fold res sel, no L2):', round(p,6))
pd.DataFrame({'id': train['id'], 'oof': oof}).to_csv('oof_stack_rawfeat_ridge.csv', index=False)

# Average test predictions across folds
test_pred = np.mean(np.vstack(test_preds), axis=0).astype(np.float32)
sub = pd.DataFrame({'id': test['id'], 'score': np.clip(test_pred, 0.0, 1.0)})
sub.to_csv('submission_rawfeat_stack.csv', index=False)
print('Saved submission_rawfeat_stack.csv; elapsed', round((time.time()-t0)/60,2), 'min')

# Optionally set as submission.csv
sub.to_csv('submission.csv', index=False)

Fold 0 select res=('035', 1) pearson= 0.705112 res-cos val p= 0.547123


Fold 1 select res=('035', 0) pearson= 0.693038 res-cos val p= 0.540785
Fold 2 select res=('035', 0) pearson= 0.679229 res-cos val p= 0.532873


Fold 3 select res=('035', 0) pearson= 0.69963 res-cos val p= 0.556859
Fold 4 select res=('035', 0) pearson= 0.695511 res-cos val p= 0.534199
OOF Pearson RAW feature Ridge stack (per-fold res sel, no L2): 0.694335


Saved submission_rawfeat_stack.csv; elapsed 0.07 min


# DISABLED: Old CE overwrite cell (MiniLM-L-6 raw, no context)
This cell previously overwrote the cross-encoder artifacts (oof_ce_stsb.csv, submission_ce_stsb.csv) with ms-marco-MiniLM-L-6-v2 raw scores and no CPC context, which weakens the stacker.

Per expert guidance, this path is disabled. Do NOT run or restore it. We'll replace the CE channel with a stronger, context-aware model (MiniLM-L-12-v2 or STSB-large) in a new cell.

In [57]:
# Cross-Encoder (CPU): ms-marco-MiniLM-L-12-v2 with asymmetric CPC-on-anchor prompt; save RAW and isotonic; also overwrite CE artifacts used by stacker
import time, numpy as np, pandas as pd
from scipy.stats import pearsonr
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LinearRegression
from sentence_transformers import CrossEncoder

SEED = 42
np.random.seed(SEED)

# CPC mapping (4->3->2 title; RAW code fallback), truncate to 10 words
CPC_DESCRIPTIONS = {
    'A61B':'Diagnosis; Surgery; Identification','A61F':'Filters; Prostheses; Braces','A61K':'Preparations for medical, dental, or toilet purposes','A61M':'Devices for introducing/circulating media into the body',
    'B01D':'Separation; Sorption; Membranes','B29C':'Shaping or joining of plastics','B32B':'Layered products','B60K':'Arrangement or mounting of vehicle power plant or auxiliaries','B60R':'Vehicles, vehicle fittings, or vehicle parts','B60T':'Vehicle brake control systems or parts',
    'C01B':'Non-metallic elements; Compounds thereof','C04B':'Cement; Concrete; Ceramics','C07C':'Acyclic or carbocyclic compounds','C07D':'Heterocyclic compounds','C07K':'Peptides','C08F':'Macromolecular compounds by polymerising monomers','C08L':'Compositions of macromolecular compounds','C09K':'Materials for miscellaneous applications',
    'C12N':'Microorganisms or enzymes; Genetic engineering','C12P':'Fermentation or enzyme-using processes','C12Q':'Measuring/testing involving enzymes or microorganisms',
    'E04B':'Building constructions; Walls; Floors; Roofs','F16B':'Screws; Rivets; Joints','F16F':'Springs; Shock-absorbers; Dampers','F16L':'Pipes; Joints or fittings for pipes',
    'G01N':'Investigating or analysing materials','G02B':'Optical elements, systems, or apparatus','G02F':'Control of light','G06F':'Electric digital data processing','G06K':'Recognition/Presentation of data','G06Q':'Data processing for administrative/financial/managerial',
    'H01L':'Semiconductor devices','H01M':'Batteries; Fuel cells','H02J':'Power supply/distribution','H04L':'Transmission of digital information','H04N':'Pictorial communication','H04W':'Wireless communication networks','H05K':'Printed circuits; Electric assemblies',
    'A01':'Agriculture; Forestry; Animal husbandry; Hunting; Trapping; Fishing','A21':'Baking; Edible doughs','A23':'Foods or foodstuffs; Treatment thereof','A24':'Tobacco; Cigars; Cigarettes; Simulated smoking devices; Smokers\' requisites','A41':'Wearing apparel','A42':'Headwear','A43':'Footwear','A45':'Hand or travelling articles','A46':'Brushware','A47':'Furniture; Domestic articles or appliances; Coffee mills; Spice mills; Suction cleaners','A61':'Medical or veterinary science; Hygiene','A62':'Life-saving; Fire-fighting','A63':'Sports; Games; Amusements',
    'B01':'Physical or chemical processes or apparatus','B02':'Crushing; pulverising; disintegrating','B03':'Separation of solid materials using liquids or jigs','B04':'Centrifugal apparatus or machines','B05':'Spraying; atomising; Applying fluent materials to surfaces','B06':'Mechanical vibrations','B07':'Separating solids; Sorting','B08':'Cleaning','B09':'Disposal of solid waste; Soil reclamation','B21':'Mechanical metal-working without removing material','B22':'Casting; Powder metallurgy','B23':'Machine tools; Metal-working n.e.c.','B24':'Grinding; Polishing','B25':'Hand tools; Portable power-driven tools; Manipulators','B26':'Hand cutting tools; Cutting; Severing','B27':'Working/preserving wood; Nailing or stapling','B28':'Working cement, clay, or stone','B29':'Working of plastics; Plastic state substances','B30':'Presses','B31':'Making paper-like articles','B32':'Layered products','B33':'Additive manufacturing','B41':'Printing; Typewriters; Stamps','B42':'Bookbinding; Albums; Files','B43':'Writing or drawing implements; Bureau accessories','B44':'Decorative arts','B60':'Vehicles in general','B61':'Railways','B62':'Land vehicles not on rails','B63':'Ships or waterborne vessels','B64':'Aircraft; Aviation; Cosmonautics','B65':'Conveying; Packing; Storing','B66':'Hoisting; Lifting; Hauling','B67':'Opening/closing containers; Liquid handling','B68':'Saddlery; Upholstery','B81':'Microstructural technology','B82':'Nanotechnology',
    'C01':'Inorganic chemistry','C02':'Treatment of water, wastewater, sewage, sludge','C03':'Glass; Mineral or slag wool','C04':'Cements; Concrete; Ceramics; Refractories','C05':'Fertilisers','C06':'Explosives; Matches','C07':'Organic chemistry','C08':'Organic macromolecular compounds; Polymers','C09':'Dyes; Paints; Polishes; Adhesives; Misc compositions','C10':'Petroleum/gas/coke industries; Fuels; Lubricants; Peat','C11':'Oils, fats, waxes; Detergents; Candles','C12':'Biochemistry; Microbiology; Enzymology; Fermentation','C13':'Sugar industry','C14':'Skins; Hides; Leather',
    'C21':'Metallurgy of iron','C22':'Metallurgy; Alloys; Treatment of non-ferrous metals','C23':'Coating metallic material; Chemical surface treatment','C25':'Electrolytic/electrophoretic processes','C30':'Crystal growth','C40':'Combinatorial technology',
    'D01':'Threads or fibres; Spinning','D02':'Yarns; Finishing; Warping or beaming','D03':'Weaving','D04':'Braiding; Lace-making; Knitting; Non-wovens','D05':'Sewing; Embroidering; Tufting','D06':'Textile treatment; Laundering; Flexible materials','D07':'Ropes; Cables other than electric','D21':'Paper-making; Cellulose production',
    'E01':'Road/rail/bridge construction','E02':'Hydraulic engineering; Foundations; Soil shifting','E03':'Water supply; Sewerage','E04':'Building','E05':'Locks; Keys; Fittings; Safes','E06':'Doors; Windows; Shutters; Ladders','E21':'Earth/rock drilling; Mining',
    'F01':'Machines or engines in general','F02':'Combustion engines; Hot-gas plants','F03':'Machines or engines for liquids; Wind/spring motors','F04':'Positive-displacement machines; Pumps','F15':'Fluid-pressure actuators; Hydraulics; Pneumatics','F16':'Engineering elements or units','F17':'Storing/distributing gases or liquids','F21':'Lighting','F22':'Steam generation','F23':'Combustion apparatus; Processes','F24':'Heating; Ranges; Ventilating','F25':'Refrigeration or cooling; Heat pumps','F26':'Drying','F27':'Furnaces; Kilns; Ovens','F28':'Heat exchange in general','F41':'Weapons','F42':'Ammunition; Blasting',
    'G01':'Measuring; Testing','G02':'Optics','G03':'Photography; Cinematography; Non-optical waves','G04':'Horology','G05':'Controlling; Regulating','G06':'Computing; Calculating; Counting','G07':'Checking-devices','G08':'Signalling','G09':'Educating; Cryptography; Display; Advertising; Seals','G10':'Musical instruments; Acoustics','G11':'Information storage','G12':'Instrument details','G16':'ICT adapted for specific fields','G21':'Nuclear physics; Nuclear engineering',
    'H01':'Electric elements','H02':'Electric power: generation, conversion, distribution','H03':'Electronic circuitry','H04':'Electric communication technique','H05':'Electric techniques not otherwise provided for'
}

def get_cpc_text(code: str) -> str:
    if not isinstance(code, str) or not code:
        return 'GENERAL'
    code = code.strip().upper()
    for k in (4,3,2):
        pref = code[:k]
        if pref in CPC_DESCRIPTIONS:
            return CPC_DESCRIPTIONS[pref]
    return code  # RAW fallback

def truncate_words(text: str, n: int = 10) -> str:
    w = text.split() if isinstance(text, str) else []
    return ' '.join(w[:n])

def build_pairs(df: pd.DataFrame):
    cpc_full = df['context'].astype(str).map(get_cpc_text)
    cpc_10w = cpc_full.map(lambda x: truncate_words(x, 10))
    a_side = (cpc_10w + ' [SEP] ' + df['anchor'].astype(str)).tolist()
    b_side = df['target'].astype(str).tolist()
    return list(zip(a_side, b_side))

t0 = time.time()
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
folds = pd.read_csv('folds_by_id.csv')
train = train.merge(folds, on='id', how='left', validate='one_to_one')
assert (train['fold']>=0).all(), 'Fold merge by id failed'

train_pairs = build_pairs(train)
test_pairs = build_pairs(test)

model_name = 'cross-encoder/ms-marco-MiniLM-L-12-v2'
ce = CrossEncoder(model_name, device='cpu', max_length=256, tokenizer_args={'use_fast': False})

print('Scoring train pairs (L-12 CE, context-aware)...', flush=True)
train_scores_raw = np.asarray(ce.predict(train_pairs, batch_size=64, show_progress_bar=True), dtype=np.float32)
print('Scoring test pairs (L-12 CE, context-aware)...', flush=True)
test_scores_raw = np.asarray(ce.predict(test_pairs, batch_size=64, show_progress_bar=True), dtype=np.float32)

# Per-fold isotonic vs linear calibration for diagnostics; save both, but raw will be primary for stacking
y = train['score'].values.astype(np.float32)
oof_iso = np.zeros_like(y, dtype=np.float32)
oof_lin = np.zeros_like(y, dtype=np.float32)
test_iso_f, test_lin_f = [], []
for f in sorted(train['fold'].unique()):
    tr_idx = np.where(train['fold'] != f)[0]
    va_idx = np.where(train['fold'] == f)[0]
    raw_p = pearsonr(train_scores_raw[va_idx], y[va_idx])[0]
    iso = IsotonicRegression(out_of_bounds='clip')
    iso.fit(train_scores_raw[tr_idx], y[tr_idx])
    oof_iso[va_idx] = iso.predict(train_scores_raw[va_idx]).astype(np.float32)
    test_iso_f.append(iso.predict(test_scores_raw).astype(np.float32))
    lr = LinearRegression()
    lr.fit(train_scores_raw[tr_idx].reshape(-1,1), y[tr_idx])
    oof_lin[va_idx] = lr.predict(train_scores_raw[va_idx].reshape(-1,1)).astype(np.float32)
    test_lin_f.append(lr.predict(test_scores_raw.reshape(-1,1)).astype(np.float32))
    print(f'L12 CE Fold {int(f)}: raw={raw_p:.6f} | iso={pearsonr(oof_iso[va_idx], y[va_idx])[0]:.6f} | lin={pearsonr(oof_lin[va_idx], y[va_idx])[0]:.6f}', flush=True)

p_raw = pearsonr(train_scores_raw, y)[0]
p_iso = pearsonr(oof_iso, y)[0]
p_lin = pearsonr(oof_lin, y)[0]
print('L12 CE OOF Pearson: raw=', round(float(p_raw),6), ' iso=', round(float(p_iso),6), ' lin=', round(float(p_lin),6))

# Save artifacts: raw and isotonic
pd.DataFrame({'id': train['id'], 'oof_raw': train_scores_raw, 'oof_iso': oof_iso}).to_csv('oof_ce_l12.csv', index=False)
pd.DataFrame({'id': test['id'], 'score_raw': test_scores_raw, 'score_iso': np.mean(np.vstack(test_iso_f), axis=0).astype(np.float32)}).to_csv('submission_ce_l12.csv', index=False)

# Also overwrite the CE files used by current stackers with RAW L12 scores to immediately upgrade CE channel
pd.DataFrame({'id': train['id'], 'oof': train_scores_raw}).to_csv('oof_ce_stsb.csv', index=False)
pd.DataFrame({'id': test['id'], 'score': test_scores_raw}).to_csv('submission_ce_stsb.csv', index=False)
print('Saved L12 CE artifacts and overwrote oof_ce_stsb.csv/submission_ce_stsb.csv with RAW L12. Elapsed', round((time.time()-t0)/60,2), 'min')

Scoring train pairs (L-12 CE, context-aware)...


Batches:   0%|          | 0/513 [00:00<?, ?it/s]

Batches:   0%|          | 1/513 [00:00<00:54,  9.36it/s]

Batches:   1%|          | 3/513 [00:00<00:49, 10.32it/s]

Batches:   1%|          | 5/513 [00:00<00:51,  9.86it/s]

...

L12 CE Fold 1: raw=0.423457 | iso=0.429673 | lin=0.423457


L12 CE Fold 2: raw=0.433308 | iso=0.432062 | lin=0.433308


L12 CE Fold 3: raw=0.420675 | iso=0.423442 | lin=0.420675


L12 CE Fold 4: raw=0.439524 | iso=0.440798 | lin=0.439524


L12 CE OOF Pearson: raw= 0.433401  iso= 0.435896  lin= 0.433219
Saved L12 CE artifacts and overwrote oof_ce_stsb.csv/submission_ce_stsb.csv with RAW L12. Elapsed 0.9 min


In [70]:
# Non-linear stacker (LightGBM, CPU): raw-distance features + TFIDF diag + lexical + CE raw + BM25 + IDF overlaps + RapidFuzz + LCS/char-ngrams + per-anchor ranks (no train-test mixing);
# IMPORTANT: Remove leakage by including ALL BGE residual variants as separate features (no per-fold selection).
import time, numpy as np, pandas as pd, re, scipy.sparse as sp
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.stats import pearsonr
import lightgbm as lgb

t0 = time.time()
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
folds = pd.read_csv('folds_by_id.csv')
train = train.merge(folds, on='id', how='left', validate='one_to_one')
assert (train['fold']>=0).all()

def merge_feats(df_ids, paths):
    df = df_ids.copy()
    for p in paths:
        tmp = pd.read_csv(p)
        df = df.merge(tmp, on='id', how='left')
    return df

# Fixed distance channels
fixed_tr_paths = ['feats_e5_asym_k0.csv', 'feats_mpnet_asym_k1.csv', 'feats_bge_nocpc_k1.csv']
fixed_te_paths = ['feats_e5_asym_k0_test.csv', 'feats_mpnet_asym_k1_test.csv', 'feats_bge_nocpc_k1_test.csv']
Xtr_fixed_all = merge_feats(train[['id']], fixed_tr_paths)
Xte_fixed_all = merge_feats(test[['id']], fixed_te_paths)
fixed_cos_cols = ['e5_asym_cos_k0','mpnet_asym_cos_k1','bge_nocpc_cos_k1']
fixed_l1_cols  = ['e5_asym_l1_k0','mpnet_asym_l1_k1','bge_nocpc_l1_k1']

# Residual candidates (BGE residualized CPC) - include ALL variants as features (no selection)
bge_res_candidates = [(a,k) for a in ('025','03','035') for k in (0,1)]
res_tr_paths = [f'feats_bge_res_a{a}_k{k}.csv' for (a,k) in bge_res_candidates]
res_te_paths = [f'feats_bge_res_a{a}_k{k}_test.csv' for (a,k) in bge_res_candidates]
Xtr_res_all = merge_feats(train[['id']], res_tr_paths)
Xte_res_all = merge_feats(test[['id']], res_te_paths)

# TF-IDF diagonal cosine (fit on train only, transform test)
def build_tfidf_diag_feature_train_only(tr_df, te_df):
    A_tr = tr_df['anchor'].astype(str).values; B_tr = tr_df['target'].astype(str).values
    A_te = te_df['anchor'].astype(str).values; B_te = te_df['target'].astype(str).values
    corpus_tr = np.concatenate([A_tr, B_tr])
    vec_c = TfidfVectorizer(analyzer='char', ngram_range=(3,5), min_df=2, max_features=300000)
    vec_w = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df=2, max_features=300000)
    Xc_tr = vec_c.fit_transform(corpus_tr); Xw_tr = vec_w.fit_transform(corpus_tr)
    nA_tr, nB_tr = len(A_tr), len(B_tr)
    Xc_A_tr = Xc_tr[:nA_tr]; Xc_B_tr = Xc_tr[nA_tr:nA_tr+nB_tr]
    Xw_A_tr = Xw_tr[:nA_tr]; Xw_B_tr = Xw_tr[nA_tr:nA_tr+nB_tr]
    XA_tr = sp.hstack([Xc_A_tr, Xw_A_tr]).tocsr()
    XB_tr = sp.hstack([Xc_B_tr, Xw_B_tr]).tocsr()
    Xc_A_te = vec_c.transform(A_te); Xc_B_te = vec_c.transform(B_te)
    Xw_A_te = vec_w.transform(A_te); Xw_B_te = vec_w.transform(B_te)
    XA_te = sp.hstack([Xc_A_te, Xw_A_te]).tocsr()
    XB_te = sp.hstack([Xc_B_te, Xw_B_te]).tocsr()
    def row_diag_cos(XA, XB):
        n = XA.shape[0]; out = np.empty(n, dtype=np.float32); bs = 4096
        for i in range(0, n, bs):
            j = min(i+bs, n); C = cosine_similarity(XA[i:j], XB[i:j])
            out[i:j] = np.diag(C).astype(np.float32)
        return out
    return row_diag_cos(XA_tr, XB_tr), row_diag_cos(XA_te, XB_te)

tf_tr, tf_te = build_tfidf_diag_feature_train_only(train, test)

# Lexical and simple length features
def tokenize_words(s): return re.findall(r"\w+", str(s).lower())
def shingles(s, k=3):
    t = str(s).lower()
    if len(t) < k: return {t}
    return {t[i:i+k] for i in range(len(t)-k+1)}
def jaccard(a, b):
    if not a and not b: return 0.0
    return len(a & b) / (len(a | b) + 1e-12)
def build_lex(df):
    A = df['anchor'].astype(str).tolist(); B = df['target'].astype(str).tolist()
    tok_a = [tokenize_words(x) for x in A]; tok_b = [tokenize_words(x) for x in B]
    uni_j = np.array([jaccard(set(ta), set(tb)) for ta, tb in zip(tok_a, tok_b)], dtype=np.float32)
    c3_a = [shingles(x, 3) for x in A]; c3_b = [shingles(x, 3) for x in B]
    c3_j = np.array([jaccard(a, b) for a, b in zip(c3_a, c3_b)], dtype=np.float32)
    len_a = np.array([len(x) for x in A], dtype=np.float32)
    len_b = np.array([len(x) for x in B], dtype=np.float32)
    tok_na = np.array([len(t) for t in tok_a], dtype=np.float32)
    tok_nb = np.array([len(t) for t in tok_b], dtype=np.float32)
    len_diff = np.abs(len_a - len_b)
    len_ratio = (np.minimum(len_a, len_b)+1.0) / (np.maximum(len_a, len_b)+1.0)
    tok_diff = np.abs(tok_na - tok_nb)
    tok_ratio = (np.minimum(tok_na, tok_nb)+1.0) / (np.maximum(tok_na, tok_nb)+1.0)
    sec = df['context'].astype(str).str[0].fillna('X')
    sec_dum = pd.get_dummies(sec, prefix='sec')
    out = pd.DataFrame({'jac_uni': uni_j, 'jac_c3': c3_j, 'len_diff': len_diff, 'len_ratio': len_ratio, 'tok_diff': tok_diff, 'tok_ratio': tok_ratio})
    out = pd.concat([out.reset_index(drop=True), sec_dum.reset_index(drop=True)], axis=1)
    return out

Xlex_tr = build_lex(train)
Xlex_te = build_lex(test)

# CE features: use RAW CE as primary; optionally also load isotonic
ce_raw = pd.read_csv('oof_ce_stsb.csv').rename(columns={'oof':'ce_raw'})
try:
    ce_l12 = pd.read_csv('oof_ce_l12.csv')[['id','oof_iso']].rename(columns={'oof_iso':'ce_iso'})
    ce_tr = ce_raw.merge(ce_l12, on='id', how='left')
except Exception:
    ce_tr = ce_raw
ce_te_raw = pd.read_csv('submission_ce_stsb.csv').rename(columns={'score':'ce_raw'})
try:
    ce_te_l12 = pd.read_csv('submission_ce_l12.csv')[['id','score_iso']].rename(columns={'score_iso':'ce_iso'})
    ce_te = ce_te_raw.merge(ce_te_l12, on='id', how='left')
except Exception:
    ce_te = ce_te_raw

# BM25 features (per-fold train-only IDF, test averaged over folds)
bm25_tr = pd.read_csv('oof_bm25.csv')  # id, bm25_ab, bm25_ba
bm25_te = pd.read_csv('bm25_test.csv')

# IDF-weighted overlap features (1-2 grams), fold-safe
idf_tr = pd.read_csv('oof_idf_overlap.csv')  # id + idf1_* and idf2_*
idf_te = pd.read_csv('idf_overlap_test.csv')

# RapidFuzz token-set features
rf_tr = pd.read_csv('oof_fuzz.csv')  # id, rf_token_set, rf_partial_token_set
rf_te = pd.read_csv('fuzz_test.csv')

# LCS (char/token) and char n-gram (3-5) similarities
lcs_tr = pd.read_csv('oof_lcs_char_ngrams.csv')  # id, lcs_char, lcs_tok, char3_jac, char4_jac, char5_jac, char3_dice, char4_dice, char5_dice
lcs_te = pd.read_csv('lcs_char_ngrams_test.csv')

# Char edit similarities (Jaro-Winkler, Levenshtein, Damerau-Levenshtein) with simple nonlinearities already included
char_tr = pd.read_csv('oof_char_edit.csv')
char_te = pd.read_csv('char_edit_test.csv')

# Nonlinear transforms for cosine-like channels
def fisher_z(x):
    x = np.clip(x.astype(np.float32), -0.999, 0.999)
    return np.arctanh(x).astype(np.float32)
def augment_with_nonlinear(Xdf, cos_cols, len_col=None, add_fz=True):
    for c in cos_cols:
        if c not in Xdf.columns: continue
        Xdf[c+'_one_minus'] = (1.0 - Xdf[c].astype(np.float32)).astype(np.float32)
        Xdf[c+'_sq'] = (Xdf[c].astype(np.float32) ** 2).astype(np.float32)
        if add_fz: Xdf[c+'_fz'] = fisher_z(Xdf[c].values.astype(np.float32))
        if len_col is not None and len_col in Xdf.columns:
            Xdf[c+'_x_len'] = (Xdf[c].astype(np.float32) * Xdf[len_col].astype(np.float32)).astype(np.float32)
    return Xdf

# Per-anchor ranks within the SAME frame only (no mixing train/test)
def add_within_frame_anchor_ranks(df: pd.DataFrame, cols):
    df = df.copy()
    for c in cols:
        if c not in df.columns: continue
        grp = df.groupby('anchor')[c]
        r = grp.rank(method='min', ascending=False).astype(np.float32)
        gsize = grp.transform('count').astype(np.float32)
        pct = np.where(gsize>1, (r-1.0)/(gsize-1.0), 0.5).astype(np.float32)
        gmax = grp.transform('max').astype(np.float32)
        gap = (gmax - df[c].astype(np.float32)).astype(np.float32)
        df[c+'_rank'] = r; df[c+'_pct'] = pct; df[c+'_gap_top'] = gap
    return df

# Base frames
feat_tr_base = train[['id','score','fold','anchor','target']].merge(Xtr_fixed_all[['id'] + fixed_cos_cols + fixed_l1_cols], on='id')
feat_te_base = test[['id','anchor','target']].merge(Xte_fixed_all[['id'] + list(Xte_fixed_all.columns.drop('id'))], on='id')
feat_tr_base['tfidf_diag'] = tf_tr; feat_te_base['tfidf_diag'] = tf_te
feat_tr_base = pd.concat([feat_tr_base.reset_index(drop=True), Xlex_tr.reset_index(drop=True)], axis=1)
feat_te_base = pd.concat([feat_te_base.reset_index(drop=True), Xlex_te.reset_index(drop=True)], axis=1)
feat_tr_base = feat_tr_base.merge(ce_tr, on='id', how='left')
feat_te_base = feat_te_base.merge(ce_te, on='id', how='left')

# Merge BM25 and add simple nonlinearities
feat_tr_base = feat_tr_base.merge(bm25_tr, on='id', how='left')
feat_te_base = feat_te_base.merge(bm25_te, on='id', how='left')
for c in ['bm25_ab','bm25_ba']:
    if c in feat_tr_base.columns:
        feat_tr_base[c+'_sq'] = (feat_tr_base[c].astype(np.float32) ** 2).astype(np.float32)
    if c in feat_te_base.columns:
        feat_te_base[c+'_sq'] = (feat_te_base[c].astype(np.float32) ** 2).astype(np.float32)

# Merge IDF-overlap features and add light transforms
feat_tr_base = feat_tr_base.merge(idf_tr, on='id', how='left')
feat_te_base = feat_te_base.merge(idf_te, on='id', how='left')
for c in ['idf1_f1','idf1_jac','idf2_f1','idf2_jac']:
    if c in feat_tr_base.columns:
        feat_tr_base[c+'_sq'] = (feat_tr_base[c].astype(np.float32) ** 2).astype(np.float32)
    if c in feat_te_base.columns:
        feat_te_base[c+'_sq'] = (feat_te_base[c].astype(np.float32) ** 2).astype(np.float32)

# Merge RapidFuzz features and simple transforms
feat_tr_base = feat_tr_base.merge(rf_tr, on='id', how='left')
feat_te_base = feat_te_base.merge(rf_te, on='id', how='left')
for c in ['rf_token_set','rf_partial_token_set']:
    if c in feat_tr_base.columns:
        feat_tr_base[c+'_sq'] = (feat_tr_base[c].astype(np.float32) ** 2).astype(np.float32)
    if c in feat_te_base.columns:
        feat_te_base[c+'_sq'] = (feat_te_base[c].astype(np.float32) ** 2).astype(np.float32)

# Merge LCS/char-ngrams features and add simple transforms
feat_tr_base = feat_tr_base.merge(lcs_tr, on='id', how='left')
feat_te_base = feat_te_base.merge(lcs_te, on='id', how='left')
for c in ['lcs_char','lcs_tok','char3_jac','char4_jac','char5_jac','char3_dice','char4_dice','char5_dice']:
    if c in feat_tr_base.columns:
        feat_tr_base[c+'_sq'] = (feat_tr_base[c].astype(np.float32) ** 2).astype(np.float32)
    if c in feat_te_base.columns:
        feat_te_base[c+'_sq'] = (feat_te_base[c].astype(np.float32) ** 2).astype(np.float32)

# Merge Char edit features (already includes *_sq and one_minus_*)
feat_tr_base = feat_tr_base.merge(char_tr, on='id', how='left')
feat_te_base = feat_te_base.merge(char_te, on='id', how='left')

# Cheap extra features
feat_tr_base['tfidf_diag_fz'] = fisher_z(feat_tr_base['tfidf_diag'].values.astype(np.float32))
feat_te_base['tfidf_diag_fz'] = fisher_z(feat_te_base['tfidf_diag'].values.astype(np.float32))
if 'ce_raw' in feat_tr_base.columns:
    feat_tr_base['ce_raw_sq'] = (feat_tr_base['ce_raw'].astype(np.float32) ** 2).astype(np.float32)
    feat_te_base['ce_raw_sq'] = (feat_te_base['ce_raw'].astype(np.float32) ** 2).astype(np.float32)
    if 'len_diff' in feat_tr_base.columns:
        feat_tr_base['ce_raw_x_len'] = (feat_tr_base['ce_raw'].astype(np.float32) * feat_tr_base['len_diff'].astype(np.float32)).astype(np.float32)
        feat_te_base['ce_raw_x_len'] = (feat_te_base['ce_raw'].astype(np.float32) * feat_te_base['len_diff'].astype(np.float32)).astype(np.float32)
feat_tr_base['e5mp_x'] = (feat_tr_base['e5_asym_cos_k0'].astype(np.float32) * feat_tr_base['mpnet_asym_cos_k1'].astype(np.float32)).astype(np.float32)
feat_te_base['e5mp_x'] = (feat_te_base['e5_asym_cos_k0'].astype(np.float32) * feat_te_base['mpnet_asym_cos_k1'].astype(np.float32)).astype(np.float32)

y = feat_tr_base['score'].values.astype(np.float32)
fold_arr = feat_tr_base['fold'].values.astype(int)

# Build list of residual feature column names for train/test mapping (ALL variants)
res_cos_cols = []
res_l1_cols = []
for (a,k) in bge_res_candidates:
    cos_c = f'bge_res_a{a}_cos_k{k}'
    l1_c  = f'bge_res_a{a}_l1_k{k}'
    if cos_c in Xtr_res_all.columns and l1_c in Xtr_res_all.columns:
        res_cos_cols.append(cos_c)
        res_l1_cols.append(l1_c)

oof = np.zeros_like(y, dtype=np.float32)
test_preds = []

# LightGBM params (CPU) - Stronger set
lgb_params = dict(
    objective='regression', metric='rmse',
    learning_rate=0.02, num_leaves=127, min_data_in_leaf=64,
    feature_fraction=0.75, bagging_fraction=0.75, bagging_freq=1,
    lambda_l1=0.0, lambda_l2=1.0, verbosity=-1, seed=42, n_jobs=-1
)

for f in sorted(np.unique(fold_arr)):
    tr_idx = np.where(fold_arr != f)[0]
    va_idx = np.where(fold_arr == f)[0]
    tr_fold = feat_tr_base.copy(); te_fold = feat_te_base.copy()
    # Add ALL residual variant features for train (OOF columns) and test (fold-specific whitened columns)
    for cos_c, l1_c in zip(res_cos_cols, res_l1_cols):
        tr_fold[cos_c] = Xtr_res_all[cos_c].values.astype(np.float32)
        tr_fold[l1_c]  = Xtr_res_all[l1_c].values.astype(np.float32)
        te_fold[cos_c] = Xte_res_all[f'{cos_c}_tef{int(f)}'].values.astype(np.float32)
        te_fold[l1_c]  = Xte_res_all[f'{l1_c}_tef{int(f)}'].values.astype(np.float32)
    # Override bge_nocpc test with fold-specific whitened features
    if f'bge_nocpc_cos_k1_tef{int(f)}' in Xte_fixed_all.columns:
        te_fold['bge_nocpc_cos_k1'] = Xte_fixed_all[f'bge_nocpc_cos_k1_tef{int(f)}'].values.astype(np.float32)
    if f'bge_nocpc_l1_k1_tef{int(f)}' in Xte_fixed_all.columns:
        te_fold['bge_nocpc_l1_k1']  = Xte_fixed_all[f'bge_nocpc_l1_k1_tef{int(f)}'].values.astype(np.float32)
    # Nonlinearities on cosine-like channels: fixed + ALL residual cos + tfidf
    cos_cols_aug = fixed_cos_cols + res_cos_cols + ['tfidf_diag']
    tr_fold = augment_with_nonlinear(tr_fold, cos_cols_aug, len_col='len_diff', add_fz=True)
    te_fold = augment_with_nonlinear(te_fold, cos_cols_aug, len_col='len_diff', add_fz=True)
    # CE-only within-frame ranks
    ce_rank_cols = ['ce_raw'] + (['ce_iso'] if 'ce_iso' in tr_fold.columns else [])
    tr_fold = add_within_frame_anchor_ranks(tr_fold, ce_rank_cols)
    te_fold = add_within_frame_anchor_ranks(te_fold, ce_rank_cols)
    # Main cosine within-frame ranks (no mixing) + BM25 + IDF + RF + LCS + char-edit ranks
    rank_cols = fixed_cos_cols + res_cos_cols + [c for c in [
        'bm25_ab','bm25_ba','idf1_f1','idf2_f1','rf_token_set','rf_partial_token_set','lcs_char','lcs_tok','char3_jac','char4_jac','char5_jac',
        'jaro_winkler','levenshtein_norm','damerau_lev_norm'
    ] if c in tr_fold.columns]
    tr_fold = add_within_frame_anchor_ranks(tr_fold, rank_cols)
    te_fold = add_within_frame_anchor_ranks(te_fold, rank_cols)
    # Fold-local interaction: couple each residual cos with ce_raw (if present)
    if 'ce_raw' in tr_fold.columns:
        for cos_c in res_cos_cols:
            tr_fold[f'{cos_c}_x_ce_raw'] = (tr_fold[cos_c].astype(np.float32) * tr_fold['ce_raw'].astype(np.float32)).astype(np.float32)
            te_fold[f'{cos_c}_x_ce_raw'] = (te_fold[cos_c].astype(np.float32) * te_fold['ce_raw'].astype(np.float32)).astype(np.float32)
    # Feature selection
    drop_cols = {'id','score','fold','anchor','target'}
    use_cols = [c for c in tr_fold.columns if c not in drop_cols]
    Xtr = tr_fold.loc[tr_idx, use_cols].values.astype(np.float32)
    Xva = tr_fold.loc[va_idx, use_cols].values.astype(np.float32)
    Xte = te_fold[use_cols].values.astype(np.float32)
    dtr = lgb.Dataset(Xtr, label=y[tr_idx])
    dva = lgb.Dataset(Xva, label=y[va_idx], reference=dtr)
    mdl = lgb.train(lgb_params, dtr, num_boost_round=4000, valid_sets=[dva], valid_names=['va'], callbacks=[lgb.early_stopping(300, verbose=False), lgb.log_evaluation(200)])
    oof[va_idx] = mdl.predict(Xva).astype(np.float32)
    test_preds.append(mdl.predict(Xte).astype(np.float32))
    print(f'Fold {int(f)} LGBM (no-residual-selection): va_pearson=', round(float(pearsonr(oof[va_idx], y[va_idx])[0]), 6), flush=True)

p = pearsonr(oof, y)[0]
print('OOF Pearson LightGBM stack (NO per-fold residual selection; all variants as features):', round(p,6))
pd.DataFrame({'id': train['id'], 'oof': oof}).to_csv('oof_stack_lgbm.csv', index=False)

test_pred = np.mean(np.vstack(test_preds), axis=0).astype(np.float32)
sub = pd.DataFrame({'id': test['id'], 'score': np.clip(test_pred, 0.0, 1.0)})
sub.to_csv('submission_stack_lgbm.csv', index=False)
print('Saved submission_stack_lgbm.csv; elapsed', round((time.time()-t0)/60,2), 'min')
sub.to_csv('submission.csv', index=False)

[200]	va's rmse: 0.170318


[400]	va's rmse: 0.169801


[600]	va's rmse: 0.170146


Fold 0 LGBM (no-residual-selection): va_pearson= 0.763218


[200]	va's rmse: 0.170856


...

Fold 3 LGBM (no-residual-selection): va_pearson= 0.757311


[200]	va's rmse: 0.172998


[400]	va's rmse: 0.172751


Fold 4 LGBM (no-residual-selection): va_pearson= 0.746866


OOF Pearson LightGBM stack (NO per-fold residual selection; all variants as features): 0.752741
Saved submission_stack_lgbm.csv; elapsed 0.46 min


In [62]:
# Cross-Encoder (CPU): stsb-roberta-large with asymmetric CPC-on-anchor prompt; save RAW and isotonic; also overwrite CE artifacts used by stacker
import time, numpy as np, pandas as pd
from scipy.stats import pearsonr
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LinearRegression
from sentence_transformers import CrossEncoder

SEED = 42
np.random.seed(SEED)

# CPC mapping (4->3->2 title; RAW code fallback), truncate to 10 words
CPC_DESCRIPTIONS = {
    'A61B':'Diagnosis; Surgery; Identification','A61F':'Filters; Prostheses; Braces','A61K':'Preparations for medical, dental, or toilet purposes','A61M':'Devices for introducing/circulating media into the body',
    'B01D':'Separation; Sorption; Membranes','B29C':'Shaping or joining of plastics','B32B':'Layered products','B60K':'Arrangement or mounting of vehicle power plant or auxiliaries','B60R':'Vehicles, vehicle fittings, or vehicle parts','B60T':'Vehicle brake control systems or parts',
    'C01B':'Non-metallic elements; Compounds thereof','C04B':'Cement; Concrete; Ceramics','C07C':'Acyclic or carbocyclic compounds','C07D':'Heterocyclic compounds','C07K':'Peptides','C08F':'Macromolecular compounds by polymerising monomers','C08L':'Compositions of macromolecular compounds','C09K':'Materials for miscellaneous applications',
    'C12N':'Microorganisms or enzymes; Genetic engineering','C12P':'Fermentation or enzyme-using processes','C12Q':'Measuring/testing involving enzymes or microorganisms',
    'E04B':'Building constructions; Walls; Floors; Roofs','F16B':'Screws; Rivets; Joints','F16F':'Springs; Shock-absorbers; Dampers','F16L':'Pipes; Joints or fittings for pipes',
    'G01N':'Investigating or analysing materials','G02B':'Optical elements, systems, or apparatus','G02F':'Control of light','G06F':'Electric digital data processing','G06K':'Recognition/Presentation of data','G06Q':'Data processing for administrative/financial/managerial',
    'H01L':'Semiconductor devices','H01M':'Batteries; Fuel cells','H02J':'Power supply/distribution','H04L':'Transmission of digital information','H04N':'Pictorial communication','H04W':'Wireless communication networks','H05K':'Printed circuits; Electric assemblies',
    'A01':'Agriculture; Forestry; Animal husbandry; Hunting; Trapping; Fishing','A21':'Baking; Edible doughs','A23':'Foods or foodstuffs; Treatment thereof','A24':'Tobacco; Cigars; Cigarettes; Simulated smoking devices; Smokers\' requisites','A41':'Wearing apparel','A42':'Headwear','A43':'Footwear','A45':'Hand or travelling articles','A46':'Brushware','A47':'Furniture; Domestic articles or appliances; Coffee mills; Spice mills; Suction cleaners','A61':'Medical or veterinary science; Hygiene','A62':'Life-saving; Fire-fighting','A63':'Sports; Games; Amusements',
    'B01':'Physical or chemical processes or apparatus','B02':'Crushing; pulverising; disintegrating','B03':'Separation of solid materials using liquids or jigs','B04':'Centrifugal apparatus or machines','B05':'Spraying; atomising; Applying fluent materials to surfaces','B06':'Mechanical vibrations','B07':'Separating solids; Sorting','B08':'Cleaning','B09':'Disposal of solid waste; Soil reclamation','B21':'Mechanical metal-working without removing material','B22':'Casting; Powder metallurgy','B23':'Machine tools; Metal-working n.e.c.','B24':'Grinding; Polishing','B25':'Hand tools; Portable power-driven tools; Manipulators','B26':'Hand cutting tools; Cutting; Severing','B27':'Working/preserving wood; Nailing or stapling','B28':'Working cement, clay, or stone','B29':'Working of plastics; Plastic state substances','B30':'Presses','B31':'Making paper-like articles','B32':'Layered products','B33':'Additive manufacturing','B41':'Printing; Typewriters; Stamps','B42':'Bookbinding; Albums; Files','B43':'Writing or drawing implements; Bureau accessories','B44':'Decorative arts','B60':'Vehicles in general','B61':'Railways','B62':'Land vehicles not on rails','B63':'Ships or waterborne vessels','B64':'Aircraft; Aviation; Cosmonautics','B65':'Conveying; Packing; Storing','B66':'Hoisting; Lifting; Hauling','B67':'Opening/closing containers; Liquid handling','B68':'Saddlery; Upholstery','B81':'Microstructural technology','B82':'Nanotechnology',
    'C01':'Inorganic chemistry','C02':'Treatment of water, wastewater, sewage, sludge','C03':'Glass; Mineral or slag wool','C04':'Cements; Concrete; Ceramics; Refractories','C05':'Fertilisers','C06':'Explosives; Matches','C07':'Organic chemistry','C08':'Organic macromolecular compounds; Polymers','C09':'Dyes; Paints; Polishes; Adhesives; Misc compositions','C10':'Petroleum/gas/coke industries; Fuels; Lubricants; Peat','C11':'Oils, fats, waxes; Detergents; Candles','C12':'Biochemistry; Microbiology; Enzymology; Fermentation','C13':'Sugar industry','C14':'Skins; Hides; Leather',
    'C21':'Metallurgy of iron','C22':'Metallurgy; Alloys; Treatment of non-ferrous metals','C23':'Coating metallic material; Chemical surface treatment','C25':'Electrolytic/electrophoretic processes','C30':'Crystal growth','C40':'Combinatorial technology',
    'D01':'Threads or fibres; Spinning','D02':'Yarns; Finishing; Warping or beaming','D03':'Weaving','D04':'Braiding; Lace-making; Knitting; Non-wovens','D05':'Sewing; Embroidering; Tufting','D06':'Textile treatment; Laundering; Flexible materials','D07':'Ropes; Cables other than electric','D21':'Paper-making; Cellulose production',
    'E01':'Road/rail/bridge construction','E02':'Hydraulic engineering; Foundations; Soil shifting','E03':'Water supply; Sewerage','E04':'Building','E05':'Locks; Keys; Fittings; Safes','E06':'Doors; Windows; Shutters; Ladders','E21':'Earth/rock drilling; Mining',
    'F01':'Machines or engines in general','F02':'Combustion engines; Hot-gas plants','F03':'Machines or engines for liquids; Wind/spring motors','F04':'Positive-displacement machines; Pumps','F15':'Fluid-pressure actuators; Hydraulics; Pneumatics','F16':'Engineering elements or units','F17':'Storing/distributing gases or liquids','F21':'Lighting','F22':'Steam generation','F23':'Combustion apparatus; Processes','F24':'Heating; Ranges; Ventilating','F25':'Refrigeration or cooling; Heat pumps','F26':'Drying','F27':'Furnaces; Kilns; Ovens','F28':'Heat exchange in general','F41':'Weapons','F42':'Ammunition; Blasting',
    'G01':'Measuring; Testing','G02':'Optics','G03':'Photography; Cinematography; Non-optical waves','G04':'Horology','G05':'Controlling; Regulating','G06':'Computing; Calculating; Counting','G07':'Checking-devices','G08':'Signalling','G09':'Educating; Cryptography; Display; Advertising; Seals','G10':'Musical instruments; Acoustics','G11':'Information storage','G12':'Instrument details','G16':'ICT adapted for specific fields','G21':'Nuclear physics; Nuclear engineering',
    'H01':'Electric elements','H02':'Electric power: generation, conversion, distribution','H03':'Electronic circuitry','H04':'Electric communication technique','H05':'Electric techniques not otherwise provided for'
}

def get_cpc_text(code: str) -> str:
    if not isinstance(code, str) or not code:
        return 'GENERAL'
    code = code.strip().upper()
    for k in (4,3,2):
        pref = code[:k]
        if pref in CPC_DESCRIPTIONS:
            return CPC_DESCRIPTIONS[pref]
    return code  # RAW fallback

def truncate_words(text: str, n: int = 10) -> str:
    w = text.split() if isinstance(text, str) else []
    return ' '.join(w[:n])

def build_pairs(df: pd.DataFrame):
    cpc_full = df['context'].astype(str).map(get_cpc_text)
    cpc_10w = cpc_full.map(lambda x: truncate_words(x, 10))
    a_side = (cpc_10w + ' [SEP] ' + df['anchor'].astype(str)).tolist()
    b_side = df['target'].astype(str).tolist()
    return list(zip(a_side, b_side))

t0 = time.time()
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
folds = pd.read_csv('folds_by_id.csv')
train = train.merge(folds, on='id', how='left', validate='one_to_one')
assert (train['fold']>=0).all(), 'Fold merge by id failed'

train_pairs = build_pairs(train)
test_pairs = build_pairs(test)

model_name = 'cross-encoder/stsb-roberta-large'
ce = CrossEncoder(model_name, device='cpu', max_length=256, tokenizer_args={'use_fast': False})

print('Scoring train pairs (STSB-large CE, context-aware)...', flush=True)
train_scores_raw = np.asarray(ce.predict(train_pairs, batch_size=16, show_progress_bar=True), dtype=np.float32)
print('Scoring test pairs (STSB-large CE, context-aware)...', flush=True)
test_scores_raw = np.asarray(ce.predict(test_pairs, batch_size=16, show_progress_bar=True), dtype=np.float32)

# Per-fold isotonic vs linear calibration for diagnostics; save both, but raw will be primary for stacking
y = train['score'].values.astype(np.float32)
oof_iso = np.zeros_like(y, dtype=np.float32)
oof_lin = np.zeros_like(y, dtype=np.float32)
test_iso_f, test_lin_f = [], []
for f in sorted(train['fold'].unique()):
    tr_idx = np.where(train['fold'] != f)[0]
    va_idx = np.where(train['fold'] == f)[0]
    raw_p = pearsonr(train_scores_raw[va_idx], y[va_idx])[0]
    iso = IsotonicRegression(out_of_bounds='clip')
    iso.fit(train_scores_raw[tr_idx], y[tr_idx])
    oof_iso[va_idx] = iso.predict(train_scores_raw[va_idx]).astype(np.float32)
    test_iso_f.append(iso.predict(test_scores_raw).astype(np.float32))
    lr = LinearRegression()
    lr.fit(train_scores_raw[tr_idx].reshape(-1,1), y[tr_idx])
    oof_lin[va_idx] = lr.predict(train_scores_raw[va_idx].reshape(-1,1)).astype(np.float32)
    test_lin_f.append(lr.predict(test_scores_raw.reshape(-1,1)).astype(np.float32))
    print(f'STSB-large CE Fold {int(f)}: raw={raw_p:.6f} | iso={pearsonr(oof_iso[va_idx], y[va_idx])[0]:.6f} | lin={pearsonr(oof_lin[va_idx], y[va_idx])[0]:.6f}', flush=True)

p_raw = pearsonr(train_scores_raw, y)[0]
p_iso = pearsonr(oof_iso, y)[0]
p_lin = pearsonr(oof_lin, y)[0]
print('STSB-large CE OOF Pearson: raw=', round(float(p_raw),6), ' iso=', round(float(p_iso),6), ' lin=', round(float(p_lin),6))

# Save dedicated artifacts
pd.DataFrame({'id': train['id'], 'oof_raw': train_scores_raw, 'oof_iso': oof_iso}).to_csv('oof_ce_large.csv', index=False)
pd.DataFrame({'id': test['id'], 'score_raw': test_scores_raw, 'score_iso': np.mean(np.vstack(test_iso_f), axis=0).astype(np.float32)}).to_csv('submission_ce_large.csv', index=False)

# Overwrite the CE files used by current stackers with RAW CE-large scores to immediately lift CE channel
pd.DataFrame({'id': train['id'], 'oof': train_scores_raw}).to_csv('oof_ce_stsb.csv', index=False)
pd.DataFrame({'id': test['id'], 'score': test_scores_raw}).to_csv('submission_ce_stsb.csv', index=False)

# Also write iso to the file pattern read by Cell 23 for optional ce_iso feature
pd.DataFrame({'id': train['id'], 'oof_raw': train_scores_raw, 'oof_iso': oof_iso}).to_csv('oof_ce_l12.csv', index=False)
pd.DataFrame({'id': test['id'], 'score_raw': test_scores_raw, 'score_iso': np.mean(np.vstack(test_iso_f), axis=0).astype(np.float32)}).to_csv('submission_ce_l12.csv', index=False)

print('Saved STSB-large CE artifacts; overwrote oof_ce_stsb/submission_ce_stsb with RAW large; elapsed', round((time.time()-t0)/60,2), 'min')

Scoring train pairs (STSB-large CE, context-aware)...


Batches:   0%|          | 0/2052 [00:00<?, ?it/s]

Batches:   0%|          | 1/2052 [00:00<10:28,  3.26it/s]

Batches:   0%|          | 2/2052 [00:00<10:01,  3.41it/s]

Batches:   0%|          | 3/2052 [00:00<09:57,  3.43it/s]

...

STSB-large CE Fold 1: raw=0.541951 | iso=0.544716 | lin=0.541951


STSB-large CE Fold 2: raw=0.544334 | iso=0.546340 | lin=0.544334


STSB-large CE Fold 3: raw=0.563538 | iso=0.564940 | lin=0.563538


STSB-large CE Fold 4: raw=0.544073 | iso=0.545247 | lin=0.544073


STSB-large CE OOF Pearson: raw= 0.549902  iso= 0.551378  lin= 0.549777
Saved STSB-large CE artifacts; overwrote oof_ce_stsb/submission_ce_stsb with RAW large; elapsed 10.59 min
